In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install attacut transformers sentence-transformers torch pandas numpy scikit-learn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 670.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Cell 3: Import Libraries
import pandas as pd
import numpy as np
from attacut import tokenize
from sentence_transformers import SentenceTransformer
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cell 4: Load and Configure Model
model_name = "Qwen/Qwen3-Embedding-0.6B"
print(f"Loading model: {model_name}")
model = SentenceTransformer(model_name)
print("Model loaded successfully")

Loading model: Qwen/Qwen3-Embedding-0.6B


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Model loaded successfully


In [17]:
# Cell 5: Define 6C Framework
sixc_definitions = {
    'ความสามารถในการรุก': 'ความสามารถในการวางแผนและเดินหมากเพื่อ "หนีบ" กินหมากฝ่ายตรงข้าม เช่น การเล็งตำแหน่งที่สามารถล้อมฝ่ายตรงข้ามได้อย่างมีประสิทธิภาพ',
    'ความสามารถในการรับ': 'การจัดวางตำแหน่งหมากและเลือกจังหวะการเดินเพื่อป้องกันไม่ให้ถูกหนีบ รวมถึงการรักษาระยะห่างและการตั้งแนวป้องกัน',
    'การประสานงานระหว่างหมาก': 'การประสานงานระหว่างหมากของตัวเองหลายตัวเพื่อช่วยกันสร้างแนวหนีบหรือแนวป้องกันอย่างมีแบบแผน',
    'ทรัพยากรที่ยังไม่ใช้': 'จำนวนหมากที่ยังเหลืออยู่บนกระดาน และความพร้อมในการใช้หมากเหล่านั้นเพื่อสร้างความได้เปรียบในการรุกหรือรับ',
    'ความได้เปรียบด้านเวลา': 'การเลือกจังหวะการเดินหมากที่เหมาะสม เช่น เร่งจังหวะเมื่อมีโอกาส หรือถ่วงเวลาเพื่อรักษาตำแหน่งเมื่อใกล้ครบเวลา 15 นาที',
    'ความเหมาะสมกับสถานการณ์': 'การตัดสินใจเดินหมากโดยอิงตามสภาพกระดานปัจจุบัน เช่น ช่องว่าง ช่องอันตราย หรือการถูกควบคุม เพื่อให้การเดินสอดคล้องกับสถานการณ์จริง'
}

sixc_labels = list(sixc_definitions.keys())
print("6C Framework defined")

6C Framework defined


In [18]:
# Cell 6: Load Strategy Data
csv_path = '/content/drive/MyDrive/AlphaZero_Backups/data/36strategies-description.csv'
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} strategies")
df.head()

Loaded 36 strategies


,ชื่อกลยุทธ์,หมวด,ลักษณะ
0,ปิดฟ้าข้ามทะเล,กลยุทธ์ชนะศึก,ทำให้ศัตรูสบายใจ คิดว่าไม่มีแผน แล้วโจมตีทันที
1,ล้อมเวยช่วยจ้าว,กลยุทธ์ชนะศึก,บุกตีจุดสำคัญที่ศัตรูเผลอให้กำลังแตก
2,ยืมดาบฆ่าคน,กลยุทธ์ชนะศึก,ใช้กำลังหรือทรัพยากรของคนอื่นแทนตัวเอง
3,รอซ้ำยามเปลี้ย,กลยุทธ์ชนะศึก,รอให้ศัตรูเหนื่อยแล้วจึงลงมือ
4,ตีชิงตามไฟ,กลยุทธ์ชนะศึก,ฉวยโอกาสตอนศัตรูกำลังวุ่นวาย


In [19]:
# Cell 6.1: สร้างคอลัมน์ข้อความรวม (หมวด + ลักษณะ)
df['combined_text'] = df['หมวด'] + ": " + df['ลักษณะ']

print("\nตัวอย่างข้อความรวม:")
df[['ชื่อกลยุทธ์', 'combined_text']].head(2)



ตัวอย่างข้อความรวม:


,ชื่อกลยุทธ์,combined_text
0,ปิดฟ้าข้ามทะเล,กลยุทธ์ชนะศึก: ทำให้ศัตรูสบายใจ คิดว่าไม่มีแผน...
1,ล้อมเวยช่วยจ้าว,กลยุทธ์ชนะศึก: บุกตีจุดสำคัญที่ศัตรูเผลอให้กำล...


In [40]:
# Cell 7: Text Preprocessing with Attacut
def preprocess_text(text):
    if pd.isna(text):
        return ""
    tokens = tokenize(str(text))
    return ' '.join(tokens)


In [41]:
# Cell 8: Compute 6C Vector for Text (รองรับ row หรือ raw string)
def compute_6c_vector(input_):
    # ถ้าเป็น Series หรือ dict-like ให้ดึงค่าจาก 'combined_text'
    if isinstance(input_, (pd.Series, dict)):
        text = input_.get('combined_text', "")
    else:
        # สมมติเป็น raw string ก็นำตรงๆ
        text = input_
    if not text or pd.isna(text):
        return np.zeros(len(sixc_definitions))

    emb = model.encode([text])
    sims = cosine_similarity(emb, sixc_embeddings)[0]
    return sims / sims.sum()

sixc_texts = list(sixc_definitions.values())
sixc_embeddings = model.encode(sixc_texts)


In [42]:
# Cell 9: Compute 6C Patterns for All Strategies
def compute_all_strategy_patterns(df):
    strategy_patterns = {}
    for _, row in df.iterrows():
        name = row['ชื่อกลยุทธ์']
        strategy_patterns[name] = compute_6c_vector(row)
    return strategy_patterns

# เรียกใช้งาน
strategy_patterns = compute_all_strategy_patterns(df)

print("\n6C Patterns with categories:")
for name, pattern in list(strategy_patterns.items()):
    print(f"\n{name}:")
    for i, label in enumerate(sixc_labels):
        print(f"  {label}: {pattern[i]:.4f}")


6C Patterns with categories:

ปิดฟ้าข้ามทะเล:
  ความสามารถในการรุก: 0.2084
  ความสามารถในการรับ: 0.1840
  การประสานงานระหว่างหมาก: 0.1871
  ทรัพยากรที่ยังไม่ใช้: 0.1386
  ความได้เปรียบด้านเวลา: 0.1441
  ความเหมาะสมกับสถานการณ์: 0.1377

ล้อมเวยช่วยจ้าว:
  ความสามารถในการรุก: 0.2004
  ความสามารถในการรับ: 0.1756
  การประสานงานระหว่างหมาก: 0.1754
  ทรัพยากรที่ยังไม่ใช้: 0.1610
  ความได้เปรียบด้านเวลา: 0.1513
  ความเหมาะสมกับสถานการณ์: 0.1364

ยืมดาบฆ่าคน:
  ความสามารถในการรุก: 0.2142
  ความสามารถในการรับ: 0.1432
  การประสานงานระหว่างหมาก: 0.2249
  ทรัพยากรที่ยังไม่ใช้: 0.1731
  ความได้เปรียบด้านเวลา: 0.1177
  ความเหมาะสมกับสถานการณ์: 0.1269

รอซ้ำยามเปลี้ย:
  ความสามารถในการรุก: 0.1945
  ความสามารถในการรับ: 0.1620
  การประสานงานระหว่างหมาก: 0.1591
  ทรัพยากรที่ยังไม่ใช้: 0.1755
  ความได้เปรียบด้านเวลา: 0.1700
  ความเหมาะสมกับสถานการณ์: 0.1390

ตีชิงตามไฟ:
  ความสามารถในการรุก: 0.1937
  ความสามารถในการรับ: 0.1604
  การประสานงานระหว่างหมาก: 0.1645
  ทรัพยากรที่ยังไม่ใช้: 0.1516
  ความได้เปร

In [43]:
# Cell 10: Create Vector Database
print("Creating vector database...")
vector_db = {
    'strategies': {},
    'embeddings': {},
    'patterns': strategy_patterns,
    'sixc_definitions': sixc_definitions
}

for _, row in df.iterrows():
    name = row['ชื่อกลยุทธ์']
    text = row['ลักษณะ']
    category = row['หมวด']

    # Store strategy info
    vector_db['strategies'][name] = {
        'text': text,
        'category': category,
        'processed_text': preprocess_text(text)
    }

    # Store embeddings
    vector_db['embeddings'][name] = model.encode([text])[0]

print(f"Vector database created with {len(vector_db['strategies'])} strategies")

Creating vector database...
Vector database created with 36 strategies


In [44]:
# Cell 11: Save Vector Database
vector_db_path = '/content/drive/MyDrive/AlphaZero_Backups/vector_database'
os.makedirs(vector_db_path, exist_ok=True)

with open(f'{vector_db_path}/vector_database.pkl', 'wb') as f:
    pickle.dump(vector_db, f)

print(f"Vector database saved to {vector_db_path}")

Vector database saved to /content/drive/MyDrive/AlphaZero_Backups/vector_database


In [45]:
# Cell 12: Load Vector Database Function
def load_vector_database():
    vector_db_path = '/content/drive/MyDrive/AlphaZero_Backups/vector_database/vector_database.pkl'
    with open(vector_db_path, 'rb') as f:
        return pickle.load(f)


In [46]:
# Cell 13: Semantic RAG Search Function
def semantic_search(query, top_k=5):
    vector_db = load_vector_database()

    # Encode query
    query_embedding = model.encode([query])[0]

    # Calculate similarities
    similarities = {}
    for name, embedding in vector_db['embeddings'].items():
        sim = cosine_similarity([query_embedding], [embedding])[0][0]
        similarities[name] = sim

    # Sort by similarity
    sorted_results = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    return sorted_results[:top_k]


In [47]:
# Cell 14: Reranking Function (ปรับให้ส่ง text ตรงๆ เข้า compute_6c_vector)
def rerank_results(query, search_results, vector_db):
    # Compute 6C pattern for query (รองรับ string)
    query_6c = compute_6c_vector(query)

    reranked_results = []
    for strategy_name, similarity in search_results:
        strategy_6c = vector_db['patterns'][strategy_name]
        sixc_similarity = cosine_similarity(
            [query_6c], [strategy_6c]
        )[0][0]
        combined_score = (similarity * 0.6) + (sixc_similarity * 0.4)
        reranked_results.append(
            (strategy_name, combined_score, similarity, sixc_similarity)
        )

    reranked_results.sort(key=lambda x: x[1], reverse=True)
    return reranked_results

In [48]:
# Cell 15: Semantic RAG with Reranking
def semantic_rag_with_reranking(query, top_k=5):
    vector_db = load_vector_database()

    # Initial semantic search
    initial_results = semantic_search(query, top_k * 2)

    # Rerank results
    reranked_results = rerank_results(query, initial_results, vector_db)

    return reranked_results[:top_k]

In [235]:
# Cell 16: Calculate Metrics (ปรับให้ compute_6c_vector รับ string)
def calculate_metrics(query, results):
    query_6c = compute_6c_vector(query)
    vector_db = load_vector_database()

    metrics = {}
    for strategy_name, score, similarity, sixc_sim in results:
        strategy_6c = vector_db['patterns'][strategy_name]
        semantic_sim = similarity
        prob_dist = normalize([strategy_6c], norm='l1')[0]
        selection_score = score
        confidence = (semantic_sim * 0.4) + (sixc_sim * 0.3) + (selection_score * 0.3)

        # KL divergence
        eps = 1e-10
        q_norm = normalize([query_6c + eps], norm='l1')[0]
        s_norm = normalize([strategy_6c + eps], norm='l1')[0]
        kl_div = np.sum(q_norm * np.log(q_norm / s_norm))

        metrics[strategy_name] = {
            'semantic_similarity': semantic_sim,
            'probability_distribution': prob_dist,
            'selection_score': selection_score,
            'confidence_score': confidence,
            'kl_divergence': kl_div,
            'sixc_similarity': sixc_sim
        }
    return metrics

In [236]:
# Cell 17: Test Function
def test_strategy_recommendation(query):
    print(f"Query: {query}")
    print("="*50)

    # Get recommendations
    results = semantic_rag_with_reranking(query, top_k=5)

    # Calculate metrics
    metrics = calculate_metrics(query, results)

    # Display results
    for i, (strategy_name, score, similarity, sixc_sim) in enumerate(results, 1):
        print(f"\n{i}. {strategy_name}")
        print(f"   Score: {score:.4f}")
        print(f"   Confidence: {metrics[strategy_name]['confidence_score']:.4f}")
        print(f"   Similarity: {similarity:.4f}")

    return results, metrics

In [237]:
# Cell 18: Detailed Analysis Function
def detailed_analysis(query, strategy_name):
    vector_db = load_vector_database()

    print(f"Detailed Analysis for: {strategy_name}")
    print("="*50)

    # Strategy info
    strategy_info = vector_db['strategies'][strategy_name]
    print(f"Category: {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}")

    # 6C Analysis
    query_6c = compute_6c_vector(query)
    strategy_6c = vector_db['patterns'][strategy_name]

    print(f"\n6C Pattern Comparison:")
    print(f"{'Parameter':<25} {'Query':<10} {'Strategy':<10} {'Match':<10}")
    print("-"*55)

    for i, label in enumerate(sixc_labels):
        match_score = min(query_6c[i], strategy_6c[i]) / max(query_6c[i], strategy_6c[i], 1e-10)
        print(f"{label:<25} {query_6c[i]:<10.4f} {strategy_6c[i]:<10.4f} {match_score:<10.4f}")

# ปิดฟ้าข้ามทะเล

In [238]:
# Cell 18: query
test_query =  """
---
ในการเดินหมากหนีบตามแนวคิดที่มุ่งเน้นการทำให้ศัตรู**ไร้ซึ่งการรับรู้ถึงแผนการหรือภัยคุกคามที่กำลังก่อตัวขึ้นโดยสิ้นเชิง** ก่อนที่จะเปิดฉากโจมตีอย่างรวดเร็วและเด็ดขาด คุณต้องผสานการเล่นที่สุขุมเข้ากับการจู่โจมที่เหนือความคาดหมายอย่างถึงที่สุด โดยจะให้ความสำคัญกับปัจจัยหลักๆ ในกรอบ 6C ดังนี้: ความสามารถในการรุก, ความสามารถในการรับ, และการประสานงานระหว่างหมาก ซึ่งมีความสำคัญอย่างยิ่ง ตามมาด้วยทรัพยากรที่ยังไม่ใช้, ความได้เปรียบด้านเวลา และความเหมาะสมกับสถานการณ์
### 1. **ช่วงเริ่มต้น: ปลูกฝังความไร้เดียงสา – ทำให้ศัตรูไม่ตระหนักถึงแผนที่ซ่อนอยู่**
* **เน้นที่: ความสามารถในการรับ และการประสานงานระหว่างหมาก**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณจัดวางหมากในลักษณะที่**ปราศจากซึ่งสัญญาณบ่งบอกถึงเจตนาแอบแฝงใดๆ ที่จะรับรู้ได้** การเดินหมากเป็นไปอย่าง**เรียบง่าย ไร้การเคลื่อนไหวที่ดึงดูดความสนใจ** สร้างแนวป้องกันที่ดู**สงบและไม่คุกคาม**อย่างยิ่ง เพื่อทำให้คู่ต่อสู้**หลงคิดว่าสถานการณ์มั่นคงและไม่มีกลยุทธ์ใดๆ ซ่อนอยู่** หมากของคุณควรจัดเรียงอย่างเป็นระเบียบและ**กลมกลืนไปกับกระดาน**จนไม่ก่อให้เกิดแม้แต่ความสงสัยเล็กน้อย โดยใช้การประสานงานของหมากเพื่อเสริมความมั่นคงของแนวรับโดยไม่ทำให้เกิดความตื่นตัว
* **วัตถุประสงค์:** ทำให้ฝ่ายตรงข้าม**ไม่สามารถรับรู้ถึงการมีอยู่ของแผนการใดๆ** ที่คุณกำลังดำเนินอยู่ และมุ่งความสนใจไปที่การเล่นของตนเองโดยปราศจากความระแวงอย่างสิ้นเชิง
### 2. **ช่วงกลางเกม: ดำเนินการโดยไร้การตรวจจับ – สร้างกำลังรบในมิติที่มองไม่เห็น**
* **เน้นที่: ทรัพยากรที่ยังไม่ใช้ และการประสานงานระหว่างหมาก**
* **วิธีการเดินหมาก:** ในขณะที่ยังคงรักษารูปแบบที่ดูเหมือนไม่คุกคามไว้ ให้คุณเริ่ม**ดำเนินการเคลื่อนย้ายหมากสำคัญอย่างลับที่สุด จนไม่มีการเปลี่ยนแปลงใดๆ ปรากฏแก่สายตาของคู่ต่อสู้** ไปยังตำแหน่งเชิงยุทธศาสตร์ที่สามารถใช้เป็นจุดเริ่มต้นของการโจมตีที่ทรงพลังในอนาคต เป็น**การก่อร่างสร้างกำลังรบที่สำคัญโดยที่ฝ่ายตรงข้ามไม่สามารถรับรู้ถึงกระบวนการใดๆ** บนกระดานเลยแม้แต่น้อย เก็บหมากสำรองไว้ และค่อยๆ สร้างรูปแบบการประสานงานที่ซับซ้อนแต่ยังคงอยู่ในความลับ เพื่อให้พร้อมสำหรับการ "หนีบ" ครั้งใหญ่ การจัดทัพเพื่อการรุกดำเนินไปในมิติที่**อยู่นอกเหนือการเฝ้าระวัง**ของฝ่ายตรงข้ามโดยสิ้นเชิง
* **วัตถุประสงค์:** นี่คือการนำกำลังรบหลักไปยังจุดยุทธศาสตร์โดยที่ศัตรูไม่รู้ถึงการเคลื่อนไหวหรือการเปลี่ยนแปลงใดๆ บนกระดาน เป็นการสร้างเงื่อนไขสำหรับการโจมตีที่เหนือความคาดหมายจากจุดที่ไม่เคยมีข้อมูลหรือสัญญาณใดๆ
### 3. **ช่วงปลายเกม: จู่โจมอย่างสายฟ้าฟาด – การระเบิดพลังจากความไม่รู้**
* **เน้นที่: ความสามารถในการรุก และความได้เปรียบด้านเวลา**
* **วิธีการเดินหมาก:** เมื่อฝ่ายตรงข้ามกำลังประมาทที่สุด **เพราะไม่เคยรับรู้ถึงการก่อตัวของภัยนี้เลย** หรือเมื่อมีการเคลื่อนที่ที่เปิดช่องว่างสำคัญขึ้นมาโดยไม่รู้ตัว และคุณมองเห็น "จังหวะ" ที่เหมาะสมที่สุด **ให้เปิดฉากโจมตีอย่างรวดเร็ว เด็ดขาด และรุนแรงทันที ราวกับปรากฏขึ้นจากความว่างเปล่า** ใช้ "ความสามารถในการรุก" ที่เตรียมมาอย่างเต็มที่ ส่งหมากที่จัดเตรียมไว้เข้า "หนีบ" กินหมากของฝ่ายตรงข้ามในจุดสำคัญอย่างไม่คาดฝัน การโจมตีนี้ควรเกิดขึ้นอย่างฉับพลันและมาจากจุดที่ฝ่ายตรงข้าม**ไม่เคยมีข้อมูลหรือสัญญาณเตือนใดๆ** ทำให้พวกเขาไม่สามารถโต้ตอบหรือตั้งตัวได้ทัน ไม่ให้มีโอกาสฟื้นตัวจากการจู่โจม
* **วัตถุประสงค์:** ทำให้ฝ่ายตรงข้ามเสียกระบวนและสูญเสียการควบคุมกระดานอย่างรวดเร็ว ด้วยการโจมตีที่เกิดขึ้นเมื่อพวกเขาคาดไม่ถึง นำไปสู่ความได้เปรียบอย่างเด็ดขาดและชัยชนะในที่สุด
---
การเดินหมากตามแนวคิดนี้จึงเป็นการอาศัยการอำพรางเจตนาจนถึงขั้นที่ฝ่ายตรงข้ามไม่รับรู้ถึงการมีอยู่ของภัย การเตรียมการที่ลับลี้จนไม่มีใครสังเกตเห็น และการลงมืออย่างเด็ดขาดในเวลาที่เหมาะสมที่สุด เพื่อให้ได้เปรียบคู่ต่อสู้อย่างเหนือความคาดหมายและทำให้ตกอยู่ในสถานการณ์ที่สิ้นหวัง
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
ในการเดินหมากหนีบตามแนวคิดที่มุ่งเน้นการทำให้ศัตรู**ไร้ซึ่งการรับรู้ถึงแผนการหรือภัยคุกคามที่กำลังก่อตัวขึ้นโดยสิ้นเชิง** ก่อนที่จะเปิดฉากโจมตีอย่างรวดเร็วและเด็ดขาด คุณต้องผสานการเล่นที่สุขุมเข้ากับการจู่โจมที่เหนือความคาดหมายอย่างถึงที่สุด โดยจะให้ความสำคัญกับปัจจัยหลักๆ ในกรอบ 6C ดังนี้: ความสามารถในการรุก, ความสามารถในการรับ, และการประสานงานระหว่างหมาก ซึ่งมีความสำคัญอย่างยิ่ง ตามมาด้วยทรัพยากรที่ยังไม่ใช้, ความได้เปรียบด้านเวลา และความเหมาะสมกับสถานการณ์
### 1. **ช่วงเริ่มต้น: ปลูกฝังความไร้เดียงสา – ทำให้ศัตรูไม่ตระหนักถึงแผนที่ซ่อนอยู่**
* **เน้นที่: ความสามารถในการรับ และการประสานงานระหว่างหมาก**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณจัดวางหมากในลักษณะที่**ปราศจากซึ่งสัญญาณบ่งบอกถึงเจตนาแอบแฝงใดๆ ที่จะรับรู้ได้** การเดินหมากเป็นไปอย่าง**เรียบง่าย ไร้การเคลื่อนไหวที่ดึงดูดความสนใจ** สร้างแนวป้องกันที่ดู**สงบและไม่คุกคาม**อย่างยิ่ง เพื่อทำให้คู่ต่อสู้**หลงคิดว่าสถานการณ์มั่นคงและไม่มีกลยุทธ์ใดๆ ซ่อนอยู่** หมากของคุณควรจัดเรียงอย่างเป็นระเบียบและ**กลมกลืนไป

In [239]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ปิดฟ้าข้ามทะเล
Category:    กลยุทธ์ชนะศึก
Description: ทำให้ศัตรูสบายใจ คิดว่าไม่มีแผน แล้วโจมตีทันที

Metrics:
  Semantic Similarity:    0.5595
  6C Similarity:          0.9962
  Selection Score:        0.7342
  Confidence Score:       0.7429
  KL Divergence:          0.0038

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2185,0.2084,0.9536
ความสามารถในการรับ,0.1788,0.1840,0.9715
การประสานงานระหว่างหมาก,0.1574,0.1871,0.8411
ทรัพยากรที่ยังไม่ใช้,0.1426,0.1386,0.9724
ความได้เปรียบด้านเวลา,0.1495,0.1441,0.9639
ความเหมาะสมกับสถานการณ์,0.1532,0.1377,0.8987


# ล้อมเวยช่วยจ้าว

In [240]:
# Cell 18: Example Usage
test_query =  """
---
ในการเดินหมากหนีบให้สอดคล้องกับแนวคิดการ**บุกโจมตีจุดสำคัญที่ศัตรูเผลอ เพื่อให้กำลังของพวกเขาแตกกระจัดกระจาย** คุณต้องวางแผนการรุกและรับอย่างชาญฉลาด โดยไม่เข้าปะทะกับภัยคุกคามหลักโดยตรง แต่เลือกเป้าหมายที่สำคัญและเปราะบาง เพื่อดึงความสนใจและบีบให้ศัตรูต้องถอยกำลังมาป้องกัน
### 1. **ช่วงเริ่มต้น: ประเมินสถานการณ์ – ค้นหาจุดอ่อนที่ซ่อนอยู่และภัยคุกคามหลัก**
* **เน้นที่:** ความสามารถในการรับ และความเหมาะสมกับสถานการณ์
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณเน้นการจัดวางหมากเพื่อสร้างแนวป้องกันที่มั่นคงให้กับตนเอง พร้อมกับ**สังเกตและวิเคราะห์**การเดินหมากของฝ่ายตรงข้ามอย่างละเอียด เพื่อ**ระบุจุดศูนย์รวมกำลังหลักหรือภัยคุกคามที่ใหญ่ที่สุด**ของพวกเขา และที่สำคัญยิ่งกว่าคือการ**ค้นหาจุดอ่อนหรือหมากสำคัญที่เปราะบางของฝ่ายตรงข้าม**ที่หากถูกโจมตีจะสร้างความเสียหายอย่างใหญ่หลวงและบีบให้พวกเขาต้องตอบสนอง ในขณะเดียวกันก็หลีกเลี่ยงการปะทะโดยตรงกับกำลังหลักของศัตรู
* **วัตถุประสงค์:** เพื่อให้เข้าใจโครงสร้างของฝ่ายตรงข้ามและ **ระบุเป้าหมายทางอ้อม** ที่จะใช้ในการดึงความสนใจและทำลายสมดุลของพวกเขา
---
### 2. **ช่วงกลางเกม: โจมตีเป้าหมายรอง – บีบให้กำลังหลักของศัตรูแตกกระจาย**
* **เน้นที่:** ความสามารถในการรุก, ทรัพยากรที่ยังไม่ใช้, และการประสานงานระหว่างหมาก
* **วิธีการเดินหมาก:** เมื่อระบุจุดอ่อนที่เหมาะสมได้แล้ว และมั่นใจว่าแนวรับของตนเองยังคงมั่นคงอยู่ ให้คุณเริ่ม**เปิดฉากโจมตีที่จุดสำคัญและเปราะบางนั้นอย่างรวดเร็วและเด็ดขาด** ใช้หมากที่ยังไม่ถูกใช้งานและประสานงานระหว่างหมากของคุณ เพื่อสร้างแรงกดดันอย่างหนัก ณ จุดนั้น การโจมตีนี้มีเป้าหมายเพื่อ**บีบให้ฝ่ายตรงข้ามต้องดึงกำลังหมากที่ใช้ในการรุกหรือป้องกันจุดอื่น มาปกป้องจุดที่ถูกโจมตี** ซึ่งจะทำให้กำลังหลักของพวกเขา "แตกกระจัดกระจาย" หรือถูกลดทอนประสิทธิภาพลง
* **วัตถุประสงค์:** เพื่อสร้างสถานการณ์ที่ฝ่ายตรงข้ามตกอยู่ในสภาวะลำบาก ต้องตัดสินใจเลือกระหว่างการทิ้งเป้าหมายหลักเดิมกับการปกป้องจุดที่ถูกโจมตี ทำให้เสียจังหวะและเสียการควบคุม
---
### 3. **ช่วงปลายเกม: พิชิตชัย – ใช้ประโยชน์จากกำลังที่กระจัดกระจาย**
* **เน้นที่:** ความสามารถในการรุก และความได้เปรียบด้านเวลา
* **วิธีการเดินหมาก:** เมื่อฝ่ายตรงข้ามเริ่มดึงหมากจากแนวหน้าหรือจุดสำคัญเพื่อไปป้องกันจุดที่ถูกโจมตี ทำให้เกิดช่องว่างหรือจุดอ่อนในตำแหน่งเดิมของพวกเขา ให้คุณ**ใช้จังหวะนั้นเปิดฉากโจมตีในพื้นที่ที่อ่อนแอลงอย่างฉับพลันและรุนแรง** การโจมตีนี้อาจเป็นไปในทิศทางเดียวกับกำลังหลักเดิมของศัตรูที่ตอนนี้อ่อนแอลง หรือเป็นการหนีบกินหมากที่เคยถูกปกป้องอย่างแน่นหนาแต่ตอนนี้ถูกทอดทิ้ง
* **วัตถุประสงค์:** ใช้ประโยชน์จากความโกลาหลและความไม่สมดุลของกำลังหมากฝ่ายตรงข้าม เพื่อสร้างความได้เปรียบอย่างเด็ดขาดและนำไปสู่ชัยชนะในที่สุด
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
ในการเดินหมากหนีบให้สอดคล้องกับแนวคิดการ**บุกโจมตีจุดสำคัญที่ศัตรูเผลอ เพื่อให้กำลังของพวกเขาแตกกระจัดกระจาย** คุณต้องวางแผนการรุกและรับอย่างชาญฉลาด โดยไม่เข้าปะทะกับภัยคุกคามหลักโดยตรง แต่เลือกเป้าหมายที่สำคัญและเปราะบาง เพื่อดึงความสนใจและบีบให้ศัตรูต้องถอยกำลังมาป้องกัน
### 1. **ช่วงเริ่มต้น: ประเมินสถานการณ์ – ค้นหาจุดอ่อนที่ซ่อนอยู่และภัยคุกคามหลัก**
* **เน้นที่:** ความสามารถในการรับ และความเหมาะสมกับสถานการณ์
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณเน้นการจัดวางหมากเพื่อสร้างแนวป้องกันที่มั่นคงให้กับตนเอง พร้อมกับ**สังเกตและวิเคราะห์**การเดินหมากของฝ่ายตรงข้ามอย่างละเอียด เพื่อ**ระบุจุดศูนย์รวมกำลังหลักหรือภัยคุกคามที่ใหญ่ที่สุด**ของพวกเขา และที่สำคัญยิ่งกว่าคือการ**ค้นหาจุดอ่อนหรือหมากสำคัญที่เปราะบางของฝ่ายตรงข้าม**ที่หากถูกโจมตีจะสร้างความเสียหายอย่างใหญ่หลวงและบีบให้พวกเขาต้องตอบสนอง ในขณะเดียวกันก็หลีกเลี่ยงการปะทะโดยตรงกับกำลังหลักของศัตรู
* **วัตถุประสงค์:** เพื่อให้เข้าใจโครงสร้างของฝ่ายตรงข้ามและ **ระบุเป้าหมายทางอ้อม** ที่จะใช้ในการดึงความสนใจและทำ

In [241]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ล้อมเวยช่วยจ้าว
Category:    กลยุทธ์ชนะศึก
Description: บุกตีจุดสำคัญที่ศัตรูเผลอให้กำลังแตก

Metrics:
  Semantic Similarity:    0.5332
  6C Similarity:          0.9974
  Selection Score:        0.7189
  Confidence Score:       0.7282
  KL Divergence:          0.0026

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2103,0.2004,0.9527
ความสามารถในการรับ,0.1819,0.1756,0.9651
การประสานงานระหว่างหมาก,0.1560,0.1754,0.8895
ทรัพยากรที่ยังไม่ใช้,0.1474,0.1610,0.9157
ความได้เปรียบด้านเวลา,0.1558,0.1513,0.9709
ความเหมาะสมกับสถานการณ์,0.1485,0.1364,0.9183


# ยืมดาบฆ่าคน

In [242]:
# Cell 18: Example Usage
test_query =  """
---
แนวคิดหลัก **"ใช้กำลังหรือทรัพยากรของคนอื่นแทนตัวเอง"**
ในเกมหมากหนีบ หมายถึงการที่เราไม่เดินหมากรุก–รบด้วยหมากของเราเองโดยตรง แต่จะสร้างสถานการณ์ให้ฝ่ายตรงข้ามเป็นคน “ยื่นหมาก” มาให้เราได้เอาไปใช้หนีบกินเอง
## 1. ล่อให้ศัตรูเปิดดาบ
* **สถานการณ์:** ฝ่ายตรงข้ามพยายามรุกเข้ามาจับกลางกระดาน เราปล่อยให้เขาวางหมากรุกนั้นโดยไม่ขัด
* **ประโยชน์:** หมากรุกของเขากลายเป็น “ทรัพยากร” ที่อยู่ในตำแหน่งโดดเดี่ยว ไม่มีหมากเพื่อนช่วยป้องกัน
**ตัวอย่างการเดิน:**
1. เราวางหมากบริเวณขอบ แค่สร้างแนวรับแน่นหนา
2. ศัตรูเห็นช่อง กล้าส่งหมากเข้าสู่กลาง เพื่อขยายรุก
3. ตำแหน่งหมากกลางนั้นกลายเป็นจุดอ่อนที่เราสามารถใช้หนีบต่อไป
---
## 2. ชักให้อาวุธหันกลับ
* **สถานการณ์:** เมื่อศัตรูมีหมากรุกอยู่กลางกระดาน เราเลื่อนหมากเบา ๆ ครอบคลุมเส้นทางถอยของเขา
* **ประโยชน์:** ดันให้หมากรุกของเขากลับเข้ามาในแนวป้องกันของตัวเอง กลายเป็น “ดาบ” ที่เขาใช้แล้วสะดุด
**ตัวอย่างการเดิน:**
1. วางหมากเสริมขอบหรือแนวทแยงให้ชิดหมากของเขาเล็กน้อย
2. เขาเห็นว่าต้องถอยหลังก็ขยับหมากกลับเข้ามา
3. การขยับนั้นเข้าไปในกรอบที่เราสามารถแทรกกลางหนีบได้
---
## 3. หนีบกินด้วยกำลังของเขา
* **สถานการณ์:** ตำแหน่งหมากของฝ่ายตรงข้ามที่ถูกเราชักนำจนเข้ากรอบเรียบร้อย
* **ประโยชน์:** เราแทบไม่ต้องใช้หมากตัวเอง กดหนีบกินหมากเขา 2 ตัว หรือมากกว่าในเทิร์นเดียว
**ตัวอย่างการเดิน:**
1. วางหมากตัวที่สาม-สี่ บังขอบสองฝั่งของหมากเขา
2. ใช้หมากแทรกกลางให้ครบ 3 ฝ่าย (2 ฝั่งของเขา + 1 ฝั่งของเรา)
3. หนีบกินหมากเขาที่อยู่กลางกระดานทันที
---
## 4. เก็บ “เกราะ” ที่เขาวางทิ้ง
* **สถานการณ์:** เขาวางหมากป้องกันหรือเสริมแนวรับของตัวเองมากเกินไป
* **ประโยชน์:** จุดเสริมแนวรับนั้นเองกลายเป็น “ทรัพยากร” ที่เราสามารถซ้อนแผน และหนีบกินหมากป้องกันหลักได้
**ตัวอย่างการเดิน:**
1. เมื่อเขาซ้อนหมากป้องกัน 2 ชั้น ตรงแนวเดียวกัน เราอาจเดินหมากเลี่ยงไปทางอื่น
2. ศัตรูเห็นช่องสำรอง ก็กระจายหมากไปปิดช่อง
3. หมายถึงเราได้ตำแหน่งดี คืนทุน จากนั้นกลับมาใช้หมากหนีบกินในจุดเดิม
---
### สรุป
* **“ยืมดาบ”** = รอให้ฝ่ายตรงข้ามวางหมากรุกหรือป้องกันจนกลายเป็นช่องโหว่
* **“ฆ่าคน”** = ใช้หมากไม่กี่ตัวของเรา แทรก–หนีบกินหมากระหว่างสองฝั่งที่เขาวางไว้เอง
* **ประหยัดกำลัง** = แทบไม่เสียหมากรุกของตัวเอง แต่ได้กินหมากของเขาเป็นจำนวนมาก
ด้วยวิธีนี้ เราจึง **ใช้ “ทรัพยากร” (หมาก) ของศัตรู** มาเป็นเครื่องมือในการชนะ แทนที่จะลงทุนกำลังตัวเองโดยตรง
ซึ่งตรงกับ"ใช้กำลังหรือทรัพยากรของคนอื่นแทนตัวเอง"
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
แนวคิดหลัก **"ใช้กำลังหรือทรัพยากรของคนอื่นแทนตัวเอง"** 
ในเกมหมากหนีบ หมายถึงการที่เราไม่เดินหมากรุก–รบด้วยหมากของเราเองโดยตรง แต่จะสร้างสถานการณ์ให้ฝ่ายตรงข้ามเป็นคน “ยื่นหมาก” มาให้เราได้เอาไปใช้หนีบกินเอง
## 1. ล่อให้ศัตรูเปิดดาบ
* **สถานการณ์:** ฝ่ายตรงข้ามพยายามรุกเข้ามาจับกลางกระดาน เราปล่อยให้เขาวางหมากรุกนั้นโดยไม่ขัด
* **ประโยชน์:** หมากรุกของเขากลายเป็น “ทรัพยากร” ที่อยู่ในตำแหน่งโดดเดี่ยว ไม่มีหมากเพื่อนช่วยป้องกัน
**ตัวอย่างการเดิน:**
1. เราวางหมากบริเวณขอบ แค่สร้างแนวรับแน่นหนา
2. ศัตรูเห็นช่อง กล้าส่งหมากเข้าสู่กลาง เพื่อขยายรุก
3. ตำแหน่งหมากกลางนั้นกลายเป็นจุดอ่อนที่เราสามารถใช้หนีบต่อไป
---
## 2. ชักให้อาวุธหันกลับ
* **สถานการณ์:** เมื่อศัตรูมีหมากรุกอยู่กลางกระดาน เราเลื่อนหมากเบา ๆ ครอบคลุมเส้นทางถอยของเขา
* **ประโยชน์:** ดันให้หมากรุกของเขากลับเข้ามาในแนวป้องกันของตัวเอง กลายเป็น “ดาบ” ที่เขาใช้แล้วสะดุด
**ตัวอย่างการเดิน:**
1. วางหมากเสริมขอบหรือแนวทแยงให้ชิดหมากของเขาเล็กน้อย
2. เขาเห็นว่าต้องถอยหลังก็ขยับหมากกลับเข้ามา
3. การขยับนั้นเข้าไปในกรอบที่เร

In [243]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ยืมดาบฆ่าคน
Category:    กลยุทธ์ชนะศึก
Description: ใช้กำลังหรือทรัพยากรของคนอื่นแทนตัวเอง

Metrics:
  Semantic Similarity:    0.5256
  6C Similarity:          0.9902
  Selection Score:        0.7115
  Confidence Score:       0.7208
  KL Divergence:          0.0094

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2199,0.2142,0.9741
ความสามารถในการรับ,0.1702,0.1432,0.8414
การประสานงานระหว่างหมาก,0.1773,0.2249,0.7884
ทรัพยากรที่ยังไม่ใช้,0.1630,0.1731,0.9418
ความได้เปรียบด้านเวลา,0.1295,0.1177,0.9085
ความเหมาะสมกับสถานการณ์,0.1400,0.1269,0.9062


# รอซ้ำยามเปลี้ย

In [244]:
# Cell 18: Example Usage
test_query =  """
อดทนตั้งรับและใช้กลอุบายเพื่อบีบให้ศัตรูใช้กำลังหรือทรัพยากรมากเกินไปจนเหนื่อยล้า** จากนั้นเมื่อศัตรูอ่อนแรงหรือกำลังสำคัญเริ่มร่อยหรอ คุณจะ**ฉวยโอกาสเข้า "หนีบ" กินหมากอย่างเด็ดขาด** สร้างความได้เปรียบที่นำไปสู่ชัยชนะ
---
กลยุทธ์นี้มุ่งเน้นการใช้ความอดทนและรอคอยจังหวะที่เหมาะสม เพื่อโจมตีเมื่อศัตรูอ่อนแอที่สุด
### 1. **ต้นเกม: "บีบให้เปลืองแรง" – ตั้งรับอย่างอดทน**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้น ให้คุณ**ตั้งรับแนวหมากของคุณให้มั่นคงและแข็งแกร่ง** หลีกเลี่ยงการเปิดเกมรุกใหญ่หรือการปะทะโดยตรง แต่ให้**ใช้การขยับหมากเพียงเล็กน้อยเพื่อ "ชิมเชิง"** หรือสร้างแรงกดดันที่ไม่รุนแรง สิ่งนี้จะบีบให้คู่ต่อสู้ต้องคิดและเคลื่อนหมากบ่อยครั้ง ทำให้พวกเขาใช้พลังงานและทรัพยากร (เช่น หมากสำรอง) มากเกินความจำเป็น
* **วัตถุประสงค์:** เพื่อ **ทำให้ศัตรูใช้กำลังกายและใจอย่างเปลืองเปล่า** โดยที่คุณยังคงรักษากำลังหลักไว้ได้อย่างครบถ้วน
### 2. **ช่วงกลางเกม: "ดึงเวลาและสะสมกำลัง" – ชะลอจังหวะและเตรียมพร้อม**
* **วิธีการเดินหมาก:** เมื่อศัตรูเริ่มแสดงสัญญาณของความเหนื่อยล้าหรือใช้หมากสำคัญไปมากแล้ว **ให้คุณเดินหมากอย่างชะลอจังหวะ** เช่น เลื่อนหมากเล็กน้อยเพื่อปรับตำแหน่ง หรือรวบรวมหมากสำรองไว้ใกล้กันเพื่อเสริมความแข็งแกร่ง คุณอาจใช้การบีบเวลาด้วยวิธีที่ไม่ตรงไปตรงมา เพื่อกระตุ้นให้ศัตรูรีบใช้หมากที่เหลืออยู่จนใกล้หมด
* **วัตถุประสงค์:** เพื่อ **เร่งให้ศัตรูอ่อนแรงลงไปอีก** และ **สะสมกำลัง** ของคุณให้พร้อมสำหรับการโจมตีครั้งสุดท้าย
### 3. **ช่วงปลายเกม: "ซ้ำเติมยามเปลี้ย" – "หนีบ" กินอย่างเด็ดขาด**
* **วิธีการเดินหมาก:** เมื่อเห็นช่องว่างที่ชัดเจนจากการที่ศัตรูยืดหมากออกไปไกล แนวรับอ่อนแอลง หรือใช้หมากสำรองไปเกือบหมดจนเหลือเพียงกำลังที่อ่อนล้า **ให้คุณเปิดฉากโจมตีด้วยการ "หนีบ" อย่างรวดเร็ว รุนแรง และประสานงานกัน** ส่งหมากแนวหลังที่เก็บไว้เข้ามาเสริมแนวหน้า เข้าปิดล้อมและกินหมากสำคัญของศัตรูจำนวนมากในเทิร์นเดียว เพื่อจบเกมอย่างรวดเร็วและเด็ดขาด
* **วัตถุประสงค์:** เพื่อใช้ความเหนื่อยล้าและความเสียเปรียบของศัตรูให้เป็นประโยชน์สูงสุดในการหนีบกินหมากสำคัญและสร้างความได้เปรียบที่ไม่อาจโต้ตอบได้ นำไปสู่ชัยชนะ
---
### **สอดคล้องกับ "กลยุทธ์ชนะศึก"
* **การเอาชนะด้วยความเหนือกว่าด้านพลังงานและทรัพยากร:** เป็นกลยุทธ์ที่มุ่งเน้นการบีบให้ศัตรูอ่อนล้าและใช้ทรัพยากรหมดไปก่อนคุณ ทำให้คุณมีกำลังที่สดใหม่กว่าในการโจมตีขั้นสุดท้าย ซึ่งเป็นปัจจัยสำคัญในการคว้าชัยชนะ
* **การรุกที่เด็ดขาดในจังหวะที่เหมาะสม:** กลยุทธ์นี้ไม่ได้เน้นการปะทะแต่แรก แต่เป็นการ **รอคอยอย่างอดทนเพื่อจังหวะที่ศัตรูอ่อนแอที่สุด** จากนั้นจึงเปิดฉากโจมตีอย่างรุนแรงและเด็ดขาด เพื่อสร้างความเสียหายสูงสุด
* **สร้างชัยชนะที่ยั่งยืน:** เป้าหมายคือการได้มาซึ่งหมากสำคัญหรือตำแหน่งเชิงกลยุทธ์ที่เหนือกว่าอย่างชัดเจนและถาวร โดยใช้ความอ่อนล้าของศัตรูเป็นช่องทาง ทำให้คุณสามารถควบคุมเกมและมุ่งสู่ชัยชนะได้อย่างมั่นคง
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
อดทนตั้งรับและใช้กลอุบายเพื่อบีบให้ศัตรูใช้กำลังหรือทรัพยากรมากเกินไปจนเหนื่อยล้า** จากนั้นเมื่อศัตรูอ่อนแรงหรือกำลังสำคัญเริ่มร่อยหรอ คุณจะ**ฉวยโอกาสเข้า "หนีบ" กินหมากอย่างเด็ดขาด** สร้างความได้เปรียบที่นำไปสู่ชัยชนะ
---
กลยุทธ์นี้มุ่งเน้นการใช้ความอดทนและรอคอยจังหวะที่เหมาะสม เพื่อโจมตีเมื่อศัตรูอ่อนแอที่สุด
### 1. **ต้นเกม: "บีบให้เปลืองแรง" – ตั้งรับอย่างอดทน**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้น ให้คุณ**ตั้งรับแนวหมากของคุณให้มั่นคงและแข็งแกร่ง** หลีกเลี่ยงการเปิดเกมรุกใหญ่หรือการปะทะโดยตรง แต่ให้**ใช้การขยับหมากเพียงเล็กน้อยเพื่อ "ชิมเชิง"** หรือสร้างแรงกดดันที่ไม่รุนแรง สิ่งนี้จะบีบให้คู่ต่อสู้ต้องคิดและเคลื่อนหมากบ่อยครั้ง ทำให้พวกเขาใช้พลังงานและทรัพยากร (เช่น หมากสำรอง) มากเกินความจำเป็น
* **วัตถุประสงค์:** เพื่อ **ทำให้ศัตรูใช้กำลังกายและใจอย่างเปลืองเปล่า** โดยที่คุณยังคงรักษากำลังหลักไว้ได้อย่างครบถ้วน
### 2. **ช่วงกลางเกม: "ดึงเวลาและสะสมกำลัง" – ชะลอจังหวะและเตรียมพร้อม**
* **วิธีการเดินหมาก:** เมื่อศัตรูเริ่มแสดงสัญญาณของความเหนื่อยล้าหรือใช้หมากสำคัญไปมากแล้

In [245]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: รอซ้ำยามเปลี้ย
Category:    กลยุทธ์ชนะศึก
Description: รอให้ศัตรูเหนื่อยแล้วจึงลงมือ

Metrics:
  Semantic Similarity:    0.5096
  6C Similarity:          0.9988
  Selection Score:        0.7053
  Confidence Score:       0.7151
  KL Divergence:          0.0012

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1989,0.1945,0.9780
ความสามารถในการรับ,0.1748,0.1620,0.9264
การประสานงานระหว่างหมาก,0.1603,0.1591,0.9925
ทรัพยากรที่ยังไม่ใช้,0.1635,0.1755,0.9318
ความได้เปรียบด้านเวลา,0.1613,0.1700,0.9485
ความเหมาะสมกับสถานการณ์,0.1412,0.1390,0.9839


# ตีชิงตามไฟ

In [246]:
# Cell 18: Example Usage
test_query =  """
**ฉวยโอกาสศัตรูวุ่นวาย → โจมตีจุดอ่อนทันที
#### 1. **ต้นเกม – เตรียมกำลังรอจังหวะ (สังเกตความวุ่น)**
* **จัดแนวรับแบบเคลื่อนที่ได้** ในพื้นที่กลางกระดาน
* **สังเกตการเดินหมากศัตรู** ที่สับสนหรือซ้ำซ้อน
* **เก็บหมากสำรอง 2-3 ตัว** ใกล้จุดยุทธศาสตร์
#### 2. **กลางเกม – กระตุ้นความวุ่นวาย (เร่งให้ศัตรูสับสน)**
* **สร้างแรงกดดันหลายจุดพร้อมกัน**:
  - ขยับหมากคู่ขนาน 2 ทาง (ซ้าย+ขวา)
  - เปิดช่องว่างเสี่ยงล่อให้ศัตรูตัดสินใจผิด
* **เน้นการเคลื่อนไหวเร็ว** ให้ศัตรูตามไม่ทัน
#### 3. **ปลายเกม – ฉกหนีบทันที (จังหวะทอง)**
* **โฟกัสจุดที่ศัตรูจัดหมากผิดพลาด**:
  - หมากโดดเดี่ยว
  - ช่องว่างระหว่างหมากเกิน 2 ช่อง
* **ส่งหมากสำรองหนีบทันที 3 ทาง**:
  - บน-ล่าง + ซ้าย-ขวา + แนวทแยง
* **กินหมากศัตรู ≥ 2 ตัวในเทิร์นเดียว**
> ศัตรูที่กำลังจัดหมากใหม่มักเปิดช่องโหว่ร้ายแรง → จู่โจมก่อนพวกเขาตั้งตัวได้!
---
###ความสอดคล้อง "กลยุทธ์ชนะศึก"
- **ชนะด้วยจังหวะเวลา**: โจมตีเมื่อศัตรูอ่อนแอที่สุด
- **แปลงความวุ่นวายเป็นโอกาส**: ใช้ข้อผิดพลาดศัตรูสร้างผลลัพธ์ทวีคูณ
- **การตัดสินใจฉับไว**: กล้าโจมตีเมื่อเห็นช่องว่างชั่วคราว
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
**ฉวยโอกาสศัตรูวุ่นวาย → โจมตีจุดอ่อนทันที
#### 1. **ต้นเกม – เตรียมกำลังรอจังหวะ (สังเกตความวุ่น)**  
* **จัดแนวรับแบบเคลื่อนที่ได้** ในพื้นที่กลางกระดาน  
* **สังเกตการเดินหมากศัตรู** ที่สับสนหรือซ้ำซ้อน  
* **เก็บหมากสำรอง 2-3 ตัว** ใกล้จุดยุทธศาสตร์  
#### 2. **กลางเกม – กระตุ้นความวุ่นวาย (เร่งให้ศัตรูสับสน)**  
* **สร้างแรงกดดันหลายจุดพร้อมกัน**:  
  - ขยับหมากคู่ขนาน 2 ทาง (ซ้าย+ขวา)  
  - เปิดช่องว่างเสี่ยงล่อให้ศัตรูตัดสินใจผิด  
* **เน้นการเคลื่อนไหวเร็ว** ให้ศัตรูตามไม่ทัน  
#### 3. **ปลายเกม – ฉกหนีบทันที (จังหวะทอง)**  
* **โฟกัสจุดที่ศัตรูจัดหมากผิดพลาด**:  
  - หมากโดดเดี่ยว  
  - ช่องว่างระหว่างหมากเกิน 2 ช่อง  
* **ส่งหมากสำรองหนีบทันที 3 ทาง**:  
  - บน-ล่าง + ซ้าย-ขวา + แนวทแยง  
* **กินหมากศัตรู ≥ 2 ตัวในเทิร์นเดียว**  
> ศัตรูที่กำลังจัดหมากใหม่มักเปิดช่องโหว่ร้ายแรง → จู่โจมก่อนพวกเขาตั้งตัวได้!  
---
###ความสอดคล้อง "กลยุทธ์ชนะศึก"  
- **ชนะด้วยจังหวะเวลา**: โจมตีเมื่อศัตรูอ่อนแอที่สุด  
- **แปลงความวุ่นวายเป็นโอกาส**: ใช้ข้อผิดพลาดศัตรูสร้างผลลัพธ์ทวีคูณ

In [247]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ตีชิงตามไฟ
Category:    กลยุทธ์ชนะศึก
Description: ฉวยโอกาสตอนศัตรูกำลังวุ่นวาย

Metrics:
  Semantic Similarity:    0.5575
  6C Similarity:          0.9972
  Selection Score:        0.7334
  Confidence Score:       0.7422
  KL Divergence:          0.0029

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1839,0.1937,0.9494
ความสามารถในการรับ,0.1854,0.1604,0.8653
การประสานงานระหว่างหมาก,0.1527,0.1645,0.9279
ทรัพยากรที่ยังไม่ใช้,0.1429,0.1516,0.9423
ความได้เปรียบด้านเวลา,0.1773,0.1728,0.9747
ความเหมาะสมกับสถานการณ์,0.1580,0.1570,0.9939


# ส่งเสียงบูรพาฝ่าตีประจิม

In [248]:
# Cell 18: Example Usage
test_query =  """
---
## 1. ต้นเกม – สร้างความคึกคักทางฝั่งตะวันออก (ล่อ)
* **ขยับหมากไปฝั่งตะวันออก** (เช่น แถวขวา) อย่างต่อเนื่องเล็กน้อย
* **วางหมากกลุ่มเล็ก ๆ** ให้ดูเหมือนคุณเตรียมบุกหนักที่นั่น
* **ผลลัพธ์:** ศัตรูสะสมหมากและตั้งแนวป้องกันฝั่งตะวันออกจนแน่น
---
## 2. กลางเกม – รักษาการล่อ (ยืนยันเป้าหมาย)
* **สลับหมากเล็ก ๆ** ระหว่างช่องฝั่งตะวันออก เพื่อย้ำว่าคุณจะบุกจุดนั้น
* **ไม่ส่งหมากลึกหรือเสริมทัพมาก** ให้ดูเหมือนคุณไม่มีหมากพร้อมบุกจริงจัง
* **ผลลัพธ์:** ศัตรูยึดฝั่งตะวันออกแน่นจนไม่เหลือช่องอ่อน
---
## 3. ปลายเกม – หักมุมไปฝั่งตะวันตก (โจมตี)
* **ฉวยโอกาสเดินหมากหลักจากฝั่งตะวันตก** เข้ามาประสานแนวหน้า–หลัง
* **ประกบสองฝั่งรอบช่องอ่อน** ที่ศัตรูละทิ้งเมื่อยกกำลังไปป้องกันฝั่งตะวันออก
* **หนีบ พร้อมกันในเทิร์นเดียว** กินหมากฝั่งตะวันตกที่ถูกทิ้งว่างไว้
---
### สอดคล้องกับ “กลยุทธ์ชนะศึก”
* ใช้ **การล่อ** (ส่งเสียงบูรพา) ให้ศัตรูตั้งรับฝั่งหนึ่ง
* เมื่อศัตรู **รวมกำลัง** ไปป้องกันจุดนั้น ก็ **หักมุม** บุกอีกด้านทันที
* เป็นการ **สร้างสมดุลกำลัง** แล้วโฟกัสแรงทั้งหมดไปยังจุดอ่อนที่ศัตรูเพิกเฉย
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
## 1. ต้นเกม – สร้างความคึกคักทางฝั่งตะวันออก (ล่อ)
* **ขยับหมากไปฝั่งตะวันออก** (เช่น แถวขวา) อย่างต่อเนื่องเล็กน้อย
* **วางหมากกลุ่มเล็ก ๆ** ให้ดูเหมือนคุณเตรียมบุกหนักที่นั่น
* **ผลลัพธ์:** ศัตรูสะสมหมากและตั้งแนวป้องกันฝั่งตะวันออกจนแน่น
---
## 2. กลางเกม – รักษาการล่อ (ยืนยันเป้าหมาย)
* **สลับหมากเล็ก ๆ** ระหว่างช่องฝั่งตะวันออก เพื่อย้ำว่าคุณจะบุกจุดนั้น
* **ไม่ส่งหมากลึกหรือเสริมทัพมาก** ให้ดูเหมือนคุณไม่มีหมากพร้อมบุกจริงจัง
* **ผลลัพธ์:** ศัตรูยึดฝั่งตะวันออกแน่นจนไม่เหลือช่องอ่อน
---
## 3. ปลายเกม – หักมุมไปฝั่งตะวันตก (โจมตี)
* **ฉวยโอกาสเดินหมากหลักจากฝั่งตะวันตก** เข้ามาประสานแนวหน้า–หลัง
* **ประกบสองฝั่งรอบช่องอ่อน** ที่ศัตรูละทิ้งเมื่อยกกำลังไปป้องกันฝั่งตะวันออก
* **หนีบ พร้อมกันในเทิร์นเดียว** กินหมากฝั่งตะวันตกที่ถูกทิ้งว่างไว้
---
### สอดคล้องกับ “กลยุทธ์ชนะศึก”
* ใช้ **การล่อ** (ส่งเสียงบูรพา) ให้ศัตรูตั้งรับฝั่งหนึ่ง
* เมื่อศัตรู **รวมกำลัง** ไปป้องกันจุดนั้น ก็ **หักมุม** บุกอีกด้านทันที
* เป็นการ **สร้างสมดุลกำลัง** แล้วโฟกัสแรงทั้งหมดไปยังจุดอ่อนที่ศ

In [249]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ส่งเสียงบูรพาฝ่าตีประจิม
Category:    กลยุทธ์ชนะศึก
Description: ล่อให้ศัตรูสนใจด้านหนึ่ง แล้วโจมตีอีกด้าน

Metrics:
  Semantic Similarity:    0.5271
  6C Similarity:          0.9972
  Selection Score:        0.7152
  Confidence Score:       0.7246
  KL Divergence:          0.0028

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2096,0.2114,0.9915
ความสามารถในการรับ,0.1831,0.1692,0.9242
การประสานงานระหว่างหมาก,0.1732,0.1944,0.8912
ทรัพยากรที่ยังไม่ใช้,0.1428,0.1532,0.9324
ความได้เปรียบด้านเวลา,0.1471,0.1357,0.9229
ความเหมาะสมกับสถานการณ์,0.1441,0.1360,0.9439


# มีในไม่มี

In [250]:
# Cell 18: Example Usage
test_query =  """
คุณจะใช้ **การสร้างภาพลวง** ให้คู่ต่อสู้โฟกัสผิดจุด จากนั้น **หักมุมบุก** ในตำแหน่งที่ทิ้งว่างไว้
### 1. **ต้นเกม – สร้างภาพลวงที่จุด A**
* **วางหมากกลุ่มเล็ก ๆ ที่จุด A** ให้ดูเหมือนคุณตั้งใจบุกตรงนั้น
* **ขยับหมากบริเวณ A นิดหน่อย** พอให้ศัตรูต้องส่งหมากมาปิดช่อง ไม่ต้องตั้งใจกินจริง
* **อย่าส่งสัญญาณบุกหนัก** ให้เขาเชื่อว่าจุด A คือเป้าหมายหลัก
### 2. **กลางเกม – กระตุ้นความเชื่อด้วยการเคลื่อนไหวซ้ำ**
* **สลับหมากระหว่าง A กับจุด A′ ใกล้กัน** เพื่อย้ำว่า A สำคัญเสมอ
* **ใช้หมากสำรองแวะเวียนผ่าน A** สร้างความมั่นใจในเป้าหมายเดียว
* **รักษารูปแบบแผนลวง** ไม่ให้มีการเคลื่อนไหวผิดปกติที่บ่งชี้แผนจริง
### 3. **ปลายเกม – เปิดโปงแผนที่จุด B**
* เมื่อศัตรู **รวมกำลังมาปิด A หมด** ให้รีบหักมุมบุกที่ **จุด B**
* **ส่งหมากแนวหลัง** เข้าเส้นทาง B ที่ขาดการป้องกัน
* **ประสานหน้าหลังเพื่อหนีบกิน** ในช่อง B ให้ได้มากที่สุดในเทิร์นเดียว
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
คุณจะใช้ **การสร้างภาพลวง** ให้คู่ต่อสู้โฟกัสผิดจุด จากนั้น **หักมุมบุก** ในตำแหน่งที่ทิ้งว่างไว้
### 1. **ต้นเกม – สร้างภาพลวงที่จุด A**
* **วางหมากกลุ่มเล็ก ๆ ที่จุด A** ให้ดูเหมือนคุณตั้งใจบุกตรงนั้น
* **ขยับหมากบริเวณ A นิดหน่อย** พอให้ศัตรูต้องส่งหมากมาปิดช่อง ไม่ต้องตั้งใจกินจริง
* **อย่าส่งสัญญาณบุกหนัก** ให้เขาเชื่อว่าจุด A คือเป้าหมายหลัก
### 2. **กลางเกม – กระตุ้นความเชื่อด้วยการเคลื่อนไหวซ้ำ**
* **สลับหมากระหว่าง A กับจุด A′ ใกล้กัน** เพื่อย้ำว่า A สำคัญเสมอ
* **ใช้หมากสำรองแวะเวียนผ่าน A** สร้างความมั่นใจในเป้าหมายเดียว
* **รักษารูปแบบแผนลวง** ไม่ให้มีการเคลื่อนไหวผิดปกติที่บ่งชี้แผนจริง
### 3. **ปลายเกม – เปิดโปงแผนที่จุด B**
* เมื่อศัตรู **รวมกำลังมาปิด A หมด** ให้รีบหักมุมบุกที่ **จุด B**
* **ส่งหมากแนวหลัง** เข้าเส้นทาง B ที่ขาดการป้องกัน
* **ประสานหน้าหลังเพื่อหนีบกิน** ในช่อง B ให้ได้มากที่สุดในเทิร์นเดียว


1. มีในไม่มี
   Score: 0.7151
   Confidence: 0.7244
   Similarity: 0.5290

2. ส่งเสียงบูรพาฝ่าตีประจิม
   Score: 0.7102
   Confidence: 0.7197
   Similarit

In [251]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: มีในไม่มี
Category:    กลยุทธ์เผชิญศึก
Description: สร้างเรื่องลวงให้ศัตรูสับสน

Metrics:
  Semantic Similarity:    0.5290
  6C Similarity:          0.9944
  Selection Score:        0.7151
  Confidence Score:       0.7244
  KL Divergence:          0.0061

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2080,0.2096,0.9923
ความสามารถในการรับ,0.1811,0.1695,0.9360
การประสานงานระหว่างหมาก,0.1706,0.2003,0.8520
ทรัพยากรที่ยังไม่ใช้,0.1439,0.1561,0.9217
ความได้เปรียบด้านเวลา,0.1517,0.1245,0.8207
ความเหมาะสมกับสถานการณ์,0.1448,0.1401,0.9676


# ลอบตีเฉินชาง

In [252]:
# Cell 18: Example Usage
test_query =  """
คุณจะใช้ **การแสดงเจตนาบุกทางหลัก** เพื่อเบี่ยงความสนใจ ก่อน **สอดแนม–บุกทางลับ** ที่คู่ต่อสู้ไม่ทันตั้งตัว
### 1. **ต้นเกม – โฆษณาทางหลัก**
* **วางหมากกลุ่มใหญ่ที่ช่อง M** ให้ดูเหมือนคุณตั้งใจเดินทะลวงจากจุดนั้น
* **ขยับหมาก M เล็กน้อย** สร้างภาพว่าเป็นแนวโจมตีหลัก
* **อย่าส่งสัญญาณบุกทางอื่น** ให้ศัตรูมุ่งปิดทาง M ทั้งหมด
### 2. **กลางเกม – เตรียมทางลับ)**
* **เคลื่อนหมากสำรองเงียบๆ ผ่านช่อง S** ซึ่งอยู่ด้านข้างของ M แต่ไม่ชัดเจนจนคู่ต่อสู้สังเกต
* **รวบรวมหมากสายลับ** ให้อยู่ในโซน S โดยยังคง “แอบ” ไม่ให้คู่ต่อสู้จับจุด
* **รักษาภาพลวงที่ M ต่อไป** เพื่อให้เขาไม่ถอนกำลังจาก M มาที่ S
### 3. **ปลายเกม – โจมตีทางลับ **
* เมื่อเห็นศัตรู **รวมกำลังมาปิด M หมด** ให้รีบสาดหมากจาก S เข้าไป
* **สอดหมากแบบประกบ–หนีบที่ช่อง S** ซึ่งขาดการป้องกันทันที
* **กวาดกินหมาก** ในบริเวณทางลับเพื่อสร้างความได้เปรียบเด็ดขาด
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
คุณจะใช้ **การแสดงเจตนาบุกทางหลัก** เพื่อเบี่ยงความสนใจ ก่อน **สอดแนม–บุกทางลับ** ที่คู่ต่อสู้ไม่ทันตั้งตัว
### 1. **ต้นเกม – โฆษณาทางหลัก**
* **วางหมากกลุ่มใหญ่ที่ช่อง M** ให้ดูเหมือนคุณตั้งใจเดินทะลวงจากจุดนั้น
* **ขยับหมาก M เล็กน้อย** สร้างภาพว่าเป็นแนวโจมตีหลัก
* **อย่าส่งสัญญาณบุกทางอื่น** ให้ศัตรูมุ่งปิดทาง M ทั้งหมด
### 2. **กลางเกม – เตรียมทางลับ)**
* **เคลื่อนหมากสำรองเงียบๆ ผ่านช่อง S** ซึ่งอยู่ด้านข้างของ M แต่ไม่ชัดเจนจนคู่ต่อสู้สังเกต
* **รวบรวมหมากสายลับ** ให้อยู่ในโซน S โดยยังคง “แอบ” ไม่ให้คู่ต่อสู้จับจุด
* **รักษาภาพลวงที่ M ต่อไป** เพื่อให้เขาไม่ถอนกำลังจาก M มาที่ S
### 3. **ปลายเกม – โจมตีทางลับ **
* เมื่อเห็นศัตรู **รวมกำลังมาปิด M หมด** ให้รีบสาดหมากจาก S เข้าไป
* **สอดหมากแบบประกบ–หนีบที่ช่อง S** ซึ่งขาดการป้องกันทันที
* **กวาดกินหมาก** ในบริเวณทางลับเพื่อสร้างความได้เปรียบเด็ดขาด


1. ลอบตีเฉินชาง
   Score: 0.7754
   Confidence: 0.7828
   Similarity: 0.6281

2. ตีหญ้าให้งูตื่น
   Score: 0.7198
   Confidence: 0.7291
   Similarity: 0.5345

3. ส่งเสียงบูรพ

In [253]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ลอบตีเฉินชาง
Category:    กลยุทธ์เผชิญศึก
Description: ทำทีจะเข้าทางหลัก แต่บุกทางลับ

Metrics:
  Semantic Similarity:    0.6281
  6C Similarity:          0.9964
  Selection Score:        0.7754
  Confidence Score:       0.7828
  KL Divergence:          0.0038

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1992,0.2166,0.9194
ความสามารถในการรับ,0.1910,0.1817,0.9517
การประสานงานระหว่างหมาก,0.1659,0.1849,0.8969
ทรัพยากรที่ยังไม่ใช้,0.1489,0.1516,0.9819
ความได้เปรียบด้านเวลา,0.1478,0.1288,0.8716
ความเหมาะสมกับสถานการณ์,0.1473,0.1362,0.9250


# ดูไฟชายฝั่ง

In [254]:
# Cell 18: Example Usage
test_query =  """
อย่าเข้าไปยุ่งในช่วงกลางเกม ปล่อยให้เขาโจมตีกันเองจนตำแหน่งอ่อน จากนั้นค่อยฉวยโอกาสเก็บหมากง่ายๆ ในช่วงปลายเกม
### 1. **ต้นเกม – รอดูท่าที**
* วางหมากแบบประคอง ไม่เปิดเกมรุกหนัก ให้กระดานดูนิ่งสงบ
* เคลื่อนไหวเพียงเล็กน้อย เพื่อไม่กระตุ้นให้คู่แข่งสนใจคุณมากเกินไป
### 2. **กลางเกม – ให้ศัตรู “แตกกันเอง”**
* ปล่อยให้คู่แข่งขยับหมากสู้กันเอง พยายามแย่งชิงพื้นที่หรือโจมตีจุดต่างๆ
* สังเกตการเคลื่อนตัวของเขา ว่ามีจุดใดที่ขาดการประสานงาน หรือมีการยืดตัวหมากจนเกินไปจนเกิดช่องว่าง
### 3. **ปลายเกม – ฉวยโอกาสหนีบกิน**
* เมื่อเห็นช่องว่างจากการกระจัดกระจายของศัตรู ให้รีบส่งหมากเข้า “หนีบ” ในตำแหน่งนั้น
* ใช้การประสานงานหมากหน้า–หลัง ประกบกินให้ได้มากสุดในเทิร์นเดียว
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
อย่าเข้าไปยุ่งในช่วงกลางเกม ปล่อยให้เขาโจมตีกันเองจนตำแหน่งอ่อน จากนั้นค่อยฉวยโอกาสเก็บหมากง่ายๆ ในช่วงปลายเกม
### 1. **ต้นเกม – รอดูท่าที**
* วางหมากแบบประคอง ไม่เปิดเกมรุกหนัก ให้กระดานดูนิ่งสงบ
* เคลื่อนไหวเพียงเล็กน้อย เพื่อไม่กระตุ้นให้คู่แข่งสนใจคุณมากเกินไป
### 2. **กลางเกม – ให้ศัตรู “แตกกันเอง”**
* ปล่อยให้คู่แข่งขยับหมากสู้กันเอง พยายามแย่งชิงพื้นที่หรือโจมตีจุดต่างๆ
* สังเกตการเคลื่อนตัวของเขา ว่ามีจุดใดที่ขาดการประสานงาน หรือมีการยืดตัวหมากจนเกินไปจนเกิดช่องว่าง
### 3. **ปลายเกม – ฉวยโอกาสหนีบกิน**
* เมื่อเห็นช่องว่างจากการกระจัดกระจายของศัตรู ให้รีบส่งหมากเข้า “หนีบ” ในตำแหน่งนั้น
* ใช้การประสานงานหมากหน้า–หลัง ประกบกินให้ได้มากสุดในเทิร์นเดียว


1. ดูไฟชายฝั่ง
   Score: 0.7226
   Confidence: 0.7317
   Similarity: 0.5414

2. จูงแพะติดมือ
   Score: 0.6874
   Confidence: 0.6978
   Similarity: 0.4799

3. ปิดฟ้าข้ามทะเล
   Score: 0.6817
   Confidence: 0.6921
   Similarity: 0.4732

4. ส่งเสียงบูรพาฝ่าตีประจิม
   Score: 0.6701
   Confidence: 0.6808
   Similarity: 0.4563


In [255]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ดูไฟชายฝั่ง
Category:    กลยุทธ์เผชิญศึก
Description: รอดูให้ศัตรูแตกกันเองก่อนค่อยลงมือ

Metrics:
  Semantic Similarity:    0.5414
  6C Similarity:          0.9945
  Selection Score:        0.7226
  Confidence Score:       0.7317
  KL Divergence:          0.0060

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1925,0.1997,0.9637
ความสามารถในการรับ,0.1894,0.1919,0.9870
การประสานงานระหว่างหมาก,0.1579,0.1849,0.8539
ทรัพยากรที่ยังไม่ใช้,0.1546,0.1512,0.9777
ความได้เปรียบด้านเวลา,0.1675,0.1347,0.8041
ความเหมาะสมกับสถานการณ์,0.1382,0.1377,0.9965


# ซ่อนดาบในรอยยิ้ม

In [256]:
# Cell 18: Example Usage
test_query =  """
การ “เสนอมิตรภาพ” ด้วยการไม่ขยับรุกอย่างชัดเจน จะทำให้คู่แข่งละความระวัง เมื่อเขาหลวมตัวแลกหมาก คุณจะสวนกลับได้อย่างรวดเร็วและเด็ดขาด
### 1. **ต้นเกม – เสนอความเป็นมิตรชัดเจน**
* วางหมากในโซนกึ่งกลาง–หลังกระดาน บ่งบอกว่าคุณตั้งรับ ไม่ตั้งใจบุก
* เคลื่อนไหวช้าลง หลีกเลี่ยงการประกบหรือกดดัน ให้ศัตรูรู้สึกปลอดภัยและไม่ระวังตัว
### 2. **กลางเกม – ชวนแลกหมากแบบไม่จริงจัง**
* เดินหมากเข้าหาเขาเล็กน้อย ราวกับ “ชวนแลก” แต่ไม่ลงกิน
* แสดงท่าทีกึ่งรับกึ่งรุก สร้างความเชื่อว่าคุณต้องการแลกหมากอย่างยุติธรรม
### 3. **ปลายเกม – “ยิงทีหลัง” เมื่อเขาหลวมตัว**
* เมื่อคู่แข่งคลายกำลังมาช่วยป้องกันหรือแลกหมากกับคุณ ให้รีบใช้หมากสองตัวประกบ–หนีบ หรือสอดตัวเดียวระหว่างสองตัว บริเวณที่เขาโฟกัสผิดจุด
* ประสานหมากหน้า–หลัง รีบกินหมากที่โผล่มา และพลิกเกมสู่ชัยชนะในเทิร์นเดียว
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
การ “เสนอมิตรภาพ” ด้วยการไม่ขยับรุกอย่างชัดเจน จะทำให้คู่แข่งละความระวัง เมื่อเขาหลวมตัวแลกหมาก คุณจะสวนกลับได้อย่างรวดเร็วและเด็ดขาด
### 1. **ต้นเกม – เสนอความเป็นมิตรชัดเจน**
* วางหมากในโซนกึ่งกลาง–หลังกระดาน บ่งบอกว่าคุณตั้งรับ ไม่ตั้งใจบุก
* เคลื่อนไหวช้าลง หลีกเลี่ยงการประกบหรือกดดัน ให้ศัตรูรู้สึกปลอดภัยและไม่ระวังตัว
### 2. **กลางเกม – ชวนแลกหมากแบบไม่จริงจัง**
* เดินหมากเข้าหาเขาเล็กน้อย ราวกับ “ชวนแลก” แต่ไม่ลงกิน
* แสดงท่าทีกึ่งรับกึ่งรุก สร้างความเชื่อว่าคุณต้องการแลกหมากอย่างยุติธรรม
### 3. **ปลายเกม – “ยิงทีหลัง” เมื่อเขาหลวมตัว**
* เมื่อคู่แข่งคลายกำลังมาช่วยป้องกันหรือแลกหมากกับคุณ ให้รีบใช้หมากสองตัวประกบ–หนีบ หรือสอดตัวเดียวระหว่างสองตัว บริเวณที่เขาโฟกัสผิดจุด
* ประสานหมากหน้า–หลัง รีบกินหมากที่โผล่มา และพลิกเกมสู่ชัยชนะในเทิร์นเดียว


1. ซ่อนดาบในรอยยิ้ม
   Score: 0.6919
   Confidence: 0.7019
   Similarity: 0.4926

2. ดูไฟชายฝั่ง
   Score: 0.6813
   Confidence: 0.6919
   Similarity: 0.4702

3. ปิดฟ้าข้ามทะเล
   Score: 0.6792
   Confidence: 0.6899
   Similarit

In [257]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ซ่อนดาบในรอยยิ้ม
Category:    กลยุทธ์เผชิญศึก
Description: แสร้งเป็นมิตร แล้วยิงทีหลัง

Metrics:
  Semantic Similarity:    0.4926
  6C Similarity:          0.9910
  Selection Score:        0.6919
  Confidence Score:       0.7019
  KL Divergence:          0.0101

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1957,0.2090,0.9365
ความสามารถในการรับ,0.1923,0.1852,0.9626
การประสานงานระหว่างหมาก,0.1674,0.2046,0.8183
ทรัพยากรที่ยังไม่ใช้,0.1500,0.1518,0.9882
ความได้เปรียบด้านเวลา,0.1529,0.1147,0.7503
ความเหมาะสมกับสถานการณ์,0.1417,0.1348,0.9515


# หลี่ตายแทนถาว

In [258]:
# Cell 18: Example Usage
test_query =  """
การเสียหมากเล็กน้อยในช่วงต้นเป็นการลงทุนระยะสั้นเพื่อเปิดทางสร้างโอกาสหนีบกินหมากใหญ่ในช่วงท้ายเกม
### 1. **ต้นเกม – เสียของเล็กน้อยเป็นเหยื่อล่อ**
* **ยอมให้ศัตรูกินหมากตัวรอง** เช่น เดินหมากออกไปตั้งแต่ต้นเกมให้ดูเหมือนอ่อนแอ
* **ไม่ใช่หมากหลัก** จึงไม่เสียเปรียบมาก แต่เป็นการล่อให้เขารู้สึกได้เปรียบ
### 2. **กลางเกม – เก็บกำไรจากช่องว่างที่เปิด**
* เมื่อศัตรูโฟกัสมาตามเหยื่อ ให้คุณ **รวบรวมหมากหลัก** ไว้ใกล้กัน พร้อมสวนกลับ
* ใช้การประสานงานระหว่างหมากเพื่อสร้างกับดักในตำแหน่งที่เขาทอดทิ้ง
### 3. **ปลายเกม – ต่อยอดกำไรใหญ่จากการหนีบ**
* เปิดฉากสวนกลับด้วย **การหนีบกินหมากตัวหลัก** ที่เขาเคลื่อนเข้ามาตามเหยื่อ
* ผลลัพธ์คือคุณได้กินหมากจำนวนมากในเทิร์นเดียว กลายเป็นกำไรใหญ่เหนือความเสียเปรียบเบื้องต้น
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
การเสียหมากเล็กน้อยในช่วงต้นเป็นการลงทุนระยะสั้นเพื่อเปิดทางสร้างโอกาสหนีบกินหมากใหญ่ในช่วงท้ายเกม
### 1. **ต้นเกม – เสียของเล็กน้อยเป็นเหยื่อล่อ**
* **ยอมให้ศัตรูกินหมากตัวรอง** เช่น เดินหมากออกไปตั้งแต่ต้นเกมให้ดูเหมือนอ่อนแอ
* **ไม่ใช่หมากหลัก** จึงไม่เสียเปรียบมาก แต่เป็นการล่อให้เขารู้สึกได้เปรียบ
### 2. **กลางเกม – เก็บกำไรจากช่องว่างที่เปิด**
* เมื่อศัตรูโฟกัสมาตามเหยื่อ ให้คุณ **รวบรวมหมากหลัก** ไว้ใกล้กัน พร้อมสวนกลับ
* ใช้การประสานงานระหว่างหมากเพื่อสร้างกับดักในตำแหน่งที่เขาทอดทิ้ง
### 3. **ปลายเกม – ต่อยอดกำไรใหญ่จากการหนีบ**
* เปิดฉากสวนกลับด้วย **การหนีบกินหมากตัวหลัก** ที่เขาเคลื่อนเข้ามาตามเหยื่อ
* ผลลัพธ์คือคุณได้กินหมากจำนวนมากในเทิร์นเดียว กลายเป็นกำไรใหญ่เหนือความเสียเปรียบเบื้องต้น


1. หลี่ตายแทนถาว
   Score: 0.7603
   Confidence: 0.7681
   Similarity: 0.6038

2. จูงแพะติดมือ
   Score: 0.7245
   Confidence: 0.7336
   Similarity: 0.5423

3. ดูไฟชายฝั่ง
   Score: 0.6904
   Confidence: 0.7005
   Similarity: 0.4868

4. กวนน้ำจับปลา
   Score: 0.6714
   Confiden

In [259]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: หลี่ตายแทนถาว
Category:    กลยุทธ์เผชิญศึก
Description: เสียของเล็กน้อยเพื่อได้กำไรใหญ่

Metrics:
  Semantic Similarity:    0.6038
  6C Similarity:          0.9950
  Selection Score:        0.7603
  Confidence Score:       0.7681
  KL Divergence:          0.0051

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2020,0.2029,0.9958
ความสามารถในการรับ,0.1792,0.1540,0.8596
การประสานงานระหว่างหมาก,0.1586,0.1867,0.8497
ทรัพยากรที่ยังไม่ใช้,0.1561,0.1677,0.9308
ความได้เปรียบด้านเวลา,0.1574,0.1462,0.9291
ความเหมาะสมกับสถานการณ์,0.1467,0.1425,0.9713


# จูงแพะติดมือ

In [260]:
# Cell 18: Example Usage
test_query =  """
โฟกัสที่ “เหยื่อเล็ก” เหล่านั้น ขณะคู่แข่งไม่ระวัง เพื่อสะสมกำไรเล็กๆ น้อยๆ จนได้เปรียบในภาพรวมเกม
### 1. **ต้นเกม – ดักรอเหยื่อเล็ก**
* วางหมากให้ศัตรูเห็นช่องว่างเล็กๆ (หมากเดี่ยวหรือหมากรอง) แต่ไม่เดินลุยหนัก
* รักษารูปแบบรับนิ่งๆ ให้เขาไม่ระวังช่องเล็กนั้น
### 2. **กลางเกม – เบี่ยงเบนความสนใจ**
* ขยับหมาก “ชิมเชิง” ที่อีกฝั่งหนึ่ง เพื่อให้คู่แข่งโฟกัสป้องกันตรงหลัก
* ขณะเดียวกัน เหยี่ยวตาไวคอยจับโอกาสเมื่อตัวเล็กโผล่มา
### 3. **ปลายเกม – ฉวยโอกาสฉกหมากเล็ก**
* เมื่อเห็นหมากเดี่ยวหรือหมากรองอยู่โดดเดี่ยวในช่องเปล่า ให้รีบเดินหมากเข้าแทรก
* ใช้การสอดหรือประกบหนีบกินหมากเล็กนั้น แล้วขยายเป็นจังหวะรุกใหญ่ต่อไป
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
โฟกัสที่ “เหยื่อเล็ก” เหล่านั้น ขณะคู่แข่งไม่ระวัง เพื่อสะสมกำไรเล็กๆ น้อยๆ จนได้เปรียบในภาพรวมเกม
### 1. **ต้นเกม – ดักรอเหยื่อเล็ก**
* วางหมากให้ศัตรูเห็นช่องว่างเล็กๆ (หมากเดี่ยวหรือหมากรอง) แต่ไม่เดินลุยหนัก
* รักษารูปแบบรับนิ่งๆ ให้เขาไม่ระวังช่องเล็กนั้น
### 2. **กลางเกม – เบี่ยงเบนความสนใจ**
* ขยับหมาก “ชิมเชิง” ที่อีกฝั่งหนึ่ง เพื่อให้คู่แข่งโฟกัสป้องกันตรงหลัก
* ขณะเดียวกัน เหยี่ยวตาไวคอยจับโอกาสเมื่อตัวเล็กโผล่มา
### 3. **ปลายเกม – ฉวยโอกาสฉกหมากเล็ก**
* เมื่อเห็นหมากเดี่ยวหรือหมากรองอยู่โดดเดี่ยวในช่องเปล่า ให้รีบเดินหมากเข้าแทรก
* ใช้การสอดหรือประกบหนีบกินหมากเล็กนั้น แล้วขยายเป็นจังหวะรุกใหญ่ต่อไป


1. จูงแพะติดมือ
   Score: 0.7625
   Confidence: 0.7704
   Similarity: 0.6044

2. ส่งเสียงบูรพาฝ่าตีประจิม
   Score: 0.7075
   Confidence: 0.7170
   Similarity: 0.5179

3. หลี่ตายแทนถาว
   Score: 0.6920
   Confidence: 0.7020
   Similarity: 0.4920

4. ดูไฟชายฝั่ง
   Score: 0.6833
   Confidence: 0.6937
   Similarity: 0.4749

5. กวนน้ำจับปลา
   Score: 0.6781
   Confidence: 

In [261]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: จูงแพะติดมือ
Category:    กลยุทธ์เผชิญศึก
Description: ฉกเอาสิ่งเล็กๆ ตอนศัตรูไม่ทันระวัง

Metrics:
  Semantic Similarity:    0.6044
  6C Similarity:          0.9996
  Selection Score:        0.7625
  Confidence Score:       0.7704
  KL Divergence:          0.0004

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1877,0.1869,0.9959
ความสามารถในการรับ,0.1932,0.1917,0.9920
การประสานงานระหว่างหมาก,0.1659,0.1732,0.9584
ทรัพยากรที่ยังไม่ใช้,0.1570,0.1587,0.9892
ความได้เปรียบด้านเวลา,0.1637,0.1550,0.9470
ความเหมาะสมกับสถานการณ์,0.1325,0.1346,0.9847


# ตีหญ้าให้งูตื่น

In [262]:
# Cell 18: Example Usage
test_query =  """
ใช้หมากสอดแนม “หมายหัว” จุดอ่อนก่อน แล้วค่อยส่งกองกำลังหลักเข้าโจมตี จะได้ผลลัพธ์ชัดเจนและปลอดภัย
1. **ต้นเกม – ส่งหมากสอดแนม**
   * เลือกหมากเดี่ยวหนึ่งตัว เดินเข้ากึ่งกลางหรือแนวขอบกระดานพอให้คู่แข่งสะดุ้ง
   * ไม่ล้ำลึกจนเกินไป แค่พอให้เขาส่งหมากมาตอบโต้
2. **กลางเกม – สังเกต–วิเคราะห์**
   * ดูว่าคู่แข่งส่งหมากมาเสริมแนวตรงใด ช่องว่างเกิดตรงจุดไหน
   * จดจำจังหวะและตำแหน่งที่เขาอ่อนแอ เพื่อเลือกเวลาที่เหมาะสมบุกจริง
3. **ปลายเกม – เปิดฉากโจมตี**
   * เมื่อพบช่องว่างหรือแนวรับบางลง ให้รวบหมากหลักเข้า “สวนกลับ”
   * ประสานแนวหน้า–หลังหนีบกินหมากในตำแหน่งที่สอดแนมชี้ช่องทันที
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
ใช้หมากสอดแนม “หมายหัว” จุดอ่อนก่อน แล้วค่อยส่งกองกำลังหลักเข้าโจมตี จะได้ผลลัพธ์ชัดเจนและปลอดภัย
1. **ต้นเกม – ส่งหมากสอดแนม**
   * เลือกหมากเดี่ยวหนึ่งตัว เดินเข้ากึ่งกลางหรือแนวขอบกระดานพอให้คู่แข่งสะดุ้ง
   * ไม่ล้ำลึกจนเกินไป แค่พอให้เขาส่งหมากมาตอบโต้
2. **กลางเกม – สังเกต–วิเคราะห์**
   * ดูว่าคู่แข่งส่งหมากมาเสริมแนวตรงใด ช่องว่างเกิดตรงจุดไหน
   * จดจำจังหวะและตำแหน่งที่เขาอ่อนแอ เพื่อเลือกเวลาที่เหมาะสมบุกจริง
3. **ปลายเกม – เปิดฉากโจมตี**
   * เมื่อพบช่องว่างหรือแนวรับบางลง ให้รวบหมากหลักเข้า “สวนกลับ”
   * ประสานแนวหน้า–หลังหนีบกินหมากในตำแหน่งที่สอดแนมชี้ช่องทันที


1. ตีหญ้าให้งูตื่น
   Score: 0.6704
   Confidence: 0.6813
   Similarity: 0.4524

2. ปิดฟ้าข้ามทะเล
   Score: 0.6666
   Confidence: 0.6776
   Similarity: 0.4466

3. ยืมทางพรางกล
   Score: 0.6594
   Confidence: 0.6706
   Similarity: 0.4355

4. ดูไฟชายฝั่ง
   Score: 0.6573
   Confidence: 0.6685
   Similarity: 0.4313

5. ลอบตีเฉินชาง
   Score: 0.6535
   Confidence: 0.6649
   Similarity: 0.4265


In [263]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ตีหญ้าให้งูตื่น
Category:    กลยุทธ์เข้าตี
Description: ส่งคนสอดแนมดูความเคลื่อนไหวก่อนบุก

Metrics:
  Semantic Similarity:    0.4524
  6C Similarity:          0.9974
  Selection Score:        0.6704
  Confidence Score:       0.6813
  KL Divergence:          0.0028

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1943,0.2099,0.9259
ความสามารถในการรับ,0.1812,0.1781,0.9829
การประสานงานระหว่างหมาก,0.1657,0.1787,0.9275
ทรัพยากรที่ยังไม่ใช้,0.1518,0.1320,0.8693
ความได้เปรียบด้านเวลา,0.1619,0.1533,0.9467
ความเหมาะสมกับสถานการณ์,0.1451,0.1481,0.9793


# ยืมซากคืนชีพ

In [264]:
# Cell 18: Example Usage
test_query =  """
คุณจะมุ่งเน้นการ**นำหมากที่อยู่ในตำแหน่งดูเหมือนไร้ค่าหรือถูกละเลย** กลับมาใช้งานได้อย่างมีประสิทธิภาพอีกครั้ง โดยการ**ค้นพบและใช้ประโยชน์จากศักยภาพที่ซ่อนอยู่** เพื่อสร้างโอกาสในการรุกหรือรับที่เหนือความคาดหมายอย่างตรงไปตรงมา
### 1. **ช่วงเริ่มต้น: ค้นหาศักยภาพที่ถูกซ่อน – มองหาหมากที่ถูกละเลย**
* **เน้นที่:** ทรัพยากรที่ยังไม่ใช้
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณ**สำรวจหมากของตนเองและของฝ่ายตรงข้ามที่อยู่ในตำแหน่งที่ดูเหมือนไม่มีบทบาทสำคัญ, โดดเดี่ยว, หรือถูกปิดกั้น** แทนที่จะมองข้าม ให้พิจารณาถึง**บทบาทใหม่ที่หมากเหล่านั้นสามารถทำได้** เช่น หมากที่ใช้ป้องกันเพียงอย่างเดียว หรือหมากที่อยู่ห่างจากการปะทะหลัก
* **วัตถุประสงค์:** ระบุหมากที่ยังไม่ได้ถูกใช้เต็มศักยภาพ หรือหมากที่สามารถปรับเปลี่ยนบทบาทเพื่อสร้างประโยชน์เชิงกลยุทธ์ได้
### 2. **ช่วงกลางเกม: ปรับเปลี่ยนและเปิดใช้งาน – มอบบทบาทใหม่ให้หมากที่ถูกลืม**
* **เน้นที่:** การประสานงานระหว่างหมาก, ความเหมาะสมกับสถานการณ์
* **วิธีการเดินหมาก:** เมื่อระบุหมากที่มีศักยภาพแล้ว ให้คุณเริ่ม**เคลื่อนย้ายหรือใช้หมากเหล่านั้นในลักษณะที่แตกต่างไปจากเดิม** นี่คือการ**เปิดใช้งานบทบาทใหม่** ให้กับหมาก เช่น:
    * **เชื่อมโยง:** ขยับหมากที่โดดเดี่ยวให้เชื่อมต่อกับแนวรุกหรือรับหลัก
    * **เปิดทาง:** ใช้หมากที่เคยอยู่นิ่งๆ เพื่อเปิดเส้นทางให้หมากสำคัญอื่นๆ เคลื่อนที่ไปทำลายแนวรับของศัตรู
    * **เสริมกำลัง:** นำหมากที่ถูกมองข้ามมาเสริมความแข็งแกร่งให้แนวป้องกันหรือสร้างแรงกดดันเพิ่มเติมในแนวรุก
    **ประสานงานหมากเหล่านี้กับหมากตัวอื่นๆ อย่างมีแบบแผน** เพื่อสร้างรูปแบบการเล่นที่ต่อยอดกัน
* **วัตถุประสงค์:** เปลี่ยนสถานะของหมากจาก "ไม่สำคัญ" ให้กลายเป็นส่วนหนึ่งที่มีชีวิตชีวาและสร้างประโยชน์โดยตรงในการดำเนินกลยุทธ์
### 3. **ช่วงปลายเกม: ใช้ประโยชน์จากบทบาทใหม่ – โจมตีด้วยหมากที่ฟื้นคืน**
* **เน้นที่:** ความสามารถในการรุก, ความได้เปรียบด้านเวลา
* **วิธีการเดินหมาก:** เมื่อหมากที่ถูก "ยืมซากคืนชีพ" ได้รับบทบาทใหม่และเริ่มส่งผลต่อกระดาน **ให้คุณเปิดฉากโจมตีอย่างรวดเร็วและเด็ดขาด** ใช้ประโยชน์จากตำแหน่งหรือการควบคุมพื้นที่ที่เกิดจากการฟื้นคืนชีพของหมากนั้น ส่งหมากหลักเข้า "หนีบ" กินหมากของฝ่ายตรงข้ามในจุดสำคัญที่ถูกสร้างขึ้นโดยความร่วมมือของหมากที่เคยถูกละเลย
* **วัตถุประสงค์:** ใช้ความสามารถใหม่ของหมากที่ถูกฟื้นคืนชีพ สร้างความได้เปรียบที่ชัดเจนและนำไปสู่ชัยชนะ
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
คุณจะมุ่งเน้นการ**นำหมากที่อยู่ในตำแหน่งดูเหมือนไร้ค่าหรือถูกละเลย** กลับมาใช้งานได้อย่างมีประสิทธิภาพอีกครั้ง โดยการ**ค้นพบและใช้ประโยชน์จากศักยภาพที่ซ่อนอยู่** เพื่อสร้างโอกาสในการรุกหรือรับที่เหนือความคาดหมายอย่างตรงไปตรงมา
### 1. **ช่วงเริ่มต้น: ค้นหาศักยภาพที่ถูกซ่อน – มองหาหมากที่ถูกละเลย**
* **เน้นที่:** ทรัพยากรที่ยังไม่ใช้
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณ**สำรวจหมากของตนเองและของฝ่ายตรงข้ามที่อยู่ในตำแหน่งที่ดูเหมือนไม่มีบทบาทสำคัญ, โดดเดี่ยว, หรือถูกปิดกั้น** แทนที่จะมองข้าม ให้พิจารณาถึง**บทบาทใหม่ที่หมากเหล่านั้นสามารถทำได้** เช่น หมากที่ใช้ป้องกันเพียงอย่างเดียว หรือหมากที่อยู่ห่างจากการปะทะหลัก
* **วัตถุประสงค์:** ระบุหมากที่ยังไม่ได้ถูกใช้เต็มศักยภาพ หรือหมากที่สามารถปรับเปลี่ยนบทบาทเพื่อสร้างประโยชน์เชิงกลยุทธ์ได้
### 2. **ช่วงกลางเกม: ปรับเปลี่ยนและเปิดใช้งาน – มอบบทบาทใหม่ให้หมากที่ถูกลืม**
* **เน้นที่:** การประสานงานระหว่างหมาก, ความเหมาะสมกับสถานการณ์
* **วิธีการเดินหมาก:** เมื่อระบุหมากที่มีศักยภาพแล้ว ให้คุณเริ่ม**เคลื่อนย้ายหรือใช้หมากเห

In [265]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ยืมซากคืนชีพ
Category:    กลยุทธ์เข้าตี
Description: ใช้สิ่งที่ยังพอใช้งานสร้างประโยชน์ใหม่

Metrics:
  Semantic Similarity:    0.4848
  6C Similarity:          0.9916
  Selection Score:        0.6875
  Confidence Score:       0.6976
  KL Divergence:          0.0092

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1948,0.2004,0.9720
ความสามารถในการรับ,0.1562,0.1384,0.8863
การประสานงานระหว่างหมาก,0.1514,0.1886,0.8031
ทรัพยากรที่ยังไม่ใช้,0.1933,0.2045,0.9450
ความได้เปรียบด้านเวลา,0.1606,0.1284,0.7994
ความเหมาะสมกับสถานการณ์,0.1437,0.1397,0.9721


# ล่อเสือออกจากถ้ำ

In [266]:
# Cell 18: Example Usage
test_query =  """
เหยื่อล่อหน้าป้อมจะชวนให้ศัตรูคลายแนวรับ พอเขาออกมาจนตำแหน่งอ่อน ให้คุณสวนกลับเข้าไปเก็บกินหมากหลักอย่างรวดเร็ว
### 1. **ต้นเกม – วางเหยื่อล่อหน้าป้อม**
* วางหมากตัวรองหรือหมากสำรองแถวหน้าป้อมของศัตรู ให้ดูเหมือนเป้าอ่อนแอ
* เคลื่อนไหววนเล็กน้อยเพื่อยั่วให้เขาอยากส่งหมากออกมาจู่โจม
### 2. **กลางเกม – ดึงให้หลุดแนวรับ**
* เมื่อศัตรูถอนหมากหลักออกมาไล่เหยื่อ ให้คุณใช้หมากสำรองอื่น ๆ คอยรักษาแนว
* ใช้การเดินหมากช้า ๆ ปล่อยให้เขาเดินออกนอก “ป้อม” จนแนวรับบาง
### 3. **ปลายเกม – บุกทะลวงเข้า “ป้อมแตก”**
* เมื่อเห็นแนวรับเขาบาง ให้ส่งหมากหน้า–หลังเข้าประสาน
* ใช้รูปแบบหนีบสองตัวหรือสอดตัวเดียวระหว่างสองตัว เพื่อ “หนีบ” กินหมากในป้อมของศัตรูทันที
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
เหยื่อล่อหน้าป้อมจะชวนให้ศัตรูคลายแนวรับ พอเขาออกมาจนตำแหน่งอ่อน ให้คุณสวนกลับเข้าไปเก็บกินหมากหลักอย่างรวดเร็ว
### 1. **ต้นเกม – วางเหยื่อล่อหน้าป้อม**
* วางหมากตัวรองหรือหมากสำรองแถวหน้าป้อมของศัตรู ให้ดูเหมือนเป้าอ่อนแอ
* เคลื่อนไหววนเล็กน้อยเพื่อยั่วให้เขาอยากส่งหมากออกมาจู่โจม
### 2. **กลางเกม – ดึงให้หลุดแนวรับ**
* เมื่อศัตรูถอนหมากหลักออกมาไล่เหยื่อ ให้คุณใช้หมากสำรองอื่น ๆ คอยรักษาแนว
* ใช้การเดินหมากช้า ๆ ปล่อยให้เขาเดินออกนอก “ป้อม” จนแนวรับบาง
### 3. **ปลายเกม – บุกทะลวงเข้า “ป้อมแตก”**
* เมื่อเห็นแนวรับเขาบาง ให้ส่งหมากหน้า–หลังเข้าประสาน
* ใช้รูปแบบหนีบสองตัวหรือสอดตัวเดียวระหว่างสองตัว เพื่อ “หนีบ” กินหมากในป้อมของศัตรูทันที


1. ล่อเสือออกจากถ้ำ
   Score: 0.7814
   Confidence: 0.7886
   Similarity: 0.6382

2. ส่งเสียงบูรพาฝ่าตีประจิม
   Score: 0.7266
   Confidence: 0.7355
   Similarity: 0.5475

3. ปิดฟ้าข้ามทะเล
   Score: 0.6904
   Confidence: 0.7007
   Similarity: 0.4851

4. ทุกข์กาย
   Score: 0.6899
   Confidence: 0.7001
   Similarity: 0.4850

5. ดูไฟชายฝั่ง
  

In [267]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ล่อเสือออกจากถ้ำ
Category:    กลยุทธ์เข้าตี
Description: หลอกล่อให้ศัตรูหลุดจากป้อม แล้วบุก

Metrics:
  Semantic Similarity:    0.6382
  6C Similarity:          0.9962
  Selection Score:        0.7814
  Confidence Score:       0.7886
  KL Divergence:          0.0039

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2033,0.2230,0.9117
ความสามารถในการรับ,0.2002,0.1879,0.9388
การประสานงานระหว่างหมาก,0.1736,0.1897,0.9149
ทรัพยากรที่ยังไม่ใช้,0.1439,0.1381,0.9594
ความได้เปรียบด้านเวลา,0.1467,0.1251,0.8532
ความเหมาะสมกับสถานการณ์,0.1324,0.1362,0.9721


# แสร้งปล่อยเพื่อจับ

In [268]:
# Cell 18: Example Usage
test_query =  """
*ปล่อยให้หนีแล้วไล่บี้จนอ่อนกำลัง*
---
### 1. **ต้นเกม – แสร้งถอยล่อให้ตาม**
* เลือกหมากหลักสักตัว (เช่น P) เดินถอยหลัง 1–2 ช่อง ออกจากแนวรับ
* แสดงท่าที “ถอยหนี” อย่างชัดเจน ไม่เสริมกำลังรุกหนัก เพื่อให้เขาเชื่อว่าคุณกำลังเสียเปรียบ
### 2. **กลางเกม – ไล่บี้ให้เหนื่อยล้า**
* เมื่อเขา **รีบไล่ตาม** ใช้หมากรอง (R) หรือหมากสำรองเข้าประกบ ทิ้งช่องว่างด้านหลัง
* เคลื่อนไหววนซ้ำตรงแนวที่เขาตามมา (เช่น P ↔ R) บังคับให้เขาใช้หมากและคิดตัดสินใจหลายเทิร์นจนเริ่มอ่อนแรง
### 3. **ปลายเกม – จับหนีบกินจังหวะอ่อนแรง**
* เมื่อสังเกตว่าแนวรับเขา “บาง” จากการไล่จนหมากกระจัดกระจาย ให้รวบหมากหลัก–สำรองเข้าประสาน
* ใช้ **หนีบสองตัว** (หรือสอดตัวเดียวระหว่างสองตัว) ที่ช่องที่ถูกทอดทิ้ง เพื่อกินหมากคู่ต่อสู้จำนวนมากในเทิร์นเดียว
ไม่ได้แค่ถอยหนีแล้วรอให้เขาพลาด แต่เป็นการ **ถอยล่อ** เพื่อให้เขาปลดแนวรับออก แล้ว **ไล่บี้จนอ่อนกำลัง** ก่อนจะ **ซ้ำด้วยการหนีบกิน**
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
*ปล่อยให้หนีแล้วไล่บี้จนอ่อนกำลัง*
---
### 1. **ต้นเกม – แสร้งถอยล่อให้ตาม**
* เลือกหมากหลักสักตัว (เช่น P) เดินถอยหลัง 1–2 ช่อง ออกจากแนวรับ
* แสดงท่าที “ถอยหนี” อย่างชัดเจน ไม่เสริมกำลังรุกหนัก เพื่อให้เขาเชื่อว่าคุณกำลังเสียเปรียบ
### 2. **กลางเกม – ไล่บี้ให้เหนื่อยล้า**
* เมื่อเขา **รีบไล่ตาม** ใช้หมากรอง (R) หรือหมากสำรองเข้าประกบ ทิ้งช่องว่างด้านหลัง
* เคลื่อนไหววนซ้ำตรงแนวที่เขาตามมา (เช่น P ↔ R) บังคับให้เขาใช้หมากและคิดตัดสินใจหลายเทิร์นจนเริ่มอ่อนแรง
### 3. **ปลายเกม – จับหนีบกินจังหวะอ่อนแรง**
* เมื่อสังเกตว่าแนวรับเขา “บาง” จากการไล่จนหมากกระจัดกระจาย ให้รวบหมากหลัก–สำรองเข้าประสาน
* ใช้ **หนีบสองตัว** (หรือสอดตัวเดียวระหว่างสองตัว) ที่ช่องที่ถูกทอดทิ้ง เพื่อกินหมากคู่ต่อสู้จำนวนมากในเทิร์นเดียว
ไม่ได้แค่ถอยหนีแล้วรอให้เขาพลาด แต่เป็นการ **ถอยล่อ** เพื่อให้เขาปลดแนวรับออก แล้ว **ไล่บี้จนอ่อนกำลัง** ก่อนจะ **ซ้ำด้วยการหนีบกิน**


1. แสร้งปล่อยเพื่อจับ
   Score: 0.7745
   Confidence: 0.7819
   Similarity: 0.6264

2. ล่อเสือออกจากถ้ำ
   Score: 0.7027
   Confidence: 0.7

In [269]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: แสร้งปล่อยเพื่อจับ
Category:    กลยุทธ์เข้าตี
Description: ปล่อยให้หนีแล้วไล่บี้จนอ่อนกำลัง

Metrics:
  Semantic Similarity:    0.6264
  6C Similarity:          0.9967
  Selection Score:        0.7745
  Confidence Score:       0.7819
  KL Divergence:          0.0035

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2102,0.2287,0.9192
ความสามารถในการรับ,0.1986,0.1842,0.9277
การประสานงานระหว่างหมาก,0.1748,0.1807,0.9672
ทรัพยากรที่ยังไม่ใช้,0.1269,0.1407,0.9020
ความได้เปรียบด้านเวลา,0.1462,0.1280,0.8759
ความเหมาะสมกับสถานการณ์,0.1433,0.1376,0.9603


# โยนกระเบื้องล่อหยก

In [270]:
# Cell 18: Example Usage
test_query =  """
ยอมลงทุนด้วยสิ่งที่ดูเหมือนไม่สำคัญ หรือสร้างสถานการณ์ที่ศัตรูคิดว่าจะได้เปรียบเพียงเล็กน้อย** เพื่อล่อให้พวกเขาหลงกลและเบนความสนใจออกจากเป้าหมายที่แท้จริง จากนั้นจึง**ฉวยโอกาสเข้ายึดหมากหรือพื้นที่ที่มีความสำคัญมากกว่า**อย่างเด็ดขาด
### 1. **ช่วงเริ่มต้น: วาง "กระเบื้อง" ล่อ – ใช้สิ่งไม่สำคัญเบี่ยงเบนความสนใจ**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณ**เสนอ "กระเบื้อง"** ซึ่งอาจเป็นหมากที่มีค่าน้อย, การยอมเสียพื้นที่เล็กน้อย, หรือการเดินหมากที่ดูเหมือนจะให้ฝ่ายตรงข้ามได้เปรียบเพียงเล็กน้อย **โดยมีจุดประสงค์เพื่อดึงดูดความสนใจของศัตรูและทำให้พวกเขาเข้าใจผิด**ว่านี่คือเป้าหมายหลัก
* **ใช้สิ่งไม่สำคัญล่อให้หลงยังไง:** คุณกำลังใช้หมากหรือการกระทำที่ดูเหมือนไม่สลักสำคัญ เพื่อ**สร้างความดึงดูดใจหรือความเชื่อมั่นจอมปลอม** ให้ศัตรูโฟกัสไปที่ "กระเบื้อง" แทนที่จะเป็น "หยก" ที่ซ่อนอยู่
* **วัตถุประสงค์:** เพื่อเบี่ยงเบนความสนใจและกำลังของศัตรูไปยังเป้าหมายลวง ทำให้เป้าหมายที่แท้จริง (หยก) ปลอดภัยและพร้อมสำหรับการโจมตี
### 2. **ช่วงกลางเกม: เคลื่อนพลสู่ "หยก" – จัดทัพลับไปยังเป้าหมายจริง**
* **วิธีการเดินหมาก:** ในขณะที่ศัตรูกำลังวุ่นวายกับการรับมือหรือไล่ตาม "กระเบื้อง" ที่คุณโยนไป ให้คุณ**ค่อยๆ เคลื่อนย้ายหมากสำคัญ**ไปยังตำแหน่งเชิงยุทธศาสตร์ที่ใกล้กับ **"หยก" (เป้าหมายที่แท้จริง)** ที่คุณซ่อนไว้ หมากของคุณควรประสานงานกันอย่างเงียบเชียบและรัดกุม เพื่อสร้างรูปขบวนพร้อมหนีบหรือเข้ายึดโดยที่ศัตรูยังคงหลงอยู่กับ "กระเบื้อง"
* **วัตถุประสงค์:** เพื่อเตรียมกำลังและวางตำแหน่งหมากให้พร้อมสำหรับการโจมตีเป้าหมายที่แท้จริง โดยที่ศัตรูยังคงถูกหลอกล่ออยู่
### 3. **ช่วงปลายเกม: "ยึดจริง" ด้วยกำลัง – เข้าหนีบ "หยก" อย่างเด็ดขาด**
* **วิธีการเดินหมาก:** เมื่อศัตรูหลงติดกับ "กระเบื้อง" อย่างเต็มที่ และหมากของคุณพร้อมแล้ว **ให้คุณเปิดฉากโจมตีเป้าหมายที่แท้จริง ("หยก") อย่างรวดเร็ว เด็ดขาด และรุนแรง** ใช้ความสามารถในการรุกและหมากที่ประสานงานกันเข้า "หนีบ" กินหมากที่มีค่ากว่า หรือเข้ายึดพื้นที่สำคัญที่ศัตรูละเลยไป การโจมตีนี้ควรเกิดขึ้นอย่างฉับพลัน เพื่อไม่ให้ศัตรูมีโอกาสปรับตัวหรือป้องกันได้ทันท่วงที
* **ยึดจริงทีหลังแบบไหนบ้าง:** การ "ยึดจริง" คือการใช้หมากหลักเข้าโจมตีหรือปิดล้อมหมากที่มีคุณค่าสูงกว่า (หยก) หรือพื้นที่เชิงยุทธศาสตร์สำคัญที่เปิดเผยออกมาหลังจากศัตรูเบนความสนใจไปที่ "กระเบื้อง" ทำให้คุณได้เปรียบอย่างเด็ดขาด
* **วัตถุประสงค์:** เพื่อเปลี่ยนการลงทุนเล็กน้อย (กระเบื้อง) ให้เป็นการได้มาซึ่งผลประโยชน์มหาศาล (หยก) ในช่วงเวลาที่เหมาะสมที่สุด
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
ยอมลงทุนด้วยสิ่งที่ดูเหมือนไม่สำคัญ หรือสร้างสถานการณ์ที่ศัตรูคิดว่าจะได้เปรียบเพียงเล็กน้อย** เพื่อล่อให้พวกเขาหลงกลและเบนความสนใจออกจากเป้าหมายที่แท้จริง จากนั้นจึง**ฉวยโอกาสเข้ายึดหมากหรือพื้นที่ที่มีความสำคัญมากกว่า**อย่างเด็ดขาด
### 1. **ช่วงเริ่มต้น: วาง "กระเบื้อง" ล่อ – ใช้สิ่งไม่สำคัญเบี่ยงเบนความสนใจ**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณ**เสนอ "กระเบื้อง"** ซึ่งอาจเป็นหมากที่มีค่าน้อย, การยอมเสียพื้นที่เล็กน้อย, หรือการเดินหมากที่ดูเหมือนจะให้ฝ่ายตรงข้ามได้เปรียบเพียงเล็กน้อย **โดยมีจุดประสงค์เพื่อดึงดูดความสนใจของศัตรูและทำให้พวกเขาเข้าใจผิด**ว่านี่คือเป้าหมายหลัก
* **ใช้สิ่งไม่สำคัญล่อให้หลงยังไง:** คุณกำลังใช้หมากหรือการกระทำที่ดูเหมือนไม่สลักสำคัญ เพื่อ**สร้างความดึงดูดใจหรือความเชื่อมั่นจอมปลอม** ให้ศัตรูโฟกัสไปที่ "กระเบื้อง" แทนที่จะเป็น "หยก" ที่ซ่อนอยู่
* **วัตถุประสงค์:** เพื่อเบี่ยงเบนความสนใจและกำลังของศัตรูไปยังเป้าหมายลวง ทำให้เป้าหมายที่แท้จริง (หยก) ปลอดภัยและพร้อมสำหรับการโจมตี
### 2. **ช่วงกลางเกม: เคลื่อนพลสู่ "หยก" – จัดทัพลับไปยังเป

In [271]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: โยนกระเบื้องล่อหยก
Category:    กลยุทธ์เข้าตี
Description: ใช้สิ่งไม่สำคัญล่อให้หลง ยึดจริงทีหลัง

Metrics:
  Semantic Similarity:    0.5721
  6C Similarity:          0.9969
  Selection Score:        0.7420
  Confidence Score:       0.7505
  KL Divergence:          0.0032

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2005,0.2123,0.9443
ความสามารถในการรับ,0.1770,0.1695,0.9576
การประสานงานระหว่างหมาก,0.1564,0.1783,0.8770
ทรัพยากรที่ยังไม่ใช้,0.1508,0.1477,0.9798
ความได้เปรียบด้านเวลา,0.1609,0.1423,0.8842
ความเหมาะสมกับสถานการณ์,0.1545,0.1499,0.9704


# จับโจรเอาหัวโจก

In [272]:
# Cell 18: Example Usage
test_query =  """
โฟกัสที่หมากตัวกลางที่คุมเส้น หากถอดหัวโจกออกได้ จะทำให้โครงสร้างฝ่ายตรงข้ามพังทลายตามมาอย่างรวดเร็ว
### 1. **ต้นเกม – ระบุตำแหน่ง “หัวโจก”**
* สังเกตว่าหมากตัวใด “เชื่อม” แนวรับ–รุกของศัตรู (เช่น ตัวกลางที่คุมเส้น)
* จับตาดูว่าเมื่อหมากตัวนี้ถูกล้อม ช่องว่างรอบ ๆ จะเกิดหรือไม่
### 2. **กลางเกม – แยกผู้นำออกจากกำลังช่วยเหลือ**
* ใช้หมากรองหรือหมากสำรองกดดันให้เขา **ถอนกำลัง** มาปกป้องหัวโจก
* ดึงหมากสนับสนุนรอบ ๆ หลุดออกจากแนวกัน ทำให้หัวโจกโดดเดี่ยว
### 3. **ปลายเกม – ล้อม–หนีบ “หัวโจก”**
* เมื่อหัวโจกโดดเดี่ยว ให้ส่งหมากหลักสองตัวไป **ประกบ** รอบ ๆ
* ใช้ทั้งการหนีบสองตัว หรือสอดตัวเดียวระหว่างสองตัว **กินหมากหัวโจก** ทันที
* การสูญเสียผู้นำจะทำให้แนวรับศัตรู **แตกกระจาย** และคุณสามารถ **กวาดกิน** หมายอื่นในต่อเนื่องได้ง่ายขึ้น
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
โฟกัสที่หมากตัวกลางที่คุมเส้น หากถอดหัวโจกออกได้ จะทำให้โครงสร้างฝ่ายตรงข้ามพังทลายตามมาอย่างรวดเร็ว
### 1. **ต้นเกม – ระบุตำแหน่ง “หัวโจก”**
* สังเกตว่าหมากตัวใด “เชื่อม” แนวรับ–รุกของศัตรู (เช่น ตัวกลางที่คุมเส้น)
* จับตาดูว่าเมื่อหมากตัวนี้ถูกล้อม ช่องว่างรอบ ๆ จะเกิดหรือไม่
### 2. **กลางเกม – แยกผู้นำออกจากกำลังช่วยเหลือ**
* ใช้หมากรองหรือหมากสำรองกดดันให้เขา **ถอนกำลัง** มาปกป้องหัวโจก
* ดึงหมากสนับสนุนรอบ ๆ หลุดออกจากแนวกัน ทำให้หัวโจกโดดเดี่ยว
### 3. **ปลายเกม – ล้อม–หนีบ “หัวโจก”**
* เมื่อหัวโจกโดดเดี่ยว ให้ส่งหมากหลักสองตัวไป **ประกบ** รอบ ๆ
* ใช้ทั้งการหนีบสองตัว หรือสอดตัวเดียวระหว่างสองตัว **กินหมากหัวโจก** ทันที
* การสูญเสียผู้นำจะทำให้แนวรับศัตรู **แตกกระจาย** และคุณสามารถ **กวาดกิน** หมายอื่นในต่อเนื่องได้ง่ายขึ้น


1. จับโจรเอาหัวโจก
   Score: 0.6844
   Confidence: 0.6947
   Similarity: 0.4802

2. ล่อเสือออกจากถ้ำ
   Score: 0.6818
   Confidence: 0.6922
   Similarity: 0.4741

3. ดูไฟชายฝั่ง
   Score: 0.6800
   Confidence: 0.6906
   Similarity: 0.4681

4. ส่งเสียง

In [273]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: จับโจรเอาหัวโจก
Category:    กลยุทธ์เข้าตี
Description: มุ่งโจมตีผู้นำศัตรูให้ล่มทั้งกอง

Metrics:
  Semantic Similarity:    0.4802
  6C Similarity:          0.9909
  Selection Score:        0.6844
  Confidence Score:       0.6947
  KL Divergence:          0.0093

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1991,0.2220,0.8967
ความสามารถในการรับ,0.1810,0.1687,0.9320
การประสานงานระหว่างหมาก,0.1694,0.2081,0.8141
ทรัพยากรที่ยังไม่ใช้,0.1602,0.1516,0.9461
ความได้เปรียบด้านเวลา,0.1529,0.1246,0.8149
ความเหมาะสมกับสถานการณ์,0.1375,0.1251,0.9100


# ถอนฟืนใต้กระทะ

In [274]:
# Cell 18: Example Usage
test_query =  """
ชัยชนะเล็ก ๆ แต่ต่อกำลังใจฝ่ายตรงข้ามจนแทบไม่เหลือแรงสู้ ก่อนคุณจะตัดสินเกมได้ในที่สุด
### 1. **ต้นเกม – บดด้วยชัยชนะเล็กน้อย**
* **หนีบกินหมากรองทีละตัว** เช่น จับกินหมากตัวเล็ก ๆ ของเขาในเทิร์นที่เขาไม่ทันตั้งตัว
* ให้เขารู้สึกว่า **ทุกครั้งที่พยายามสู้ เขาก็แพ้อย่างง่ายดาย**
### 2. **กลางเกม – กดดันเชิงจิตใจ**
* **ขยับชิมเชิงแล้วหนีบกินต่อเนื่อง** ไม่ให้เขาได้โอกาสตั้งหลัก
* เลี่ยงการปะทะใหญ่ แต่ทำให้เขา **แพ้ซ้ำซาก** จนเกิดความท้อ
### 3. **ปลายเกม – พังทลายกำลังใจ**
* เมื่อเขาเริ่ม **ลังเลหรือถอยหนี** ให้ใช้ **หนีบสองตัว** หรือ **สอดตัวเดียวระหว่างสองตัว** จับกินหมากหลัก (ตัวสำคัญ)
* การสูญเสียหมากหลักจะทำให้แนวรับเขา **แตกพัง** และเขา **ยอมแพ้โดยไม่มีแรงสู้ต่อ**
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
ชัยชนะเล็ก ๆ แต่ต่อกำลังใจฝ่ายตรงข้ามจนแทบไม่เหลือแรงสู้ ก่อนคุณจะตัดสินเกมได้ในที่สุด
### 1. **ต้นเกม – บดด้วยชัยชนะเล็กน้อย**
* **หนีบกินหมากรองทีละตัว** เช่น จับกินหมากตัวเล็ก ๆ ของเขาในเทิร์นที่เขาไม่ทันตั้งตัว
* ให้เขารู้สึกว่า **ทุกครั้งที่พยายามสู้ เขาก็แพ้อย่างง่ายดาย**
### 2. **กลางเกม – กดดันเชิงจิตใจ**
* **ขยับชิมเชิงแล้วหนีบกินต่อเนื่อง** ไม่ให้เขาได้โอกาสตั้งหลัก
* เลี่ยงการปะทะใหญ่ แต่ทำให้เขา **แพ้ซ้ำซาก** จนเกิดความท้อ
### 3. **ปลายเกม – พังทลายกำลังใจ**
* เมื่อเขาเริ่ม **ลังเลหรือถอยหนี** ให้ใช้ **หนีบสองตัว** หรือ **สอดตัวเดียวระหว่างสองตัว** จับกินหมากหลัก (ตัวสำคัญ)
* การสูญเสียหมากหลักจะทำให้แนวรับเขา **แตกพัง** และเขา **ยอมแพ้โดยไม่มีแรงสู้ต่อ**


1. ถอนฟืนใต้กระทะ
   Score: 0.6954
   Confidence: 0.7053
   Similarity: 0.4971

2. หลี่ตายแทนถาว
   Score: 0.6792
   Confidence: 0.6897
   Similarity: 0.4681

3. จูงแพะติดมือ
   Score: 0.6668
   Confidence: 0.6777
   Similarity: 0.4479

4. ดูไฟชายฝั่ง
   Score: 0.6606
   Confidence: 0.6717
   Similarity: 0.4386



In [275]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ถอนฟืนใต้กระทะ
Category:    กลยุทธ์ติดพัน
Description: ตัดกำลังใจศัตรูให้หมดแรงสู้

Metrics:
  Semantic Similarity:    0.4971
  6C Similarity:          0.9929
  Selection Score:        0.6954
  Confidence Score:       0.7053
  KL Divergence:          0.0077

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2065,0.2095,0.9854
ความสามารถในการรับ,0.1671,0.1698,0.9843
การประสานงานระหว่างหมาก,0.1605,0.1960,0.8191
ทรัพยากรที่ยังไม่ใช้,0.1514,0.1522,0.9945
ความได้เปรียบด้านเวลา,0.1591,0.1265,0.7948
ความเหมาะสมกับสถานการณ์,0.1553,0.1460,0.9398


# กวนน้ำจับปลา

In [276]:
# Cell 18: Example Usage
test_query =  """
จงใจสร้างความสับสนวุ่นวาย หรือสถานการณ์ที่ซับซ้อนบนกระดาน** เพื่อทำให้ฝ่ายตรงข้ามมองไม่เห็นภาพรวม ตัดสินใจผิดพลาด หรือเสียสมาธิ จากนั้นจึง**ฉวยโอกาสจากความโกลาหลนั้นเข้าหนีบโจมตีอย่างเด็ดขาด**
### 1. **ช่วงเริ่มต้น: สร้าง "น้ำขุ่น" – ก่อความสับสนและแรงกดดัน**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณเริ่มเดินหมากที่**สร้างความซับซ้อนหลายมิติ** เช่น การสร้างภัยคุกคามพร้อมกันหลายจุด, การแลกหมากที่ไม่คาดคิด, หรือการเคลื่อนย้ายหมากที่ทำให้กระดานดูยุ่งเหยิง **จงใจทำให้สถานการณ์ดูไม่ชัดเจนและยากต่อการคาดเดา** เพื่อทำให้ศัตรูต้องใช้พลังงานในการวิเคราะห์มากเกินไป
* **วัตถุประสงค์:** เพื่อทำให้ฝ่ายตรงข้ามรู้สึกสับสนและมองเห็นสถานการณ์ไม่ชัดเจน นำไปสู่การตัดสินใจที่ผิดพลาดหรือการเสียเวลา
### 2. **ช่วงกลางเกม: เพิ่ม "ความวุ่นวาย" – บีบให้ศัตรูยิ่งสับสน**
* **วิธีการเดินหมาก:** ในขณะที่ศัตรูกำลังพยายามทำความเข้าใจสถานการณ์ที่ซับซ้อน ให้คุณ**เพิ่มแรงกดดันหรือสร้างความวุ่นวายเพิ่มเติม** เช่น การเดินหมากที่สร้างเงื่อนไขเวลาให้ฝ่ายตรงข้ามต้องรีบตัดสินใจ, การเปิดแนวรุกใหม่ที่คาดไม่ถึง, หรือการแลกหมากที่ทำให้หมากของศัตรูกระจัดกระจาย **จงใจทำให้พวกเขาวุ่นวายและเสียสมาธิจากการมองเห็นภัยคุกคามรอบด้าน**
* **ใช้อาการวุ่นวายของศัตรูให้เป็นประโยชน์ยังไง:** คุณใช้ประโยชน์จากความวุ่นวายที่เกิดขึ้นเพื่อทำให้ศัตรู**ตัดสินใจผิดพลาด**, **เคลื่อนหมากผิดตำแหน่ง**, หรือ**ละเลยจุดสำคัญ**
* **ใช้อาการวุ่นวายของศัตรูให้เป็นประโยชน์แบบไหนบ้าง:** คุณใช้ประโยชน์จาก**ความผิดพลาดในการวางตำแหน่งหมาก**, **การมองข้ามภัยคุกคาม**, **ความไม่สามารถประสานงานหมากได้**, หรือ**ความรีบร้อนภายใต้แรงกดดันด้านเวลา**
* **วัตถุประสงค์:** เพื่อบีบให้ฝ่ายตรงข้ามอยู่ในสภาวะที่สับสนอลหม่าน ตัดสินใจผิดพลาด และเผยจุดอ่อนออกมา
### 3. **ช่วงปลายเกม: "จับปลา" ใน "น้ำขุ่น" – เข้าหนีบอย่างเด็ดขาด**
* **วิธีการเดินหมาก:** เมื่อฝ่ายตรงข้ามอยู่ในภาวะวุ่นวายสูงสุด ไม่สามารถมองเห็นภาพรวมหรือตอบโต้ได้อย่างมีประสิทธิภาพ **ให้คุณเปิดฉากโจมตีอย่างรวดเร็วและเด็ดขาด** ใช้ความสามารถในการรุกและการประสานงานระหว่างหมากที่เตรียมมาอย่างเต็มที่ เพื่อเข้า "หนีบ" กินหมากสำคัญของศัตรูในจุดที่พวกเขาอ่อนแอที่สุด
* **วัตถุประสงค์:** เพื่อฉวยโอกาสจากความสับสนของฝ่ายตรงข้าม เข้ายึดหมากหรือพื้นที่สำคัญ และนำไปสู่ชัยชนะอย่างรวดเร็ว
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
จงใจสร้างความสับสนวุ่นวาย หรือสถานการณ์ที่ซับซ้อนบนกระดาน** เพื่อทำให้ฝ่ายตรงข้ามมองไม่เห็นภาพรวม ตัดสินใจผิดพลาด หรือเสียสมาธิ จากนั้นจึง**ฉวยโอกาสจากความโกลาหลนั้นเข้าหนีบโจมตีอย่างเด็ดขาด**
### 1. **ช่วงเริ่มต้น: สร้าง "น้ำขุ่น" – ก่อความสับสนและแรงกดดัน**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้นของเกม ให้คุณเริ่มเดินหมากที่**สร้างความซับซ้อนหลายมิติ** เช่น การสร้างภัยคุกคามพร้อมกันหลายจุด, การแลกหมากที่ไม่คาดคิด, หรือการเคลื่อนย้ายหมากที่ทำให้กระดานดูยุ่งเหยิง **จงใจทำให้สถานการณ์ดูไม่ชัดเจนและยากต่อการคาดเดา** เพื่อทำให้ศัตรูต้องใช้พลังงานในการวิเคราะห์มากเกินไป
* **วัตถุประสงค์:** เพื่อทำให้ฝ่ายตรงข้ามรู้สึกสับสนและมองเห็นสถานการณ์ไม่ชัดเจน นำไปสู่การตัดสินใจที่ผิดพลาดหรือการเสียเวลา
### 2. **ช่วงกลางเกม: เพิ่ม "ความวุ่นวาย" – บีบให้ศัตรูยิ่งสับสน**
* **วิธีการเดินหมาก:** ในขณะที่ศัตรูกำลังพยายามทำความเข้าใจสถานการณ์ที่ซับซ้อน ให้คุณ**เพิ่มแรงกดดันหรือสร้างความวุ่นวายเพิ่มเติม** เช่น การเดินหมากที่สร้างเงื่อนไขเวลาให้ฝ่ายตรงข้ามต้องรีบตัดสินใจ, การเปิดแนวรุกใหม่ที่คาดไม่ถึง,

In [277]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ตีชิงตามไฟ
Category:    กลยุทธ์ชนะศึก
Description: ฉวยโอกาสตอนศัตรูกำลังวุ่นวาย

Metrics:
  Semantic Similarity:    0.5088
  6C Similarity:          0.9971
  Selection Score:        0.7041
  Confidence Score:       0.7139
  KL Divergence:          0.0030

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2003,0.1937,0.9669
ความสามารถในการรับ,0.1653,0.1604,0.9703
การประสานงานระหว่างหมาก,0.1613,0.1645,0.9803
ทรัพยากรที่ยังไม่ใช้,0.1416,0.1516,0.9341
ความได้เปรียบด้านเวลา,0.1535,0.1728,0.8883
ความเหมาะสมกับสถานการณ์,0.1780,0.1570,0.8820


# จักจั่นลอกคราบ

In [278]:
# Cell 18: Example Usage
test_query =  """
ให้หมากบางตัว *แกล้งอยู่นิ่ง* จนเหมือนตายสนิท แล้วจู่ๆ ก็พลิกสถานการณ์ขึ้นมาเป็นแนวโจมตีหลักแบบไม่ให้ตั้งตัว — เหมือนจักจั่นที่ทิ้งร่างเก่าไว้ ก่อนโผล่มาอีกที่หนึ่งอย่างอิสระ
กลยุทธ์นี้ต่างจากกลยุทธ์ลวงทั่วไป เพราะไม่ได้ “หลอกเป้าหมาย” แต่หลอกว่า “ไม่มีแผนอะไรเลย” — แล้วจึง *เปลี่ยนสถานะจากเฉื่อยเป็นหนีบทันที*
---
### 1. **ต้นเกม – แสร้งไม่ขยับ**
* วางหมากแนวรับ–แนวหลังให้อยู่ในตำแหน่ง **ไม่เคลื่อนไหวชัดเจน**
* เดินแต่ละเทิร์นแบบ *ไม่ก้าวรุก* เช่น วางซ้อนแนวหรือหมุนเฉพาะในพื้นที่ของตัวเอง
* เป้าหมายคือให้ศัตรูรู้สึกว่า “คุณยังไม่พร้อมบุก” และคลายความระแวดระวัง
---
### 2. **กลางเกม – ตีเนียนต่อเนื่อง**
* เคลื่อนไหวแผ่วเบาเท่านั้น เช่น ขยับกลับไปตำแหน่งเดิม หรือหมุนแนวหมากแต่ไม่สร้างภัยคุกคาม
* ใช้หมากรองหรือหมากที่เหมือน “ถูกลืม” ขยับอย่างเชื่องช้า
* หลอกศัตรูให้เคลื่อนกำลังไปอีกฝั่ง หรือประมาทแนวรับคุณ
---
### 3. **ปลายเกม – ลอกคราบจู่โจม**
* เปลี่ยนแผนทันที: ใช้หมากที่ซุ่มอยู่ “ปะทะซ้อน” หรือ “หนีบสวน” จากแนวที่ดูเฉื่อยชา
* ส่งหมากจากแนวหลังขึ้นหน้า แล้วใช้การหนีบสองด้านกับแนวเดิมที่เขาไม่คิดว่าอันตราย
* ใช้ช่องที่เขาปล่อยทิ้งไว้หนีบกินรวดเดียวหลายตัวในเทิร์นเดียว
---
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
ให้หมากบางตัว *แกล้งอยู่นิ่ง* จนเหมือนตายสนิท แล้วจู่ๆ ก็พลิกสถานการณ์ขึ้นมาเป็นแนวโจมตีหลักแบบไม่ให้ตั้งตัว — เหมือนจักจั่นที่ทิ้งร่างเก่าไว้ ก่อนโผล่มาอีกที่หนึ่งอย่างอิสระ
กลยุทธ์นี้ต่างจากกลยุทธ์ลวงทั่วไป เพราะไม่ได้ “หลอกเป้าหมาย” แต่หลอกว่า “ไม่มีแผนอะไรเลย” — แล้วจึง *เปลี่ยนสถานะจากเฉื่อยเป็นหนีบทันที*
---
### 1. **ต้นเกม – แสร้งไม่ขยับ**
* วางหมากแนวรับ–แนวหลังให้อยู่ในตำแหน่ง **ไม่เคลื่อนไหวชัดเจน**
* เดินแต่ละเทิร์นแบบ *ไม่ก้าวรุก* เช่น วางซ้อนแนวหรือหมุนเฉพาะในพื้นที่ของตัวเอง
* เป้าหมายคือให้ศัตรูรู้สึกว่า “คุณยังไม่พร้อมบุก” และคลายความระแวดระวัง
---
### 2. **กลางเกม – ตีเนียนต่อเนื่อง**
* เคลื่อนไหวแผ่วเบาเท่านั้น เช่น ขยับกลับไปตำแหน่งเดิม หรือหมุนแนวหมากแต่ไม่สร้างภัยคุกคาม
* ใช้หมากรองหรือหมากที่เหมือน “ถูกลืม” ขยับอย่างเชื่องช้า
* หลอกศัตรูให้เคลื่อนกำลังไปอีกฝั่ง หรือประมาทแนวรับคุณ
---
### 3. **ปลายเกม – ลอกคราบจู่โจม**
* เปลี่ยนแผนทันที: ใช้หมากที่ซุ่มอยู่ “ปะทะซ้อน” หรือ “หนีบสวน” จากแนวที่ดูเฉื่อยชา
* ส่งหมากจากแนวหลังขึ้นหน้า แล้วใช้การหนีบสองด้านกับแนว

In [279]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: จักจั่นลอกคราบ
Category:    กลยุทธ์ติดพัน
Description: ทำเหมือนไม่ขยับ ก่อนเปลี่ยนแผนฉับพลัน

Metrics:
  Semantic Similarity:    0.5333
  6C Similarity:          0.9965
  Selection Score:        0.7186
  Confidence Score:       0.7279
  KL Divergence:          0.0036

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2060,0.2073,0.9936
ความสามารถในการรับ,0.1919,0.1794,0.9346
การประสานงานระหว่างหมาก,0.1625,0.1889,0.8600
ทรัพยากรที่ยังไม่ใช้,0.1458,0.1313,0.9005
ความได้เปรียบด้านเวลา,0.1533,0.1450,0.9457
ความเหมาะสมกับสถานการณ์,0.1406,0.1482,0.9487


# ปิดประตูจับโจร

In [280]:
# Cell 18: Example Usage
test_query =  """
แนวคิด*ล้อมรอบจนศัตรูหนีไม่ได้*
---
## 1. ช่วงเริ่มต้น – เปิด “ประตู” ล่อติดพัน
* **เดินหมากล่อหลายจุดเล็กๆ พร้อมกัน** (เช่น เปิดช่อง A กับช่อง B สลับกัน)
* **สร้างแรงกดดันจิ๋ว** ให้ศัตรูต้องเดินมาเคลียร์ทั้งสองจุด สลับตอบสนองไปมา
* **ผลลัพธ์:** ศัตรู “ติดพัน” กับการตอบโต้จุดเล็ก ๆ สร้างจังหวะให้คุณควบคุมพื้นที่หลักได้
---
## 2. ช่วงกลางเกม – ขยายวงติดพัน & ปิด “ประตู”
* **เคลื่อนหมากเติมช่องตอบโต้** ทุกครั้งที่ศัตรูพยายามหลุดจากวงล้อม ยืดเวลาให้ระบบล้อมแน่นขึ้น
* **ใช้หมากหลายตัวร่วมมือ** สร้างแนวล้อมชั้นใน–ชั้นนอกสลับกัน ให้ศัตรูต้องแก้หลายขั้นตอน
* **ผลลัพธ์:** ศัตรูยังคง “ติดพัน” ในการหาทางหลบทุกชั้น จนไม่ทันระวังช่องทางหลบสุดท้าย
---
## 3. ช่วงปลายเกม – ปิด “ประตูสุดท้าย” แล้วหนีบกิน
* **เก็บหมากสำรองเข้าประสาน** ทันทีที่ช่องเล็ก ๆ ถูกเติมเต็มจนไม่มีช่องให้วิ่งหลุด
* **ประกบสองฝั่งหนีบพร้อมกัน** ใช้แรงร่วมของหมากหลายตัว ปิดทุกทางหนีเหลือแต่จุดเดียว
* **หนีบ (capture) ครั้งเดียว** กินหมากที่ถูกติดพันล้อมในวงกลมสุดท้าย
---
### สรุป “ติดพัน”
1. **ล่อหลายจุด:** ให้ศัตรูต้องวิ่งตอบโต้ไม่หยุด
2. **ขยายวงล้อมเป็นชั้น:** รวบศัตรูไว้ให้หาทางหลุดยาก
3. **ปิดประตูสุดท้าย:** จับในตาเดียว เมื่อศัตรูยังคงติดพันจากชั้นแรก
คำอธิบายนี้เน้นการ **จับศัตรูให้อยู่ในภาวะ “ติดพัน”** กับหลายขั้นตอน ก่อนตัดทางหนีสุดท้ายและหนีบกิน
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
แนวคิด*ล้อมรอบจนศัตรูหนีไม่ได้*
---
## 1. ช่วงเริ่มต้น – เปิด “ประตู” ล่อติดพัน
* **เดินหมากล่อหลายจุดเล็กๆ พร้อมกัน** (เช่น เปิดช่อง A กับช่อง B สลับกัน)
* **สร้างแรงกดดันจิ๋ว** ให้ศัตรูต้องเดินมาเคลียร์ทั้งสองจุด สลับตอบสนองไปมา
* **ผลลัพธ์:** ศัตรู “ติดพัน” กับการตอบโต้จุดเล็ก ๆ สร้างจังหวะให้คุณควบคุมพื้นที่หลักได้
---
## 2. ช่วงกลางเกม – ขยายวงติดพัน & ปิด “ประตู”
* **เคลื่อนหมากเติมช่องตอบโต้** ทุกครั้งที่ศัตรูพยายามหลุดจากวงล้อม ยืดเวลาให้ระบบล้อมแน่นขึ้น
* **ใช้หมากหลายตัวร่วมมือ** สร้างแนวล้อมชั้นใน–ชั้นนอกสลับกัน ให้ศัตรูต้องแก้หลายขั้นตอน
* **ผลลัพธ์:** ศัตรูยังคง “ติดพัน” ในการหาทางหลบทุกชั้น จนไม่ทันระวังช่องทางหลบสุดท้าย
---
## 3. ช่วงปลายเกม – ปิด “ประตูสุดท้าย” แล้วหนีบกิน
* **เก็บหมากสำรองเข้าประสาน** ทันทีที่ช่องเล็ก ๆ ถูกเติมเต็มจนไม่มีช่องให้วิ่งหลุด
* **ประกบสองฝั่งหนีบพร้อมกัน** ใช้แรงร่วมของหมากหลายตัว ปิดทุกทางหนีเหลือแต่จุดเดียว
* **หนีบ (capture) ครั้งเดียว** กินหมากที่ถูกติดพันล้อมในวงกลมสุดท้าย
---
### สรุป “ติดพัน”
1. **ล่อหลายจุด:** ให้ศัตรูต้องวิ่

In [281]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ปิดประตูจับโจร
Category:    กลยุทธ์ติดพัน
Description: ล้อมรอบจนศัตรูหนีไม่ได้

Metrics:
  Semantic Similarity:    0.5905
  6C Similarity:          0.9945
  Selection Score:        0.7521
  Confidence Score:       0.7601
  KL Divergence:          0.0064

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2034,0.2153,0.9451
ความสามารถในการรับ,0.1988,0.1922,0.9671
การประสานงานระหว่างหมาก,0.1791,0.2016,0.8883
ทรัพยากรที่ยังไม่ใช้,0.1191,0.1354,0.8790
ความได้เปรียบด้านเวลา,0.1481,0.1273,0.8595
ความเหมาะสมกับสถานการณ์,0.1516,0.1282,0.8458


# คบไกลตีใกล้

In [282]:
# Cell 18: Example Usage
test_query =  """
---
## 1. ต้นเกม – ผูกไมตรีกับที่ไกล
* **ผูกไมตรีกับที่ไกลยังไง:**
  1. **ส่งหมากสำรวจระยะไกล** ออกไปในโซนที่ห่างจากแนวหมากหลักของคุณ (2–3 แถวหลังแนวแรก)
  2. **วางหมากแบบไม่กดดัน** แม้จะอยู่ในแนวตรงสายเดียวกับศัตรู ให้ดูเหมือนคุณแค่ “ขยายขอบเขต” ไม่ได้ตั้งใจบุก
  3. **สลับจังหวะเดินน้อยๆ** เพื่อรักษาสถานะเป็นมิตร (ไม่ดันลึกเกินไป) และสร้างความเชื่อว่าโซนนี้ปลอดภัย
> *ผลลัพธ์:* ฝ่ายตรงข้ามจะลดความระแวดระวัง และโฟกัสที่แนวหน้า–กลาง มากกว่าโซนไกลที่คุณตั้งใจให้ “คบ” ไว้
---
## 2. กลางเกม – รักษาความสัมพันธ์ระยะไกล
* **วางหมากไกลเป็นแนวสำรอง** ให้ยังคงคุมทางเลี้ยวหรือช่องมุมบางจุด แต่ไม่แสดงท่าทีบุกหนัก
* **เติมหมากเล็กน้อยแต่สม่ำเสมอ** ตรงโซนไกล เพื่อให้ตำแหน่งนั้นดูมีประโยชน์ (defensive) มากกว่า offensive
* **อย่าเดินไล่หรือเปลี่ยนแนวไกล** ให้ฝ่ายตรงข้ามมั่นใจว่าพื้นที่นี้ยังไม่มีอันตราย
> *ผลลัพธ์:* ฝ่ายตรงข้ามจะโฟกัสจัดแนวปิดล้อมโซนหน้า–กลาง โดยไม่สนใจแนวไกลที่คุณเก็บไว้เป็น “มิตร”
---
## 3. ปลายเกม – ตีที่ใกล้
* **ปลดหมากสำรองจากโซนไกล** เดินเข้าประสานกับหมากหลักที่อยู่ใกล้ศัตรู
* **สร้างรูปแบบหนีบสองฝั่ง** ใกล้ตัวศัตรู (ช่องรอบๆ) ใช้หมากไกล+หมากหน้า ประกบฝ่ายตรงข้าม
* **สลับเดินหมากให้จังหวะเร็ว** เพื่อปิดช่องทางหนีทันทีที่เริ่มเข้าประชิด
* เมื่อทุกทางออกใกล้หมด ให้ใช้รูปแบบ **หนีบ (capture)** กินหมากฝ่ายตรงข้ามในพื้นที่กลางแบบเด็ดขาด

> **ตีที่ใกล้แบบไหนบ้าง:**
>
> 1. **จากแถวสำรองขยับขึ้น** มาเป็นแนวร่วมปิดช่องเฉียง
> 2. **ใช้หมากไกลเฉียงประกบ** กับหมากหน้าในแนวตรง
> 3. **จังหวะเดียวหนีบกิน** เมื่อช่องรอบตัวศัตรูถูกรัดแน่น
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
## 1. ต้นเกม – ผูกไมตรีกับที่ไกล
* **ผูกไมตรีกับที่ไกลยังไง:**
  1. **ส่งหมากสำรวจระยะไกล** ออกไปในโซนที่ห่างจากแนวหมากหลักของคุณ (2–3 แถวหลังแนวแรก)
  2. **วางหมากแบบไม่กดดัน** แม้จะอยู่ในแนวตรงสายเดียวกับศัตรู ให้ดูเหมือนคุณแค่ “ขยายขอบเขต” ไม่ได้ตั้งใจบุก
  3. **สลับจังหวะเดินน้อยๆ** เพื่อรักษาสถานะเป็นมิตร (ไม่ดันลึกเกินไป) และสร้างความเชื่อว่าโซนนี้ปลอดภัย
> *ผลลัพธ์:* ฝ่ายตรงข้ามจะลดความระแวดระวัง และโฟกัสที่แนวหน้า–กลาง มากกว่าโซนไกลที่คุณตั้งใจให้ “คบ” ไว้
---
## 2. กลางเกม – รักษาความสัมพันธ์ระยะไกล
* **วางหมากไกลเป็นแนวสำรอง** ให้ยังคงคุมทางเลี้ยวหรือช่องมุมบางจุด แต่ไม่แสดงท่าทีบุกหนัก
* **เติมหมากเล็กน้อยแต่สม่ำเสมอ** ตรงโซนไกล เพื่อให้ตำแหน่งนั้นดูมีประโยชน์ (defensive) มากกว่า offensive
* **อย่าเดินไล่หรือเปลี่ยนแนวไกล** ให้ฝ่ายตรงข้ามมั่นใจว่าพื้นที่นี้ยังไม่มีอันตราย
> *ผลลัพธ์:* ฝ่ายตรงข้ามจะโฟกัสจัดแนวปิดล้อมโซนหน้า–กลาง โดยไม่สนใจแนวไกลที่คุณเก็บไว้เป็น “มิตร”
---
## 3. ปลายเกม – ตีที่ใกล้
* **ปลดหมากสำรองจากโซนไกล** เดินเข้าประสานกับหมากหลักที่อยู่ใกล้ศั

In [283]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: คบไกลตีใกล้
Category:    กลยุทธ์ติดพัน
Description: ผูกไมตรีกับที่ไกล แล้วตีที่ใกล้

Metrics:
  Semantic Similarity:    0.6971
  6C Similarity:          0.9986
  Selection Score:        0.8177
  Confidence Score:       0.8238
  KL Divergence:          0.0014

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1987,0.1993,0.9967
ความสามารถในการรับ,0.2065,0.1950,0.9444
การประสานงานระหว่างหมาก,0.1632,0.1791,0.9114
ทรัพยากรที่ยังไม่ใช้,0.1361,0.1337,0.9822
ความได้เปรียบด้านเวลา,0.1601,0.1521,0.9499
ความเหมาะสมกับสถานการณ์,0.1354,0.1408,0.9616


# ยืมทางพรางกล

In [284]:
# Cell 18: Example Usage
test_query =  """
## 1. ต้นเกม – ยืมทางสร้างโอกาส
* **เปิดทางเล็ก ๆ** โดยเดินหมากให้ดูเหมือนให้ช่องทางแถวหนึ่งหรือแถวสองเปิดโล่ง (เลื่อนหมากแนวหน้าไปข้างหนึ่ง)
* **สร้างความเชื่อว่าคุณไม่สนใจช่องนั้น** ไม่เติมหมากเสริม ให้ศัตรูคิดว่า “ถนนเส้นนี้ปลอดภัย”
* **ใช้ช่องทางยืมทาง** ให้หมากสำรองของคุณฉวยโอกาสเคลื่อนผ่านเข้ามาใกล้แนวกลาง
> *ผลลัพธ์:* ศัตรูจะพะวงไปปิดล้อมแนวตรงหลัก ไม่ทันระวังว่าเส้นทางสำรองกำลังถูกใช้งาน
---
## 2. กลางเกม – เสริมเส้นทางพรางกล
* **สลับหมากเข้า–ออกเส้นทางยืม** ระหว่างหมากหน้ากับหมากรอง เพื่อรักษาช่องทางไม่ถูกปิด
* **เติมหมากคุมช่องเฉียงเล็กน้อย** ให้แนวยืมทางดู defensively ปลอดภัย ไม่ใช่ช่องโจมตีตรง ๆ
* **แสดงความสนใจน้อยที่สุด** ไม่เดินหมากลึกเกินไป จนฝ่ายตรงข้ามมองข้ามว่าเป็นจุดบุกจริง
> *ผลลัพธ์:* เส้นทางสำรองของคุณจะอยู่ใต้เรดาร์ ฝ่ายตรงข้ามไม่เสียหมากมาปิด แต่คุณเตรียมพร้อมโจมตี
---
## 3. ปลายเกม – โจมตีผ่านเส้นทางยืม
* เมื่อแนวหลักยังถูกศัตรูปิดล้อมแน่น ให้ **ปลดหมากสำรอง** จากเส้นทางพรางกลวิ่งเข้าช่องว่างของศัตรู
* **ใช้รูปแบบหนีบ** โดยให้หมากจากเส้นทางยืมทางกับหมากหน้าเดิมประกบสองฝั่ง จับหมากกลางล้อม
* **จังหวะเดียวเก็บกิน** เมื่อช่องตรงกลางถูกรัดซ้ำสองชั้น ศัตรูจะไม่มีทางหนี
> **โจมตีแบบไหน:**
>
> * จากแนวรองเลี้ยงเข้าประกบแถวหน้า
> * ใช้หมากสองฝั่งยืมทางเข้าจับตรงช่วงแคบ
> * สลับจังหวะให้โอกาสโจมตีไวในเทิร์นเดียว
---
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
## 1. ต้นเกม – ยืมทางสร้างโอกาส
* **เปิดทางเล็ก ๆ** โดยเดินหมากให้ดูเหมือนให้ช่องทางแถวหนึ่งหรือแถวสองเปิดโล่ง (เลื่อนหมากแนวหน้าไปข้างหนึ่ง)
* **สร้างความเชื่อว่าคุณไม่สนใจช่องนั้น** ไม่เติมหมากเสริม ให้ศัตรูคิดว่า “ถนนเส้นนี้ปลอดภัย”
* **ใช้ช่องทางยืมทาง** ให้หมากสำรองของคุณฉวยโอกาสเคลื่อนผ่านเข้ามาใกล้แนวกลาง
> *ผลลัพธ์:* ศัตรูจะพะวงไปปิดล้อมแนวตรงหลัก ไม่ทันระวังว่าเส้นทางสำรองกำลังถูกใช้งาน
---
## 2. กลางเกม – เสริมเส้นทางพรางกล
* **สลับหมากเข้า–ออกเส้นทางยืม** ระหว่างหมากหน้ากับหมากรอง เพื่อรักษาช่องทางไม่ถูกปิด
* **เติมหมากคุมช่องเฉียงเล็กน้อย** ให้แนวยืมทางดู defensively ปลอดภัย ไม่ใช่ช่องโจมตีตรง ๆ
* **แสดงความสนใจน้อยที่สุด** ไม่เดินหมากลึกเกินไป จนฝ่ายตรงข้ามมองข้ามว่าเป็นจุดบุกจริง
> *ผลลัพธ์:* เส้นทางสำรองของคุณจะอยู่ใต้เรดาร์ ฝ่ายตรงข้ามไม่เสียหมากมาปิด แต่คุณเตรียมพร้อมโจมตี
---
## 3. ปลายเกม – โจมตีผ่านเส้นทางยืม
* เมื่อแนวหลักยังถูกศัตรูปิดล้อมแน่น ให้ **ปลดหมากสำรอง** จากเส้นทางพรางกลวิ่งเข้าช่องว่างของศัตรู
* **ใช้รูปแบบหนีบ** โดยให้หมากจากเส้นทางยืมทางกับหมา

In [285]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ยืมทางพรางกล
Category:    กลยุทธ์ติดพัน
Description: ขอผ่านแล้วใช้เส้นทางโจมตี

Metrics:
  Semantic Similarity:    0.5350
  6C Similarity:          0.9923
  Selection Score:        0.7180
  Confidence Score:       0.7271
  KL Divergence:          0.0080

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1897,0.2099,0.9037
ความสามารถในการรับ,0.1861,0.1689,0.9074
การประสานงานระหว่างหมาก,0.1525,0.1798,0.8482
ทรัพยากรที่ยังไม่ใช้,0.1483,0.1456,0.9818
ความได้เปรียบด้านเวลา,0.1686,0.1353,0.8024
ความเหมาะสมกับสถานการณ์,0.1548,0.1605,0.9641


# ลักขื่อเปลี่ยนเสา

In [286]:
# Cell 18: Example Usage
test_query =  """
## 1. ต้นเกม – ตั้งกับดักบังคับสลับ
* **สร้างภัยคุกคามที่จุด X** (เช่น ขยับหมากไปใกล้มุมหนึ่ง) ให้ศัตรูรู้สึกว่าต้องขยับหมากสำคัญมาป้องกัน
* **วางหมากล่อให้โจมตี** ในโซนที่คุณต้องการให้ศัตรูย้ายหมากหลัก (เช่น แถวกลาง ถ้าต้องการล่อให้ย้ายจากมุมมา)
* **รอให้ศัตรูสลับตำแหน่ง** ระหว่างหมากหน้า (สำคัญ) กับหมากรอง เพื่อปิดจุดอ่อนที่คุณสร้าง
> *ผลลัพธ์:* ศัตรูจะ “สลับเสา” ย้ายหมากสำคัญจากที่เดิมไปปิดจุดที่คุณกำหนด
---
## 2. กลางเกม – กดดันช่องว่างที่เกิดขึ้น
* **เติมหมากไปยังจุดเดิมของหมากสำคัญ** ที่ศัตรูเพิ่งสลับออกมา ให้เป็นช่องอ่อน
* **ประสานหมากประกบสองฝั่ง** รอบช่องอ่อนนั้น ทั้งแนวตรงและเฉียง
* **เดินหมากกดดันต่อเนื่อง** ให้ศัตรูรู้ว่าช่องเดิมโดนคุม ถ้าจะรักษาตำแหน่งต้องแลกหมากใหญ่
> *ผลลัพธ์:* ช่องว่างที่เกิดจากการ “เปลี่ยนเสา” จะกลายเป็นกับดักให้คุณโจมตี
---
## 3. ปลายเกม – หนีบกินจากจุดสลับ
* **ปิดช่องทางหนีรอบกลุ่มหมากอ่อน** ที่คุณล่อให้ศัตรูย้ายมา
* **ใช้รูปแบบหนีบ (capture)** ให้หมากสองข้าง (จากหน้า–หลัง หรือซ้าย–ขวา) ประกบรอบ เพื่อกินหมากสำคัญ
* **เก็บกินเด็ดขาด** ในเทิร์นเดียว เมื่อศัตรู “จนมุม” ในพื้นที่ที่เพิ่งสลับมา

> **สลับตำแหน่งสำคัญแบบไหนบ้าง:**
>
> 1. ให้ศัตรูย้ายหมากหน้าจากกลางบอร์ดไปมุม
> 2. ให้ย้ายหมากรองจากแนวหลังขึ้นมาปิดช่องกลาง
> 3. ให้แลกหมากเล็ก–ใหญ่ว่างช่องที่คุณเตรียมไว้
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
## 1. ต้นเกม – ตั้งกับดักบังคับสลับ
* **สร้างภัยคุกคามที่จุด X** (เช่น ขยับหมากไปใกล้มุมหนึ่ง) ให้ศัตรูรู้สึกว่าต้องขยับหมากสำคัญมาป้องกัน
* **วางหมากล่อให้โจมตี** ในโซนที่คุณต้องการให้ศัตรูย้ายหมากหลัก (เช่น แถวกลาง ถ้าต้องการล่อให้ย้ายจากมุมมา)
* **รอให้ศัตรูสลับตำแหน่ง** ระหว่างหมากหน้า (สำคัญ) กับหมากรอง เพื่อปิดจุดอ่อนที่คุณสร้าง
> *ผลลัพธ์:* ศัตรูจะ “สลับเสา” ย้ายหมากสำคัญจากที่เดิมไปปิดจุดที่คุณกำหนด
---
## 2. กลางเกม – กดดันช่องว่างที่เกิดขึ้น
* **เติมหมากไปยังจุดเดิมของหมากสำคัญ** ที่ศัตรูเพิ่งสลับออกมา ให้เป็นช่องอ่อน
* **ประสานหมากประกบสองฝั่ง** รอบช่องอ่อนนั้น ทั้งแนวตรงและเฉียง
* **เดินหมากกดดันต่อเนื่อง** ให้ศัตรูรู้ว่าช่องเดิมโดนคุม ถ้าจะรักษาตำแหน่งต้องแลกหมากใหญ่
> *ผลลัพธ์:* ช่องว่างที่เกิดจากการ “เปลี่ยนเสา” จะกลายเป็นกับดักให้คุณโจมตี
---
## 3. ปลายเกม – หนีบกินจากจุดสลับ
* **ปิดช่องทางหนีรอบกลุ่มหมากอ่อน** ที่คุณล่อให้ศัตรูย้ายมา
* **ใช้รูปแบบหนีบ (capture)** ให้หมากสองข้าง (จากหน้า–หลัง หรือซ้าย–ขวา) ประกบรอบ เพื่อกินหมากสำคัญ
* **เก็บกินเด็ดขาด** ในเทิร์น

In [287]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ลักขื่อเปลี่ยนเสา
Category:    กลยุทธ์ร่วมรบ
Description: สลับตำแหน่งสำคัญของศัตรู

Metrics:
  Semantic Similarity:    0.5899
  6C Similarity:          0.9882
  Selection Score:        0.7492
  Confidence Score:       0.7571
  KL Divergence:          0.0128

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1974,0.2035,0.9701
ความสามารถในการรับ,0.1982,0.1847,0.9318
การประสานงานระหว่างหมาก,0.1587,0.2046,0.7756
ทรัพยากรที่ยังไม่ใช้,0.1421,0.1569,0.9059
ความได้เปรียบด้านเวลา,0.1541,0.1353,0.8782
ความเหมาะสมกับสถานการณ์,0.1496,0.1151,0.7695


# ชี้ต้นหม่อนด่าต้นไหว

In [288]:
# Cell 18: Example Usage
test_query =  """
สร้างสถานการณ์กดดันหรือคุกคามกับหมากหรือพื้นที่ที่ใกล้เคียงกับเป้าหมายจริงแต่ไม่ใช่ตัวเป้าหมายโดยตรง** เพื่อเป็นการส่งสัญญาณเตือนหรือข่มขู่ ทำให้เป้าหมายที่แท้จริงต้องขยับตัว, เสียขวัญ, หรือเปิดเผยจุดอ่อน จากนั้นจึง**ฉวยโอกาสเข้า "หนีบ" โจมตีเป้าหมายหลัก**
เน้นที่การ **ข่มขู่ทางอ้อม** กับหมาก/พื้นที่รอง เพื่อ **กระตุ้นหรือบีบให้เป้าหมายจริงต้องเคลื่อนไหวหรือเปิดเผยจุดอ่อน** โดยไม่จำเป็นต้องโจมตีเป้าหมายรองอย่างจริงจัง
### 1. **ช่วงเริ่มต้น: "ชี้ต้นหม่อน" – สร้างภัยคุกคามทางอ้อม**
* **วิธีการเดินหมาก:** ในช่วงต้นเกม ให้คุณ**มุ่งความสนใจไปที่หมากหรือพื้นที่ที่ไม่ใช่เป้าหมายหลักของคุณ แต่มีความสำคัญหรือเกี่ยวข้องกับเป้าหมายจริงของศัตรู** อาจเป็นการวางหมากในตำแหน่งที่คุกคามหมากรอง, สร้างแรงกดดันในแนวที่ไม่ใช่แนวบุกตรงๆ, หรือแม้แต่การแลกหมากเล็กน้อยเพื่อแสดงความแข็งกร้าวในบริเวณใกล้เคียง
* **วัตถุประสงค์:** เพื่อส่งสัญญาณเตือนและสร้างความไม่สบายใจให้แก่เป้าหมายหลักโดยไม่จำเป็นต้องเข้าโจมตีโดยตรงในทันที
### 2. **ช่วงกลางเกม: "ด่าต้นไหว" – กดดันทางอ้อมให้เป้าหมายหลักเคลื่อนไหว**
* **วิธีการเดินหมาก:** เมื่อคุณได้ "ชี้" ไปที่ "ต้นหม่อน" แล้ว ให้คุณ**คงแรงกดดันหรือเพิ่มความคุกคามในบริเวณนั้น** เพื่อบีบให้ "ต้นไหว" (เป้าหมายจริง) ต้องตัดสินใจหรือตอบสนอง การเคลื่อนไหวของคุณจะทำให้ศัตรูต้องพิจารณาถึงความเสี่ยงที่หมากสำคัญจะถูกคุกคามทางอ้อม ทำให้พวกเขาอาจ:
    * **เคลื่อนหมากสำคัญออกนอกตำแหน่ง:** เผยให้เห็นช่องโหว่
    * **ส่งหมากมาช่วย:** ทำให้กำลังกระจายตัว
    * **เสียสมาธิ:** มองไม่เห็นภัยคุกคามที่แท้จริง
* **วัตถุประสงค์:** เพื่อบีบให้เป้าหมายหลักของศัตรูต้องทำในสิ่งที่คุณต้องการ ไม่ว่าจะเป็นการเปิดช่องโหว่ การแยกกำลัง หรือการเสียเวลา
### 3. **ช่วงปลายเกม: "หนีบ" เป้าหมายจริง – โจมตีในขณะที่ศัตรูสับสน/เสียรูป**
* **วิธีการเดินหมาก:** เมื่อเป้าหมายหลัก (ต้นไหว) ตอบสนองต่อการกดดันทางอ้อมของคุณ ไม่ว่าจะด้วยการเคลื่อนที่, การส่งกำลังเสริม, หรือการเสียขบวน **ให้คุณเปิดฉากโจมตีอย่างรวดเร็วและเด็ดขาดไปยังเป้าหมายจริง** ใช้การประสานงานของหมากเข้า "หนีบ" กินหมากสำคัญของศัตรู หรือยึดพื้นที่หลักที่ถูกเปิดเผยออกมาจากความลังเลหรือการตอบสนองต่อการคุกคามทางอ้อม
* **วัตถุประสงค์:** เพื่อใช้ประโยชน์จากผลลัพธ์ของการข่มขู่ทางอ้อม (ไม่ว่าจะในรูปแบบของการเคลื่อนไหวที่ผิดพลาด หรือการเสียสมดุล) เพื่อให้ได้มาซึ่งชัยชนะเหนือเป้าหมายหลักอย่างมีประสิทธิภาพ
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
สร้างสถานการณ์กดดันหรือคุกคามกับหมากหรือพื้นที่ที่ใกล้เคียงกับเป้าหมายจริงแต่ไม่ใช่ตัวเป้าหมายโดยตรง** เพื่อเป็นการส่งสัญญาณเตือนหรือข่มขู่ ทำให้เป้าหมายที่แท้จริงต้องขยับตัว, เสียขวัญ, หรือเปิดเผยจุดอ่อน จากนั้นจึง**ฉวยโอกาสเข้า "หนีบ" โจมตีเป้าหมายหลัก**
เน้นที่การ **ข่มขู่ทางอ้อม** กับหมาก/พื้นที่รอง เพื่อ **กระตุ้นหรือบีบให้เป้าหมายจริงต้องเคลื่อนไหวหรือเปิดเผยจุดอ่อน** โดยไม่จำเป็นต้องโจมตีเป้าหมายรองอย่างจริงจัง
### 1. **ช่วงเริ่มต้น: "ชี้ต้นหม่อน" – สร้างภัยคุกคามทางอ้อม**
* **วิธีการเดินหมาก:** ในช่วงต้นเกม ให้คุณ**มุ่งความสนใจไปที่หมากหรือพื้นที่ที่ไม่ใช่เป้าหมายหลักของคุณ แต่มีความสำคัญหรือเกี่ยวข้องกับเป้าหมายจริงของศัตรู** อาจเป็นการวางหมากในตำแหน่งที่คุกคามหมากรอง, สร้างแรงกดดันในแนวที่ไม่ใช่แนวบุกตรงๆ, หรือแม้แต่การแลกหมากเล็กน้อยเพื่อแสดงความแข็งกร้าวในบริเวณใกล้เคียง
* **วัตถุประสงค์:** เพื่อส่งสัญญาณเตือนและสร้างความไม่สบายใจให้แก่เป้าหมายหลักโดยไม่จำเป็นต้องเข้าโจมตีโดยตรงในทันที
### 2. **ช่วงกลางเกม: "ด่าต้นไหว" – กดดันทางอ้อมให้เป้าหมายหลักเคลื่อนไหว**
* **ว

In [289]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ชี้ต้นหม่อนด่าต้นไหว
Category:    กลยุทธ์ร่วมรบ
Description: ขู่ให้ฝ่ายอื่นเกรงใจแทนเป้าหมายจริง

Metrics:
  Semantic Similarity:    0.5171
  6C Similarity:          0.9919
  Selection Score:        0.7070
  Confidence Score:       0.7165
  KL Divergence:          0.0083

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1979,0.1936,0.9779
ความสามารถในการรับ,0.1853,0.1755,0.9467
การประสานงานระหว่างหมาก,0.1633,0.2030,0.8047
ทรัพยากรที่ยังไม่ใช้,0.1455,0.1596,0.9112
ความได้เปรียบด้านเวลา,0.1502,0.1376,0.9160
ความเหมาะสมกับสถานการณ์,0.1578,0.1308,0.8292


# แสร้งทำบอแต่ไม่บ้า

In [290]:
# Cell 18: Example Usage
test_query =  """
** ทำเป็นโง่ก่อน แล้วออกหมัดหนัก
---
### 1. ต้นเกม – แสร้งโง่ (ยอมหยิบหมากเล็ก)
* **ยอมแลกหมากตัวเล็ก** ในช่องข้าง ๆ ให้ศัตรูคิดว่านี่คือ “ช่องปลอดภัย”
* **ไม่เติมหมากคุมต่อ** ปล่อยให้ช่องนั้นเป็นอิสระ เหมือนคุณไม่สนใจ
* **ผลลัพธ์:** ศัตรูจะเข้ามายึดช่องนั้นเต็มที่ โดยไม่ระวังกับดัก
---
### 2. กลางเกม – รักษาบทบอ (เก็บข้อมูล)
* **เดินหมากหลอกเล็กน้อย** สลับระหว่างช่องที่แลกมา ให้ดูไร้คาดเดา
* **จงเก็บหมากสำรอง** ไว้หลังแนวหน้า (ไม่รีบปลดออก) รอจังหวะเหมาะ
* **ผลลัพธ์:** คุณเห็นตำแหน่งศัตรูชัดเจน และศัตรูรู้สึกว่า “ยังคุมเกมได้”
---
### 3. ปลายเกม – ออกหมัดหนัก (หนีบกินรวดเดียว)
* **ปลดหมากสำรองมาเสริม** รอบตัวศัตรูที่ติดกับดักในช่องกลาง
* **ประกบสองฝั่งแนวตรง–เฉียง** ปิดทุกทางหนีให้แน่น
* **หนีบ (capture) ทีเดียว** กินหมากที่ถูกล่อมาแบบไม่ให้เหลือช่องหลบ
---
### สรุป
1. **ทำเป็นโง่:** แลกหมากเล็ก เปิดช่องให้ศัตรูเข้า
2. **เก็บข้อมูล:** รักษาบทบอ รอปลดหมากสำรอง
3. **ออกหมัด:** ดึงหมากสำรองมาหนีบกินในเทิร์นเดียวอย่างชัดเจน
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
** ทำเป็นโง่ก่อน แล้วออกหมัดหนัก
---
### 1. ต้นเกม – แสร้งโง่ (ยอมหยิบหมากเล็ก)
* **ยอมแลกหมากตัวเล็ก** ในช่องข้าง ๆ ให้ศัตรูคิดว่านี่คือ “ช่องปลอดภัย”
* **ไม่เติมหมากคุมต่อ** ปล่อยให้ช่องนั้นเป็นอิสระ เหมือนคุณไม่สนใจ
* **ผลลัพธ์:** ศัตรูจะเข้ามายึดช่องนั้นเต็มที่ โดยไม่ระวังกับดัก
---
### 2. กลางเกม – รักษาบทบอ (เก็บข้อมูล)
* **เดินหมากหลอกเล็กน้อย** สลับระหว่างช่องที่แลกมา ให้ดูไร้คาดเดา
* **จงเก็บหมากสำรอง** ไว้หลังแนวหน้า (ไม่รีบปลดออก) รอจังหวะเหมาะ
* **ผลลัพธ์:** คุณเห็นตำแหน่งศัตรูชัดเจน และศัตรูรู้สึกว่า “ยังคุมเกมได้”
---
### 3. ปลายเกม – ออกหมัดหนัก (หนีบกินรวดเดียว)
* **ปลดหมากสำรองมาเสริม** รอบตัวศัตรูที่ติดกับดักในช่องกลาง
* **ประกบสองฝั่งแนวตรง–เฉียง** ปิดทุกทางหนีให้แน่น
* **หนีบ (capture) ทีเดียว** กินหมากที่ถูกล่อมาแบบไม่ให้เหลือช่องหลบ
---
### สรุป
1. **ทำเป็นโง่:** แลกหมากเล็ก เปิดช่องให้ศัตรูเข้า
2. **เก็บข้อมูล:** รักษาบทบอ รอปลดหมากสำรอง
3. **ออกหมัด:** ดึงหมากสำรองมาหนีบกินในเทิร์นเดียวอย่างชัดเจน


1. แสร้งทำบอแต่ไม่บ้า
   Score: 0.7431
   Confidence: 0

In [291]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: แสร้งทำบอแต่ไม่บ้า
Category:    กลยุทธ์ร่วมรบ
Description: ทำเป็นโง่ก่อนแล้วออกหมัดหนัก

Metrics:
  Semantic Similarity:    0.5802
  6C Similarity:          0.9876
  Selection Score:        0.7431
  Confidence Score:       0.7513
  KL Divergence:          0.0133

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2026,0.2102,0.9638
ความสามารถในการรับ,0.1801,0.1672,0.9281
การประสานงานระหว่างหมาก,0.1624,0.2072,0.7835
ทรัพยากรที่ยังไม่ใช้,0.1439,0.1607,0.8957
ความได้เปรียบด้านเวลา,0.1489,0.1303,0.8755
ความเหมาะสมกับสถานการณ์,0.1621,0.1244,0.7672


# ขึ้นบ้านชักบันได

In [292]:
# Cell 18: Example Usage
test_query =  """
---
ค่อยๆ ปิดช่องถอยหลัง แล้วปิดทับด้วยหลังคา จึงทำให้ศัตรูติดกับและถูกหนีบกิน
## 1. ต้นเกม – เปิดประตูหน้าให้ศัตรูเข้ามา
* **ยืดแนวหมากตรงกลาง** ให้ดูเหมือนคุณยังไม่พร้อมบุก เพื่อเชิญชวนให้ฝ่ายตรงข้ามเคลื่อนหมากเข้ามาลึก
* **อย่าเสริมแนวหลังหรือปิดช่องเฉียง** ให้ศัตรูรู้สึกว่า “ยังมีที่ถอย” ข้างหลังเยอะพอ
* **ผลลัพธ์:** ศัตรูจะย้ายหมากเข้ามาในเขตที่คุณควบคุมได้ดี
---
## 2. กลางเกม – ชักบันได (ปิดช่องถอยหลังทีละขั้น)
* **เดินหมากแนวรองขึ้นมา 1–2 ตัว** ปิดช่องถอยหลังหลัก (ด้านหลังแนวหมากศัตรู)
* **สลับเติมหมากปิดช่องเฉียง** ให้แนวหลังเป็น “บันไดขั้นแรก” ที่ศัตรูไม่อาจก้าวลง
* **ผลลัพธ์:** ฝ่ายตรงข้ามถูกฟตัดเส้นทางถอย จึงไม่มีทางถอยให้ชัด
---
## 3. ปลายเกม – ขึ้นบ้าน (หนีบและจับกิน)
* **เสริมหมากขั้นสุดท้าย** รอบตัวศัตรูทั้งซ้าย–ขวา และด้านหน้า ให้กลายเป็น “หลังคาบ้าน” ปิดทับหมด
* **ตรวจสอบว่าศัตรูไม่มีช่องถอย** (เช็คแนวหลังและเฉียง) จึงถือว่า “ขึ้นบ้าน” สมบูรณ์
* **ใช้รูปแบบหนีบ (capture)** ครอบหมากที่ถูกล้อมไว้ แล้วกินหมากในเทิร์นเดียว
---
### สรุปกุญแจ
1. **เปิดทางหน้า:** เชิญให้ศัตรูเข้ามาลึก
2. **ชักบันได:** ปิดช่องถอยหลังทีละขั้นจนไร้ทางถอย
3. **ขึ้นบ้าน:** ล้อมหนีบและจับกินอย่างเด็ดขาด
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
ค่อยๆ ปิดช่องถอยหลัง แล้วปิดทับด้วยหลังคา จึงทำให้ศัตรูติดกับและถูกหนีบกิน
## 1. ต้นเกม – เปิดประตูหน้าให้ศัตรูเข้ามา
* **ยืดแนวหมากตรงกลาง** ให้ดูเหมือนคุณยังไม่พร้อมบุก เพื่อเชิญชวนให้ฝ่ายตรงข้ามเคลื่อนหมากเข้ามาลึก
* **อย่าเสริมแนวหลังหรือปิดช่องเฉียง** ให้ศัตรูรู้สึกว่า “ยังมีที่ถอย” ข้างหลังเยอะพอ
* **ผลลัพธ์:** ศัตรูจะย้ายหมากเข้ามาในเขตที่คุณควบคุมได้ดี
---
## 2. กลางเกม – ชักบันได (ปิดช่องถอยหลังทีละขั้น)
* **เดินหมากแนวรองขึ้นมา 1–2 ตัว** ปิดช่องถอยหลังหลัก (ด้านหลังแนวหมากศัตรู)
* **สลับเติมหมากปิดช่องเฉียง** ให้แนวหลังเป็น “บันไดขั้นแรก” ที่ศัตรูไม่อาจก้าวลง
* **ผลลัพธ์:** ฝ่ายตรงข้ามถูกฟตัดเส้นทางถอย จึงไม่มีทางถอยให้ชัด
---
## 3. ปลายเกม – ขึ้นบ้าน (หนีบและจับกิน)
* **เสริมหมากขั้นสุดท้าย** รอบตัวศัตรูทั้งซ้าย–ขวา และด้านหน้า ให้กลายเป็น “หลังคาบ้าน” ปิดทับหมด
* **ตรวจสอบว่าศัตรูไม่มีช่องถอย** (เช็คแนวหลังและเฉียง) จึงถือว่า “ขึ้นบ้าน” สมบูรณ์
* **ใช้รูปแบบหนีบ (capture)** ครอบหมากที่ถูกล้อมไว้ แล้วกินหมากในเทิร์นเดียว
---
### สรุปกุญแจ
1. **เปิดทางหน้า:** เชิญ

In [293]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ขึ้นบ้านชักบันได
Category:    กลยุทธ์ร่วมรบ
Description: ตัดหนทางถอยหลังเมื่อศัตรูเข้ามา

Metrics:
  Semantic Similarity:    0.5918
  6C Similarity:          0.9934
  Selection Score:        0.7524
  Confidence Score:       0.7605
  KL Divergence:          0.0064

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2047,0.2077,0.9855
ความสามารถในการรับ,0.1959,0.1727,0.8813
การประสานงานระหว่างหมาก,0.1603,0.1982,0.8090
ทรัพยากรที่ยังไม่ใช้,0.1352,0.1398,0.9668
ความได้เปรียบด้านเวลา,0.1434,0.1297,0.9045
ความเหมาะสมกับสถานการณ์,0.1605,0.1520,0.9467


# ต้นไม้ผลิดอก

In [294]:
# Cell 18: Example Usage
test_query =  """
เริ่มต้นจากหมากเล็ก ๆ หรือหน่วยย่อยที่ไม่โดดเด่น** จากนั้นจะ**ค่อย ๆ เสริมกำลังและขยายอิทธิพลของกลุ่มหมากเหล่านั้นอย่างเป็นระบบ** จนกลายเป็นกำลังที่ดูแข็งแกร่งและน่าเกรงขาม เพื่อใช้ในการ "หนีบ" กินหมากศัตรู ซึ่งสอดคล้องกับเสริมกำลังเล็กให้ดูเข้มแข็ง
---
## 1. ต้นเกม – "ปลูกเมล็ด" สร้างฐานเล็ก
* **วิธีการเดินหมาก:** ในช่วงต้นเกม ให้คุณ**วางหมากเล็ก 1-2 ตัวในตำแหน่งที่ดูเหมือนไม่มีอันตรายมากนัก หรือเป็นพื้นที่ริมกระดาน** เพื่อสร้างจุดเริ่มต้น (เหมือนการปลูกเมล็ดต้นไม้) หมากเหล่านี้อาจดูไม่เป็นภัยคุกคามโดยตรง และอาจถูกมองข้ามจากฝ่ายตรงข้าม
* **วัตถุประสงค์:** เพื่อสร้างจุดเริ่มต้นที่ไม่โดดเด่นนัก แต่มีศักยภาพในการขยายตัวในอนาคต
---
## 2. กลางเกม – "แตกแขนงผลิดอก" เสริมกำลังจากภายในสู่ภายนอก
* **วิธีการเดินหมาก:** เมื่อคุณได้ "ปลูกเมล็ด" แล้ว **ให้คุณค่อย ๆ เคลื่อนหมากเพิ่มเติมเข้ามาสนับสนุนกลุ่มหมากเล็กๆ นั้นอย่างต่อเนื่องและเป็นระบบ** อาจเป็นการนำหมากสำรองเข้ามาเสริม, เชื่อมโยงเส้นทางเดินหมากให้แน่นแฟ้นขึ้น, หรือขยายการควบคุมพื้นที่รอบๆ กลุ่มหมากเล็กๆ นั้น เพื่อให้ "ต้นไม้" เริ่มผลิดอกและแผ่ขยายอิทธิพลออกไป
* **เสริมกำลังเล็กให้ดูเข้มแข็ง:** คุณจะ **เสริมกำลัง** หมากย่อยๆ ด้วยการ:
    * **เพิ่มจำนวนหมาก:** ค่อยๆ นำหมากตัวอื่นเข้ามาสมทบกลุ่มเล็กๆ นั้น
    * **สร้างเครือข่าย:** ทำให้หมากแต่ละตัวเชื่อมโยงและสนับสนุนกันและกันอย่างมีประสิทธิภาพ
    * **ขยายอิทธิพล:** แผ่ขยายการควบคุมพื้นที่จากจุดเล็กๆ ให้กว้างขวางขึ้น ทำให้กลุ่มหมากนั้นดูใหญ่โตและทรงพลังมากขึ้น
* **วัตถุประสงค์:** เพื่อเปลี่ยนกลุ่มหมากเล็กๆ ให้กลายเป็นกำลังที่แข็งแกร่งและมั่นคง คอยคุกคามฝ่ายตรงข้ามในแนวหน้าหรือแนวรุกโดยที่ศัตรูไม่ทันรู้ตัวถึงการเติบโตนี้
---
## 3. ปลายเกม – "ดอกสุกงอม" รวมพลังเพื่อหนีบกินอย่างเด็ดขาด
* **วิธีการเดินหมาก:** เมื่อกลุ่มหมากที่คุณได้ "ผลิดอก" ออกมานั้นแข็งแกร่งเต็มที่ และมีอิทธิพลครอบคลุมพื้นที่ที่สำคัญ **ให้คุณระดมพลังทั้งหมดของกลุ่มหมากที่เสริมสร้างขึ้นมานั้นเข้าทำการ "หนีบ" กินหมากสำคัญของศัตรู** ใช้การประสานงานที่สมบูรณ์แบบเพื่อปิดล้อมและโจมตีด้วยกำลังที่เหนือกว่า ซึ่งเป็นผลมาจากการเสริมสร้างกำลังตั้งแต่ต้นเกม
* **วัตถุประสงค์:** เพื่อใช้กำลังที่เสริมสร้างขึ้นมาให้กลายเป็นพลังที่สามารถหนีบกินหมากสำคัญของศัตรูได้อย่างเด็ดขาด
---
### สรุป
* **จาก "เมล็ด" สู่ "ดอก"**: การเริ่มจากหมากเล็ก (เมล็ด) แล้วค่อยๆ เสริมกำลังและขยายอิทธิพล (ผลิดอก) คือหัวใจของกลยุทธ์
* **การเติบโตอย่างเป็นระบบ**: กลยุทธ์นี้เน้นการเพิ่มพลังและอิทธิพลทีละเล็กละน้อยอย่างต่อเนื่อง จนเกิดเป็นกำลังที่แข็งแกร่งในที่สุด
* **เสริมกำลังเล็กให้ดูเข้มแข็ง**: ผลลัพธ์สุดท้ายคือการที่กลุ่มหมากที่เคยดูเล็กน้อยกลายเป็นภัยคุกคามที่สำคัญและสามารถทำการ "หนีบ" ได้อย่างมีประสิทธิภาพ
---
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
เริ่มต้นจากหมากเล็ก ๆ หรือหน่วยย่อยที่ไม่โดดเด่น** จากนั้นจะ**ค่อย ๆ เสริมกำลังและขยายอิทธิพลของกลุ่มหมากเหล่านั้นอย่างเป็นระบบ** จนกลายเป็นกำลังที่ดูแข็งแกร่งและน่าเกรงขาม เพื่อใช้ในการ "หนีบ" กินหมากศัตรู ซึ่งสอดคล้องกับเสริมกำลังเล็กให้ดูเข้มแข็ง
---
## 1. ต้นเกม – "ปลูกเมล็ด" สร้างฐานเล็ก
* **วิธีการเดินหมาก:** ในช่วงต้นเกม ให้คุณ**วางหมากเล็ก 1-2 ตัวในตำแหน่งที่ดูเหมือนไม่มีอันตรายมากนัก หรือเป็นพื้นที่ริมกระดาน** เพื่อสร้างจุดเริ่มต้น (เหมือนการปลูกเมล็ดต้นไม้) หมากเหล่านี้อาจดูไม่เป็นภัยคุกคามโดยตรง และอาจถูกมองข้ามจากฝ่ายตรงข้าม
* **วัตถุประสงค์:** เพื่อสร้างจุดเริ่มต้นที่ไม่โดดเด่นนัก แต่มีศักยภาพในการขยายตัวในอนาคต
---
## 2. กลางเกม – "แตกแขนงผลิดอก" เสริมกำลังจากภายในสู่ภายนอก
* **วิธีการเดินหมาก:** เมื่อคุณได้ "ปลูกเมล็ด" แล้ว **ให้คุณค่อย ๆ เคลื่อนหมากเพิ่มเติมเข้ามาสนับสนุนกลุ่มหมากเล็กๆ นั้นอย่างต่อเนื่องและเป็นระบบ** อาจเป็นการนำหมากสำรองเข้ามาเสริม, เชื่อมโยงเส้นทางเดินหมากให้แน่นแฟ้นขึ้น, หรือขยายการควบคุมพื้นที่รอบๆ กลุ่มหมากเล็กๆ นั้น เพื่อให้ "ต้นไม้" เริ่ม

In [295]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ต้นไม้ผลิดอก
Category:    กลยุทธ์ร่วมรบ
Description: เสริมกำลังเล็กให้ดูเข้มแข็ง

Metrics:
  Semantic Similarity:    0.5221
  6C Similarity:          0.9925
  Selection Score:        0.7103
  Confidence Score:       0.7197
  KL Divergence:          0.0093

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2006,0.1951,0.9728
ความสามารถในการรับ,0.1646,0.1585,0.9627
การประสานงานระหว่างหมาก,0.1804,0.2140,0.8428
ทรัพยากรที่ยังไม่ใช้,0.1753,0.1894,0.9255
ความได้เปรียบด้านเวลา,0.1463,0.1454,0.9941
ความเหมาะสมกับสถานการณ์,0.1329,0.0976,0.7345


# สลับแขกเป็นเจ้าบ้าน

In [296]:
# Cell 18: Example Usage
test_query =  """
เริ่มต้นจากการเป็นฝ่ายรับหรือตั้งรับในพื้นที่ของคุณเอง** จากนั้นจึง**ค่อยๆ ใช้การเคลื่อนไหวที่ชาญฉลาดและประสานงานกัน** เพื่อเข้าควบคุมสถานการณ์ พลิกบทบาทจากผู้ถูกกระทำหรือผู้มาเยือนให้กลายเป็น **"เจ้าบ้าน" ที่คุมเกมและกำหนดทิศทางการรบ** ก่อนจะเข้า "หนีบ" กินหมากศัตรู
---
### 1. **ช่วงต้นเกม: เป็น "แขก" – ตั้งรับและปรับตัว*
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้น ให้คุณ**วางหมากในลักษณะตั้งรับหรือเสริมแนวป้องกันในพื้นที่ของคุณเอง** อาจเป็นการยอมให้ศัตรูเข้ามาในพื้นที่บางส่วน, ดูเหมือนไม่มีแผนการรุกที่ชัดเจน, หรือใช้หมากเพียงน้อยตัวในการเฝ้าระวัง เพื่อให้ศัตรูรู้สึกว่าตัวเองเป็นฝ่าย "เจ้าบ้าน" ที่กำลังบุกเข้ามา
* **วัตถุประสงค์:** เพื่อให้ศัตรูประเมินสถานการณ์ผิดพลาดและบุกเข้ามาในพื้นที่ที่คุณเลือก โดยไม่รู้ว่าคุณกำลังเตรียมการพลิกสถานการณ์
---
### 2. **ช่วงกลางเกม: แย่งชิง "ความเป็นเจ้าบ้าน" – สร้างอิทธิพลและควบคุม**
* **วิธีการเดินหมาก:** เมื่อศัตรูเข้ามาอยู่ในตำแหน่งที่คุณต้องการแล้ว **ให้คุณเริ่มเคลื่อนหมากเข้าครอบครองจุดยุทธศาสตร์สำคัญ หรือสร้างแนวปิดกั้นเส้นทางเดินของศัตรูอย่างเป็นระบบ** ใช้การประสานงานระหว่างหมากเพื่อจำกัดพื้นที่การเคลื่อนที่ของศัตรู ค่อยๆ บีบให้ศัตรูรู้สึกว่าพื้นที่ที่เคยคิดว่าเป็นของตัวเองนั้นกำลังถูกคุณควบคุม และกลายเป็น "แขก" แทน
* **วัตถุประสงค์:** เพื่อพลิกสถานการณ์ให้คุณเป็นฝ่ายที่เริ่มควบคุมพื้นที่และกำหนดทิศทางการเดินหมาก ทำให้ศัตรูตกเป็นฝ่ายที่ถูกบีบและเสียเปรียบ
---
### 3. **ช่วงปลายเกม: เป็น "เจ้าบ้าน" อย่างสมบูรณ์ – "หนีบ" กินอย่างเด็ดขาด**
* **วิธีการเดินหมาก:** เมื่อคุณได้เข้าควบคุมพื้นที่และบีบให้ศัตรูตกอยู่ในสภาพที่เสียเปรียบจนหนีไม่ได้ **ให้คุณเปิดฉากโจมตีด้วยการ "หนีบ" หมากสำคัญของศัตรูอย่างรวดเร็วและเด็ดขาด** ใช้หมากที่คุณวางตำแหน่งไว้เพื่อควบคุมสถานการณ์เข้าประกบกินหมากของฝ่ายตรงข้ามที่ถูกปิดล้อมอย่างสมบูรณ์ในพื้นที่ที่คุณได้กลายเป็น "เจ้าบ้าน"
* **วัตถุประสงค์:** เพื่อใช้ความได้เปรียบจากการพลิกสถานการณ์มาเป็นผู้ควบคุมเกม เข้ายึดหมากสำคัญและนำไปสู่ชัยชนะอย่างเบ็ดเสร็จ
---
### สอดคล้องกับ "กลยุทธ์ร่วมรบ"
* **การประสานงานเพื่อพลิกบทบาท:** กลยุทธ์นี้เป็น "กลยุทธ์ร่วมรบ" อย่างชัดเจน เพราะการเปลี่ยนจาก "แขก" เป็น "เจ้าบ้าน" นั้นต้องอาศัย **การทำงานร่วมกันของหมากหลายตัว** ตั้งแต่การตั้งรับอย่างมีกลยุทธ์ การเคลื่อนเข้าควบคุมพื้นที่ ไปจนถึงการปิดล้อมและหนีบกิน ซึ่งทั้งหมดนี้ต้องอาศัย **การประสานงานที่ไร้ที่ติ** เพื่อให้การเปลี่ยนผ่านบทบาททำได้อย่างราบรื่นและมีประสิทธิภาพ
* **การร่วมมือสร้างอำนาจ:** หมากแต่ละตัวทำหน้าที่ของตัวเองในการสร้างแนวรับ, แนวรุก, หรือแนวควบคุม ซึ่งเมื่อรวมกันแล้วจะสร้างพลังอำนาจที่เหนือกว่า ทำให้คุณสามารถ "ครอบงำ" สนามรบได้ เหมือนเจ้าบ้านที่ควบคุมทุกอย่างในบ้านของตัวเอง
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
เริ่มต้นจากการเป็นฝ่ายรับหรือตั้งรับในพื้นที่ของคุณเอง** จากนั้นจึง**ค่อยๆ ใช้การเคลื่อนไหวที่ชาญฉลาดและประสานงานกัน** เพื่อเข้าควบคุมสถานการณ์ พลิกบทบาทจากผู้ถูกกระทำหรือผู้มาเยือนให้กลายเป็น **"เจ้าบ้าน" ที่คุมเกมและกำหนดทิศทางการรบ** ก่อนจะเข้า "หนีบ" กินหมากศัตรู
---
### 1. **ช่วงต้นเกม: เป็น "แขก" – ตั้งรับและปรับตัว*
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้น ให้คุณ**วางหมากในลักษณะตั้งรับหรือเสริมแนวป้องกันในพื้นที่ของคุณเอง** อาจเป็นการยอมให้ศัตรูเข้ามาในพื้นที่บางส่วน, ดูเหมือนไม่มีแผนการรุกที่ชัดเจน, หรือใช้หมากเพียงน้อยตัวในการเฝ้าระวัง เพื่อให้ศัตรูรู้สึกว่าตัวเองเป็นฝ่าย "เจ้าบ้าน" ที่กำลังบุกเข้ามา
* **วัตถุประสงค์:** เพื่อให้ศัตรูประเมินสถานการณ์ผิดพลาดและบุกเข้ามาในพื้นที่ที่คุณเลือก โดยไม่รู้ว่าคุณกำลังเตรียมการพลิกสถานการณ์
---
### 2. **ช่วงกลางเกม: แย่งชิง "ความเป็นเจ้าบ้าน" – สร้างอิทธิพลและควบคุม**
* **วิธีการเดินหมาก:** เมื่อศัตรูเข้ามาอยู่ในตำแหน่งที่คุณต้องการแล้ว **ให้คุณเริ่มเคลื่อนหมากเข้าครอบครองจุดยุทธศาสตร์สำคัญ หรือสร้างแนวปิดกั้นเส้นทางเดินของศัตรูอย่

In [297]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: สลับแขกเป็นเจ้าบ้าน
Category:    กลยุทธ์ร่วมรบ
Description: เปลี่ยนฝ่ายให้ดูเหมือนเราเป็นเจ้าคุม

Metrics:
  Semantic Similarity:    0.5323
  6C Similarity:          0.9949
  Selection Score:        0.7173
  Confidence Score:       0.7266
  KL Divergence:          0.0049

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2080,0.2121,0.9807
ความสามารถในการรับ,0.1672,0.1484,0.8878
การประสานงานระหว่างหมาก,0.1749,0.2091,0.8365
ทรัพยากรที่ยังไม่ใช้,0.1489,0.1501,0.9919
ความได้เปรียบด้านเวลา,0.1418,0.1303,0.9191
ความเหมาะสมกับสถานการณ์,0.1592,0.1499,0.9416


# สาวงาม

In [298]:
# Cell 18: Example Usage
test_query =  """
---
## 1. ต้นเกม – เสน่ห์ล่อ (โปรยกลยุทธ์)
* **เดินหมากเล็ก ๆ เป็นจังหวะพริ้มพลิ้ว** เช่น ขยับหมากออกจากแถวหลักเล็กน้อย ให้ดูเหมือนไม่ตั้งใจบุก
* **สร้างจุดสนใจด้วยหมากตาเดียว** ให้ศัตรูโฟกัสว่าจุดนั้นจะเป็นเป้าหมาย
* **ผลลัพธ์:** ฝ่ายตรงข้ามตาลาย คิดว่าคุณกำลังอ่อนด้อยจริง ๆ
---
## 2. กลางเกม – รักษาเสน่ห์ (ยืดเวลาให้ศัตรูติดกับ)
* **สลับเดินหมากโปรยเสน่ห์ซ้ำ ๆ** ระหว่างสองจุดเบา ๆ ให้ศัตรูสับสนว่าเป้าหมายหลักอยู่ที่ใด
* **อย่าเติมหมากหนัก** ในจุดใดจุดหนึ่ง ให้ดูเหมือนคุณไม่มีหมากเหลือบุกจริง
* **ผลลัพธ์:** ศัตรูเสียเวลาแก้กับดักเล็กๆ น้อยๆ และไม่ทันระวังแนวหลักของคุณ
---
## 3. ปลายเกม – ทำให้สับสนแล้วซัด (หนีบกิน)
* **ฉวยโอกาสเดินหมากหลักเข้าประสาน** ในจุดที่ศัตรูละเลยเพราะมัวติดเสน่ห์
* **ประกบสองฝั่งแนวตรง–เฉียง** ปิดช่องทางหนีให้หมด
* **หนีบ (capture) พร้อมกัน** กินหมากฝ่ายตรงข้ามในเทิร์นเดียว เมื่อเขาติดกับดัก
---
### สอดคล้องกับ “กลยุทธ์ยามพ่าย”
* เมื่อสถานการณ์ไม่เป็นใจ (พ่าย) เราใช้ **เสน่ห์** ดึงศัตรูให้สบายใจจนประมาท
* **โปรยอ่อนแต่ลับแข็ง** คือการดึงความเชื่อใจ ก่อนหักมุมโจมตี
* กลยุทธ์นี้ช่วยพลิกสถานการณ์ จากแพ้ให้กลายเป็นได้เปรียบในพริบตาครับ
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
## 1. ต้นเกม – เสน่ห์ล่อ (โปรยกลยุทธ์)
* **เดินหมากเล็ก ๆ เป็นจังหวะพริ้มพลิ้ว** เช่น ขยับหมากออกจากแถวหลักเล็กน้อย ให้ดูเหมือนไม่ตั้งใจบุก
* **สร้างจุดสนใจด้วยหมากตาเดียว** ให้ศัตรูโฟกัสว่าจุดนั้นจะเป็นเป้าหมาย
* **ผลลัพธ์:** ฝ่ายตรงข้ามตาลาย คิดว่าคุณกำลังอ่อนด้อยจริง ๆ
---
## 2. กลางเกม – รักษาเสน่ห์ (ยืดเวลาให้ศัตรูติดกับ)
* **สลับเดินหมากโปรยเสน่ห์ซ้ำ ๆ** ระหว่างสองจุดเบา ๆ ให้ศัตรูสับสนว่าเป้าหมายหลักอยู่ที่ใด
* **อย่าเติมหมากหนัก** ในจุดใดจุดหนึ่ง ให้ดูเหมือนคุณไม่มีหมากเหลือบุกจริง
* **ผลลัพธ์:** ศัตรูเสียเวลาแก้กับดักเล็กๆ น้อยๆ และไม่ทันระวังแนวหลักของคุณ
---
## 3. ปลายเกม – ทำให้สับสนแล้วซัด (หนีบกิน)
* **ฉวยโอกาสเดินหมากหลักเข้าประสาน** ในจุดที่ศัตรูละเลยเพราะมัวติดเสน่ห์
* **ประกบสองฝั่งแนวตรง–เฉียง** ปิดช่องทางหนีให้หมด
* **หนีบ (capture) พร้อมกัน** กินหมากฝ่ายตรงข้ามในเทิร์นเดียว เมื่อเขาติดกับดัก
---
### สอดคล้องกับ “กลยุทธ์ยามพ่าย”
* เมื่อสถานการณ์ไม่เป็นใจ (พ่าย) เราใช้ **เสน่ห์** ดึงศัตรูให้สบายใจจนประมาท
* **โปรยอ่อนแต่ลับแข็ง** คือการดึงความเชื่อใจ ก่อน

In [299]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: สาวงาม
Category:    กลยุทธ์ยามพ่าย
Description: ใช้เสน่ห์หลอกใจศัตรูให้สับสน

Metrics:
  Semantic Similarity:    0.5215
  6C Similarity:          0.9931
  Selection Score:        0.7101
  Confidence Score:       0.7196
  KL Divergence:          0.0072

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2040,0.2096,0.9731
ความสามารถในการรับ,0.1877,0.1713,0.9125
การประสานงานระหว่างหมาก,0.1571,0.1886,0.8330
ทรัพยากรที่ยังไม่ใช้,0.1398,0.1565,0.8929
ความได้เปรียบด้านเวลา,0.1640,0.1391,0.8479
ความเหมาะสมกับสถานการณ์,0.1474,0.1349,0.9149


# เปิดเมือง

In [300]:
# Cell 18: Example Usage
test_query =  """
**แสร้งอ่อนแอ → ล่อให้ศัตรูลังเล → โจมตีตอบโต้**
#### 1. **ต้นเกม – เปิดประตูเมือง (แสดงจุดอ่อนปลอม)**
* **จัดวางหมากแบบ "หลวมๆ"** ปล่อยช่องว่างเสี่ยงกลางแนวรับ
* **เดินหมากช้าๆ ไม่รุกคืบ** แสดงท่าทีไม่มั่นใจ
* **วางหมากสำคัญใกล้ขอบกระดาน** ให้ดูเหมือนถอยหนี
#### 2. **กลางเกม – ยั่วยุด้วยความว่างเปล่า (เสริมภาพลวง)**
* **ถอนหมากออกจากจุดยุทธศาสตร์** 2-3 จุดอย่างจงใจ
* **เปิดช่องโหว่ใหญ่บริเวณกลางกระดาน** แต่ซุ่มหมากสำรองใกล้ๆ
* **หลอกล่อให้ศัตรูเห็น "โอกาสทอง"** ที่แท้เป็นกับดัก
#### 3. **ปลายเกม – ปิดล้อมสายฟ้า (โจมตีขณะลังเล)**
* **เมื่อศัตรูลังเลไม่บุกช่องว่าง** → รีบเคลื่อนหมากซุ่มประกบทันที
* **ใช้ช่องว่างเป็นจุดรวมพล** เปลี่ยนจากจุดอ่อนเป็นฐานรุก
* **หนีบกินหมากศัตรูที่เข้ามาใกล้ช่องว่าง** แบบไม่ตั้งตัว
---
### ความสอดคล้อง "กลยุทธ์ยามพ่าย"
- **แปลงสภาพพ่าย → ได้เปรียบ**: ใช้ความอ่อนแอปลอมเป็นเหยื่อล่อ
- **เอาชนะด้วยจิตวิทยา**: บังคับให้ศัตรูสงสัยในโอกาสของตัวเอง
- **รอดพ้นวิกฤตโดยไม่เสียทรัพยากร**: หลีกเลี่ยงการปะทะตรงขณะเสียเปรียบ
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
**แสร้งอ่อนแอ → ล่อให้ศัตรูลังเล → โจมตีตอบโต้**  
#### 1. **ต้นเกม – เปิดประตูเมือง (แสดงจุดอ่อนปลอม)**  
* **จัดวางหมากแบบ "หลวมๆ"** ปล่อยช่องว่างเสี่ยงกลางแนวรับ  
* **เดินหมากช้าๆ ไม่รุกคืบ** แสดงท่าทีไม่มั่นใจ  
* **วางหมากสำคัญใกล้ขอบกระดาน** ให้ดูเหมือนถอยหนี  
#### 2. **กลางเกม – ยั่วยุด้วยความว่างเปล่า (เสริมภาพลวง)**  
* **ถอนหมากออกจากจุดยุทธศาสตร์** 2-3 จุดอย่างจงใจ  
* **เปิดช่องโหว่ใหญ่บริเวณกลางกระดาน** แต่ซุ่มหมากสำรองใกล้ๆ  
* **หลอกล่อให้ศัตรูเห็น "โอกาสทอง"** ที่แท้เป็นกับดัก  
#### 3. **ปลายเกม – ปิดล้อมสายฟ้า (โจมตีขณะลังเล)**  
* **เมื่อศัตรูลังเลไม่บุกช่องว่าง** → รีบเคลื่อนหมากซุ่มประกบทันที  
* **ใช้ช่องว่างเป็นจุดรวมพล** เปลี่ยนจากจุดอ่อนเป็นฐานรุก  
* **หนีบกินหมากศัตรูที่เข้ามาใกล้ช่องว่าง** แบบไม่ตั้งตัว  
---
### ความสอดคล้อง "กลยุทธ์ยามพ่าย"  
- **แปลงสภาพพ่าย → ได้เปรียบ**: ใช้ความอ่อนแอปลอมเป็นเหยื่อล่อ  
- **เอาชนะด้วยจิตวิทยา**: บังคับให้ศัตรูสงสัยในโอกาสของตัวเอง  
- **รอดพ้นวิกฤตโดยไม่เสียทรัพยากร**: หลีกเลี่ยงการปะทะตรงขณะเสียเปรียบ


1. เป

In [301]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: เปิดเมือง
Category:    กลยุทธ์ยามพ่าย
Description: แสร้งอ่อนแอให้ศัตรูลังเลก่อนบุก

Metrics:
  Semantic Similarity:    0.5328
  6C Similarity:          0.9953
  Selection Score:        0.7178
  Confidence Score:       0.7271
  KL Divergence:          0.0050

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1904,0.2051,0.9284
ความสามารถในการรับ,0.1883,0.1813,0.9631
การประสานงานระหว่างหมาก,0.1503,0.1746,0.8610
ทรัพยากรที่ยังไม่ใช้,0.1536,0.1583,0.9707
ความได้เปรียบด้านเวลา,0.1620,0.1478,0.9124
ความเหมาะสมกับสถานการณ์,0.1554,0.1329,0.8555


# ไส้ศึก

In [302]:
# Cell 18: Example Usage
test_query =  """
---
## 1. ต้นเกม – แทรกหมาก “คนใน”
* **เลื่อนหมากคุณเข้าไปในแนวหมากศัตรู** คล้ายกับหมากฝ่ายตรงข้าม เพื่อไม่ให้โดดเด่น
* **อย่าเดินรุกหนัก** ให้มันดูเหมือนหมากป้องกันตัวหนึ่งของศัตรูเอง
* **ผลลัพธ์:** ฝ่ายตรงข้ามสับสนว่าไอ้หมากนั้นเป็นหมากเราหรือหมากเขา เกิดรอยรั่วในแถว
---
## 2. กลางเกม – สร้างความแตกแยก
* **ใช้หมาก “คนใน” ขวางทางเชื่อมต่อ** ระหว่างหมากศัตรูสองฝั่ง ให้เกิดช่องว่าง
* **ประสานหมากของคุณรอบ ๆ ช่องว่างนั้น** ให้เหมือนเป็นการเสริมแนวป้องกัน
* **ผลลัพธ์:** แถวของฝ่ายตรงข้ามถูกแบ่งออกเป็นส่วนย่อย ๆ ที่แยกกันทำงาน
---
## 3. ปลายเกม – หนีบกินหมากแตกแถว
* **ปิดช่องทางหนีรอบกองหมากที่ถูกแยก** โดยเดินหมากของคุณประกบสองฝั่งทั้งตรง–เฉียง
* **ใช้รูปแบบหนีบ (capture)** กินหมากส่วนที่ถูกแบ่งแยกออกมา โดยไม่ให้พึ่งพิงส่วนอื่นได้
* **ผลลัพธ์:** ฝ่ายตรงข้ามที่ถูก “ไส้ศึก” ตามหลอกล่อ จะถูกล้อมหนีบกินในเทิร์นเดียว
---
### สอดคล้องกับ “กลยุทธ์ยามพ่าย”
* เมื่อสูญเสียความได้เปรียบ ให้ **ใช้หมากตัวเองหลอกเป็นของศัตรู (คนใน)** สร้างรอยรั่ว
* ใช้ **ความสับสนภายในแถว** ดึงให้ศัตรูแตกแยก ก่อนจะ **ล้อมหนีบกิน** อย่างเด็ดขาดครับ
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
---
## 1. ต้นเกม – แทรกหมาก “คนใน”
* **เลื่อนหมากคุณเข้าไปในแนวหมากศัตรู** คล้ายกับหมากฝ่ายตรงข้าม เพื่อไม่ให้โดดเด่น
* **อย่าเดินรุกหนัก** ให้มันดูเหมือนหมากป้องกันตัวหนึ่งของศัตรูเอง
* **ผลลัพธ์:** ฝ่ายตรงข้ามสับสนว่าไอ้หมากนั้นเป็นหมากเราหรือหมากเขา เกิดรอยรั่วในแถว
---
## 2. กลางเกม – สร้างความแตกแยก
* **ใช้หมาก “คนใน” ขวางทางเชื่อมต่อ** ระหว่างหมากศัตรูสองฝั่ง ให้เกิดช่องว่าง
* **ประสานหมากของคุณรอบ ๆ ช่องว่างนั้น** ให้เหมือนเป็นการเสริมแนวป้องกัน
* **ผลลัพธ์:** แถวของฝ่ายตรงข้ามถูกแบ่งออกเป็นส่วนย่อย ๆ ที่แยกกันทำงาน
---
## 3. ปลายเกม – หนีบกินหมากแตกแถว
* **ปิดช่องทางหนีรอบกองหมากที่ถูกแยก** โดยเดินหมากของคุณประกบสองฝั่งทั้งตรง–เฉียง
* **ใช้รูปแบบหนีบ (capture)** กินหมากส่วนที่ถูกแบ่งแยกออกมา โดยไม่ให้พึ่งพิงส่วนอื่นได้
* **ผลลัพธ์:** ฝ่ายตรงข้ามที่ถูก “ไส้ศึก” ตามหลอกล่อ จะถูกล้อมหนีบกินในเทิร์นเดียว
---
### สอดคล้องกับ “กลยุทธ์ยามพ่าย”
* เมื่อสูญเสียความได้เปรียบ ให้ **ใช้หมากตัวเองหลอกเป็นของศัตรู (คนใน)** สร้างรอยรั่ว
* ใช้ **ความสับสนภายในแถว** ดึงให้ศัตรูแตกแยก ก่อ

In [303]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ไส้ศึก
Category:    กลยุทธ์ยามพ่าย
Description: ใช้คนในกองทัพศัตรูสร้างความแตกแยก

Metrics:
  Semantic Similarity:    0.4991
  6C Similarity:          0.9943
  Selection Score:        0.6972
  Confidence Score:       0.7071
  KL Divergence:          0.0063

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2141,0.2068,0.9657
ความสามารถในการรับ,0.1722,0.1747,0.9861
การประสานงานระหว่างหมาก,0.1835,0.2138,0.8585
ทรัพยากรที่ยังไม่ใช้,0.1538,0.1684,0.9132
ความได้เปรียบด้านเวลา,0.1345,0.1173,0.8717
ความเหมาะสมกับสถานการณ์,0.1418,0.1191,0.8399


# ทุกข์กาย

In [304]:
# Cell 18: Example Usage
test_query =  """
จงใจทำให้หมากของคุณดูอ่อนแอ ได้รับความเสียหาย หรือตกอยู่ในสถานการณ์ที่เสียเปรียบอย่างเห็นได้ชัด** เพื่อล่อให้ศัตรูเกิดความประมาท วางใจ และเข้ามาใกล้คุณมากขึ้น จากนั้นจึงใช้โอกาสที่ศัตรูเข้าประชิดตัว **เปิดฉาก "หนีบ" กินหมากอย่างรุนแรงและไม่คาดฝัน**
---
### 1. **ช่วงต้นเกม: "แสร้งบาดเจ็บ" – สร้างภาพความอ่อนแอ**
* **วิธีการเดินหมาก:** ในช่วงต้นเกม คุณจะ**จงใจเดินหมากที่ดูเหมือนผิดพลาด ยอมเสียเปรียบเล็กน้อย หรือปล่อยให้หมากบางตัวตกอยู่ในตำแหน่งที่ดูเหมือนถูกคุกคามง่าย ๆ** โดยเฉพาะหมากที่อาจเป็น "เหยื่อล่อ" ซึ่งทำให้ศัตรูคิดว่าคุณกำลังตกที่นั่งลำบากหรือไม่มีพิษสง
* **วัตถุประสงค์:** เพื่อสร้างความมั่นใจที่ผิดพลาดให้กับศัตรู ทำให้พวกเขาคิดว่าคุณอ่อนแอจริง ๆ และกล้าที่จะบุกเข้ามาใกล้เพื่อฉวยโอกาส
---
### 2. **ช่วงกลางเกม: "เจ็บเพื่อล่อ" – ชักนำศัตรูเข้าใกล้**
* **วิธีการเดินหมาก:** เมื่อศัตรูเห็น "ความอ่อนแอ" ของคุณและเริ่มประมาท **ให้คุณคงท่าทีที่ดูเหมือนอ่อนแอต่อไป** อาจเป็นการถอยหมากที่ดูเหมือนกำลังหนี หรือไม่ตอบโต้การคุกคามเล็ก ๆ น้อย ๆ อย่างรุนแรง สิ่งนี้จะกระตุ้นให้ศัตรูยิ่งวางใจและเดินหมากเข้ามาในระยะที่อันตราย โดยไม่รู้ตัวว่ากำลังก้าวเข้าสู่กับดัก
* **วัตถุประสงค์:** เพื่อล่อให้หมากสำคัญของศัตรูเข้ามาอยู่ในตำแหน่งที่คุณต้องการสำหรับการ "หนีบ" โดยอาศัยความประมาทที่เกิดจากการเห็นคุณ "ทุกข์กาย"
---
### 3. **ช่วงปลายเกม: "เจ็บแต่ยังไหว" – "หนีบ" กินอย่างเด็ดขาด**
* **วิธีการเดินหมาก:** เมื่อศัตรูวางใจและเดินหมากเข้ามาจนอยู่ในระยะประชิดตัวที่คุณเตรียมไว้แล้ว **ให้คุณเปิดเผยพลังที่ซ่อนอยู่และเปิดฉากโจมตีด้วยการ "หนีบ" อย่างรวดเร็ว รุนแรง และไม่ทันตั้งตัว** ใช้หมากที่คุณดูเหมือนจะยอมทิ้ง หรือหมากที่ซุ่มซ่อนไว้เข้าปิดล้อมและกินหมากสำคัญของศัตรูที่หลงเข้ามาอย่างมั่นใจเกินไป
* **วัตถุประสงค์:** เพื่อใช้ประโยชน์จากความประมาทและความไว้ใจที่ศัตรูมีต่อ "ความอ่อนแอ" ของคุณในการหนีบกินหมากสำคัญและพลิกสถานการณ์ให้เป็นฝ่ายได้เปรียบอย่างเบ็ดเสร็จ
---
### สอดคล้องกับ "กลยุทธ์ยามพ่าย"
กลยุทธ์ **ทุกข์กาย** ถูกจัดอยู่ในหมวด **กลยุทธ์ยามพ่าย** ไม่ได้หมายความว่าคุณกำลังจะแพ้จริง ๆ แต่เป็นกลยุทธ์ที่ใช้ **ในสถานการณ์ที่ดูเหมือนคุณกำลังเสียเปรียบ หรือกำลังเผชิญกับแรงกดดัน** โดยมีลักษณะสำคัญดังนี้:
* **การพลิกสถานการณ์ด้วยการหลอกลวง:** แม้จะอยู่ในสภาพที่ดูอ่อนแอหรือใกล้จะพ่ายแพ้ กลยุทธ์นี้คือการใช้ "ความอ่อนแอจอมปลอม" นั้นเป็นเครื่องมืออันชาญฉลาดในการหลอกล่อและดึงศัตรูเข้ามาใกล้ เพื่อสร้างโอกาสในการโต้กลับ
* **ใช้ความไว้ใจของศัตรูเป็นกับดัก:** หัวใจสำคัญคือการทำให้ศัตรูวางใจและประมาทในสถานการณ์ที่พวกเขาเห็นว่าคุณ "บาดเจ็บ" ซึ่งเป็นโอกาสให้คุณสามารถ "หนีบ" กลับได้อย่างเหนือความคาดหมาย
* **การตั้งรับที่แฝงด้วยการรุก:** เป็นการตั้งรับที่ดูเหมือนยอมแพ้หรือเสียหาย แต่แท้จริงแล้วเป็นการเตรียมการเพื่อ "รุกกลับ" โดยการชักนำศัตรูให้เข้ามาในพื้นที่ที่คุณกำหนดไว้ เพื่อจัดการพวกเขาในภายหลัง
กลยุทธ์นี้แสดงให้เห็นว่าแม้ในยามที่ดูเหมือนกำลังจะแพ้ หรืออยู่ในสถานการณ์ที่ยากลำบาก ก็ยังสามารถใช้ปัญญาและความแยบยลในการพลิกสถานการณ์และนำไปสู่ชัยชนะได้
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
จงใจทำให้หมากของคุณดูอ่อนแอ ได้รับความเสียหาย หรือตกอยู่ในสถานการณ์ที่เสียเปรียบอย่างเห็นได้ชัด** เพื่อล่อให้ศัตรูเกิดความประมาท วางใจ และเข้ามาใกล้คุณมากขึ้น จากนั้นจึงใช้โอกาสที่ศัตรูเข้าประชิดตัว **เปิดฉาก "หนีบ" กินหมากอย่างรุนแรงและไม่คาดฝัน**
---
### 1. **ช่วงต้นเกม: "แสร้งบาดเจ็บ" – สร้างภาพความอ่อนแอ**
* **วิธีการเดินหมาก:** ในช่วงต้นเกม คุณจะ**จงใจเดินหมากที่ดูเหมือนผิดพลาด ยอมเสียเปรียบเล็กน้อย หรือปล่อยให้หมากบางตัวตกอยู่ในตำแหน่งที่ดูเหมือนถูกคุกคามง่าย ๆ** โดยเฉพาะหมากที่อาจเป็น "เหยื่อล่อ" ซึ่งทำให้ศัตรูคิดว่าคุณกำลังตกที่นั่งลำบากหรือไม่มีพิษสง
* **วัตถุประสงค์:** เพื่อสร้างความมั่นใจที่ผิดพลาดให้กับศัตรู ทำให้พวกเขาคิดว่าคุณอ่อนแอจริง ๆ และกล้าที่จะบุกเข้ามาใกล้เพื่อฉวยโอกาส
---
### 2. **ช่วงกลางเกม: "เจ็บเพื่อล่อ" – ชักนำศัตรูเข้าใกล้**
* **วิธีการเดินหมาก:** เมื่อศัตรูเห็น "ความอ่อนแอ" ของคุณและเริ่มประมาท **ให้คุณคงท่าทีที่ดูเหมือนอ่อนแอต่อไป** อาจเป็นการถอยหมากที่ดูเหมือนกำลังหนี หรือไม่ตอบโต้การคุกคามเล็ก ๆ น้อย ๆ อย่างรุนแรง สิ่งนี้จะกระตุ้นให้ศัตรูยิ่งวาง

In [305]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ทุกข์กาย
Category:    กลยุทธ์ยามพ่าย
Description: ทำตัวเจ็บเพื่อเรียกให้ศัตรูไว้ใจ

Metrics:
  Semantic Similarity:    0.5466
  6C Similarity:          0.9965
  Selection Score:        0.7265
  Confidence Score:       0.7355
  KL Divergence:          0.0034

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.2086,0.1938,0.9290
ความสามารถในการรับ,0.1694,0.1761,0.9618
การประสานงานระหว่างหมาก,0.1618,0.1871,0.8645
ทรัพยากรที่ยังไม่ใช้,0.1507,0.1508,0.9996
ความได้เปรียบด้านเวลา,0.1526,0.1517,0.9944
ความเหมาะสมกับสถานการณ์,0.1569,0.1405,0.8951


# ลูกโซ่

In [306]:
# Cell 18: Example Usage
test_query =  """
* ผูกแผนหลายขั้นให้ศัตรูต้องตอบโต้เป็นทอด ๆ ตามไม่ทัน จนสุดท้ายติดกับดัก
---
## 1. ต้นเกม – จุดเชื่อมโซ่
* **แลกหมากตัวเล็กในจุด A** เปิดประตูให้ศัตรูเดินมาแก้ไขช่องว่าง
* **รอให้เขาตอบโต้** ด้วยการปิดช่อง A ก่อน เหมือนติดโซ่ข้อแรกที่ดึงเขาเข้ามา
* **สอดคล้องยามพ่าย:** เมื่อกำลังเสียเปรียบ ต้องใช้ลูกโซ่ข้อแรกล่อให้ศัตรูหลงทาง
---
## 2. กลางเกม – ต่อโซ่
* **เดินหมากข้อสองที่ B** ทันทีที่เขาปิด A ให้ดูเหมือนคุณบกพร่อง ที่จริงเป็นกับดักข้อสอง
* **สลับจังหวะให้เขาต้องตามแก้ทั้ง A–B** ทุกครั้งที่ตอบโต้ จะไปปล่อยช่องใหม่ให้คุณล่อต่อ
* **สอดคล้องยามพ่าย:** ผสานหลายขั้นตอนจนศัตรูตามแก้ไม่ทัน กลายเป็นโดนสับสนอยู่ในลูกโซ่
---
## 3. ปลายเกม – ดับโซ่
* **ปิดช่องสุดท้าย C** ที่เกิดขึ้นจากขั้นก่อนหน้า เมื่อเขาติดโซ่จนไม่เหลือช่องหลบ
* **ประกบหมากสองฝั่งหนีบกินพร้อมกัน** ให้ศัตรูถูกดึงเข้ามาในวงล้อมจนไร้ทางออก
* **สอดคล้องยามพ่าย:** ศัตรูที่เสียเปรียบถูกลากผ่านลูกโซ่หลายทอด พลาดท่าถูกจับกินในเทิร์นเดียว
---
### สรุปการสอดคล้องกับกลยุทธ์ยามพ่าย
* **ใช้ข้อแลกเปลี่ยน–ตอบโต้เป็นลูกโซ่ ดึงศัตรูผ่านกับดักหลายทอด
* **พื้นที่อ่อนแอคือเหยื่อ:** เมื่อกำลังพ่าย เปิดช่องให้ศัตรูไล่ตามแต่ละข้อจนล้าหลัง
* **การหนีบสำเร็จ:** สุดท้าย ปล่อยกับดักขั้นสุดท้าย ล้อมกินหมากฝ่ายตรงข้ามแบบเด็ดขาดครับ
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
* ผูกแผนหลายขั้นให้ศัตรูต้องตอบโต้เป็นทอด ๆ ตามไม่ทัน จนสุดท้ายติดกับดัก
---
## 1. ต้นเกม – จุดเชื่อมโซ่ 
* **แลกหมากตัวเล็กในจุด A** เปิดประตูให้ศัตรูเดินมาแก้ไขช่องว่าง
* **รอให้เขาตอบโต้** ด้วยการปิดช่อง A ก่อน เหมือนติดโซ่ข้อแรกที่ดึงเขาเข้ามา
* **สอดคล้องยามพ่าย:** เมื่อกำลังเสียเปรียบ ต้องใช้ลูกโซ่ข้อแรกล่อให้ศัตรูหลงทาง
---
## 2. กลางเกม – ต่อโซ่ 
* **เดินหมากข้อสองที่ B** ทันทีที่เขาปิด A ให้ดูเหมือนคุณบกพร่อง ที่จริงเป็นกับดักข้อสอง
* **สลับจังหวะให้เขาต้องตามแก้ทั้ง A–B** ทุกครั้งที่ตอบโต้ จะไปปล่อยช่องใหม่ให้คุณล่อต่อ
* **สอดคล้องยามพ่าย:** ผสานหลายขั้นตอนจนศัตรูตามแก้ไม่ทัน กลายเป็นโดนสับสนอยู่ในลูกโซ่
---
## 3. ปลายเกม – ดับโซ่ 
* **ปิดช่องสุดท้าย C** ที่เกิดขึ้นจากขั้นก่อนหน้า เมื่อเขาติดโซ่จนไม่เหลือช่องหลบ
* **ประกบหมากสองฝั่งหนีบกินพร้อมกัน** ให้ศัตรูถูกดึงเข้ามาในวงล้อมจนไร้ทางออก
* **สอดคล้องยามพ่าย:** ศัตรูที่เสียเปรียบถูกลากผ่านลูกโซ่หลายทอด พลาดท่าถูกจับกินในเทิร์นเดียว
---
### สรุปการสอดคล้องกับกลยุทธ์ยามพ่าย
* **ใช้ข้อแลกเปลี่ยน–ตอบโต้เป็นลูกโซ่ ดึงศัตรู

In [307]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: ลูกโซ่
Category:    กลยุทธ์ยามพ่าย
Description: ผูกแผนหลายขั้นให้ศัตรูตามไม่ทัน

Metrics:
  Semantic Similarity:    0.5816
  6C Similarity:          0.9978
  Selection Score:        0.7481
  Confidence Score:       0.7564
  KL Divergence:          0.0025

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1954,0.2007,0.9731
ความสามารถในการรับ,0.1831,0.1801,0.9833
การประสานงานระหว่างหมาก,0.1845,0.1974,0.9345
ทรัพยากรที่ยังไม่ใช้,0.1321,0.1379,0.9573
ความได้เปรียบด้านเวลา,0.1526,0.1540,0.9905
ความเหมาะสมกับสถานการณ์,0.1524,0.1298,0.8516


# หลบหนี

In [308]:
# Cell 18: Example Usage
test_query =  """
แก่นของกลยุทธ์ คือ **"การถอยเพื่อรอเวลาที่ดีกว่า"** ซึ่งไม่ใช่แค่การล่อให้ประมาท หรือการซ่อนแผน แต่เป็นการ **ตัดสินใจเชิงรุกที่จะถอนตัวจากสถานการณ์ที่เสียเปรียบอย่างเห็นได้ชัด** เพื่อ **รักษาทรัพยากร (หมาก)** และ **สร้างจังหวะที่เหนือกว่า** ในการพลิกกลับมาโจมตีอย่างเด็ดขาด
---
*จงใจถอยหมากหรือหลีกเลี่ยงการเผชิญหน้าโดยตรงอย่างมีแบบแผน** เมื่อตกอยู่ในสถานการณ์ที่เสียเปรียบอย่างไม่อาจชนะได้ เพื่อ**รักษากำลังหลักไว้และรอคอยจังหวะที่เหมาะสมกว่า** จากนั้นเมื่อศัตรูประมาท เคลื่อนพลผิดพลาด หรืออยู่ในตำแหน่งที่เปิดช่องว่าง คุณจะ**พลิกกลับมา "หนีบ" กินหมากอย่างรวดเร็ว เด็ดขาด และเหนือความคาดหมาย**
---
### 1. **ช่วงต้นเกม: "ถอนตัวอย่างมีกลยุทธ์" – เลี่ยงการปะทะที่ไร้ประโยชน์**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้น เมื่อพบว่ากำลังเผชิญหน้ากับสถานการณ์ที่เสียเปรียบอย่างหนัก เช่น หมากของศัตรูอยู่ในตำแหน่งที่ได้เปรียบมากเกินไป หรือมีกำลังเหนือกว่าอย่างเห็นได้ชัด ให้คุณ**จงใจถอยหมากสำคัญออกจากแนวปะทะโดยตรง หรือยอมสละพื้นที่บางส่วนที่ไม่มีความสำคัญในระยะยาว** นี่ไม่ใช่การยอมแพ้ แต่เป็นการถอนตัวอย่างมีแผนเพื่อลดความเสียหายและยืดเวลาออกไป
* **วัตถุประสงค์:** เพื่อ **รักษากำลังรบหลัก** ไว้ไม่ให้ถูกทำลาย และหลีกเลี่ยงการปะทะที่อาจนำไปสู่ความพ่ายแพ้โดยไม่จำเป็น พร้อมทั้งซื้อเวลาเพื่อหาโอกาสพลิกเกมในอนาคต
### 2. **ช่วงกลางเกม: "ซุ่มรอคอยจังหวะ" – เตรียมการในที่ปลอดภัย**
* **วิธีการเดินหมาก:** หลังจากถอยหมากแล้ว ให้คุณ**จัดวางตำแหน่งหมากที่ถอยมาใหม่ในจุดที่ปลอดภัย แต่ยังคงสามารถสร้างแรงกดดัน หรือเข้าถึงเป้าหมายได้ในอนาคต** คุณอาจใช้จังหวะที่ศัตรุกำลังไล่ตามอย่างประมาท หรือกำลังจัดทัพใหม่เพื่อเข้าควบคุมพื้นที่ที่ถูกทิ้งว่างไว้ โดยคุณจะใช้เวลาในการซุ่มกำลัง ฟื้นฟูสถานะ และสังเกตความเคลื่อนไหวของศัตรูอย่างละเอียด เพื่อรอจังหวะ **"เปิดช่องโหว่ที่แท้จริง"**
* **วัตถุประสงค์:** เพื่อ **รักษากำลังหลักไว้ให้สมบูรณ์พร้อมใช้** และรอคอยให้ศัตรูเริ่มประมาท เปิดช่องโหว่ที่สำคัญ หรือเคลื่อนหมากมาอยู่ในตำแหน่งที่คุณสามารถโจมตีกลับได้อย่างมีประสิทธิภาพ
### 3. **ช่วงปลายเกม: "พลิกกลับหนีบแบบไม่ทันตั้งตัว" – โจมตีเมื่อโอกาสมาถึง**
* **วิธีการเดินหมาก:** เมื่อจังหวะมาถึงอย่างเหมาะเจาะ เช่น ศัตรูประมาทเต็มที่ เคลื่อนหมากเข้าสู่กับดักที่คุณวางไว้ หรือกระจายกำลังจนแนวป้องกันอ่อนแออย่างไม่คาดคิด **ให้คุณเปิดฉากโจมตีด้วยการ "หนีบ" อย่างรวดเร็ว รุนแรง และไม่ทันตั้งตัว** ใช้หมากที่คุณเก็บรักษาไว้หรือหมากที่ซุ่มรออยู่เข้าปิดล้อมและกินหมากสำคัญของศัตรูที่คาดไม่ถึงว่าคุณจะกลับมาโจมตีอย่างรุนแรงและมีประสิทธิภาพ
* **วัตถุประสงค์:** เพื่อใช้ **ความได้เปรียบจากจังหวะที่รอคอยมานาน** ในการเข้าหนีบกินหมากสำคัญของศัตรู และพลิกสถานการณ์จากฝ่ายที่เคยเสียเปรียบให้กลายเป็นผู้ควบคุมเกมและได้รับชัยชนะอย่างเด็ดขาด
---
### สอดคล้องกับ "กลยุทธ์ยามพ่าย"
กลยุทธ์ **หลบหนี** จัดอยู่ในหมวด **กลยุทธ์ยามพ่าย** ไม่ได้หมายถึงการยอมแพ้หรือสิ้นหวัง แต่เป็นการใช้ปัญญาและวินัยในการจัดการสถานการณ์ที่ยากลำบาก:
* **การเอาตัวรอดเพื่อสู้ต่ออย่างชาญฉลาด:** แทนที่จะเผชิญหน้าและแพ้อย่างหมดรูป คุณจะเลือกที่จะ **"ถอย" อย่างมีกลยุทธ์** เพื่อรักษากำลังไว้และลดความเสียหาย ซึ่งเป็นสิ่งสำคัญยิ่งในสถานการณ์ที่ดูเหมือนกำลังจะพ่ายแพ้
* **การพลิกสถานการณ์ด้วยความอดทนและวิสัยทัศน์:** กลยุทธ์นี้สอนให้รู้จัก **อดทนรอคอยจังหวะที่เหมาะสมที่สุด** ไม่ใช่การยอมแพ้ แต่เป็นการตั้งรับเชิงกลยุทธ์และการเตรียมพร้อมเพื่อรอโอกาสในการโต้กลับเมื่อสถานการณ์เอื้ออำนวย
* **เปลี่ยนบทบาทจาก "ผู้ถูกไล่ล่า" เป็น "ผู้ล่า" อย่างเฉียบขาด:** ในยามพ่าย คุณจะทำตัวเหมือนผู้ที่กำลังอ่อนแอและถูกไล่ล่า แต่ในความเป็นจริง คุณกำลังวางแผนอย่างเงียบๆ เพื่อพลิกกลับมาเป็นฝ่าย **"หนีบ"** ศัตรูเมื่อพวกเขาประมาท ทำให้สถานการณ์พลิกกลับจากผู้เสียเปรียบเป็นผู้ได้เปรียบในที่สุดด้วยการวางแผนที่เหนือกว่า
"""
test_results, test_metrics = test_strategy_recommendation(test_query)

Query: 
แก่นของกลยุทธ์ คือ **"การถอยเพื่อรอเวลาที่ดีกว่า"** ซึ่งไม่ใช่แค่การล่อให้ประมาท หรือการซ่อนแผน แต่เป็นการ **ตัดสินใจเชิงรุกที่จะถอนตัวจากสถานการณ์ที่เสียเปรียบอย่างเห็นได้ชัด** เพื่อ **รักษาทรัพยากร (หมาก)** และ **สร้างจังหวะที่เหนือกว่า** ในการพลิกกลับมาโจมตีอย่างเด็ดขาด
---
*จงใจถอยหมากหรือหลีกเลี่ยงการเผชิญหน้าโดยตรงอย่างมีแบบแผน** เมื่อตกอยู่ในสถานการณ์ที่เสียเปรียบอย่างไม่อาจชนะได้ เพื่อ**รักษากำลังหลักไว้และรอคอยจังหวะที่เหมาะสมกว่า** จากนั้นเมื่อศัตรูประมาท เคลื่อนพลผิดพลาด หรืออยู่ในตำแหน่งที่เปิดช่องว่าง คุณจะ**พลิกกลับมา "หนีบ" กินหมากอย่างรวดเร็ว เด็ดขาด และเหนือความคาดหมาย**
---
### 1. **ช่วงต้นเกม: "ถอนตัวอย่างมีกลยุทธ์" – เลี่ยงการปะทะที่ไร้ประโยชน์**
* **วิธีการเดินหมาก:** ในช่วงเริ่มต้น เมื่อพบว่ากำลังเผชิญหน้ากับสถานการณ์ที่เสียเปรียบอย่างหนัก เช่น หมากของศัตรูอยู่ในตำแหน่งที่ได้เปรียบมากเกินไป หรือมีกำลังเหนือกว่าอย่างเห็นได้ชัด ให้คุณ**จงใจถอยหมากสำคัญออกจากแนวปะทะโดยตรง หรือยอมสละพื้นที่บางส่วนที่ไม่มีความสำคัญในระยะยาว** นี่ไม่ใช่การยอมแพ้ แต่เป็นการถอนตัว

In [309]:
# Cell 19: Detailed Analysis
if test_results:
    top_strategy = test_results[0][0]
    vector_db = load_vector_database()

    # ดึง metrics ของ top_strategy
    metrics = test_metrics[top_strategy]

    print(f"Detailed Analysis for: {top_strategy}")
    print("="*80)

    # Strategy info
    strategy_info = vector_db['strategies'][top_strategy]
    print(f"Category:    {strategy_info['category']}")
    print(f"Description: {strategy_info['text']}\n")

    # แสดงเมตริกทั้งหมด
    print("Metrics:")
    print(f"  Semantic Similarity:    {metrics['semantic_similarity']:.4f}")
    print(f"  6C Similarity:          {metrics['sixc_similarity']:.4f}")
    print(f"  Selection Score:        {metrics['selection_score']:.4f}")
    print(f"  Confidence Score:       {metrics['confidence_score']:.4f}")
    print(f"  KL Divergence:          {metrics['kl_divergence']:.4f}")
    print()

    # 6C Analysis
    query_6c = compute_6c_vector(test_query)
    strategy_6c = vector_db['patterns'][top_strategy]

    print("-"*70)

    sixc_data = []
    epsilon = 1e-10
    for i, label in enumerate(sixc_labels):
        query_score    = query_6c[i]
        strategy_score = strategy_6c[i]
        if max(query_score, strategy_score) < epsilon:
             match_score = 1.0
        else:
             match_score = min(query_score + epsilon, strategy_score + epsilon) / max(query_score + epsilon, strategy_score + epsilon)
        sixc_data.append([label, query_score, strategy_score, match_score])

    sixc_df = pd.DataFrame(sixc_data,
                           columns=['Parameter', 'Query Score', 'Strategy Score', 'Match Ratio'])

    from IPython.display import display
    display(
        sixc_df.style.format({
            'Query Score':    '{:.4f}',
            'Strategy Score': '{:.4f}',
            'Match Ratio':    '{:.4f}'
        }).hide(axis="index")
    )


Detailed Analysis for: หลบหนี
Category:    กลยุทธ์ยามพ่าย
Description: ถอยเพื่อรอเวลาที่ดีกว่า

Metrics:
  Semantic Similarity:    0.4619
  6C Similarity:          0.9968
  Selection Score:        0.6759
  Confidence Score:       0.6866
  KL Divergence:          0.0032

----------------------------------------------------------------------


Parameter,Query Score,Strategy Score,Match Ratio
ความสามารถในการรุก,0.1996,0.1916,0.9596
ความสามารถในการรับ,0.1867,0.1762,0.9438
การประสานงานระหว่างหมาก,0.1526,0.1608,0.9492
ทรัพยากรที่ยังไม่ใช้,0.1493,0.1497,0.9975
ความได้เปรียบด้านเวลา,0.1586,0.1834,0.8647
ความเหมาะสมกับสถานการณ์,0.1531,0.1383,0.9033
